To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [blog post](https://unsloth.ai/blog/r1-reasoning) for guidance on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [11]:
%%capture
# Normally using pip install unsloth is enough

# Temporarily as of Jan 31st 2025, Colab has some issues with Pytorch
# Using pip install unsloth will take 3 minutes, whilst the below takes <1 minute:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

### Unsloth

In [12]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [13]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [vicgalle](https://huggingface.co/datasets/vicgalle/alpaca-gpt4), which is a version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html) generated from GPT4. You can replace this code section with your own data prep.

In [14]:
from datasets import Dataset
data = [
  {
    "instruction": "Table of contents",
    "input": "",
    "output": "[Getting Started](README.md)\n [Introduction](introduction-to-feebak/README.md)\n   [Dashboard](introduction-to-feebak/dashboard/README.md)\n     [Agent Dashboard (legacy)](introduction-to-feebak/dashboard/agent-dashboard.md)\n     [Survey Activity Timeline](introduction-to-feebak/dashboard/supervisor-dashboard-1-1-1-1-1.md)\n     [Interactions](introduction-to-feebak/dashboard/interactions.md)\n     [Survey Activity Timeline](introduction-to-feebak/dashboard/survey-activity-timeline.md)\n     [Survey Score Summary](introduction-to-feebak/dashboard/survey-score-summary.md)\n     [Survey Response Summary](introduction-to-feebak/dashboard/survey-response-summary.md)\n     [NPS Summary](introduction-to-feebak/dashboard/nps-summary.md)\n     [CSAT Summary](introduction-to-feebak/dashboard/csat-summary.md)\n     [Agent Scoreboard](introduction-to-feebak/dashboard/agent-scoreboard.md)\n     [Agent Dashboard](introduction-to-feebak/dashboard/agent-dashboard-1.md)\n     [Sentiment Score Summary](introduction-to-feebak/dashboard/sentiment-score-summary.md)\n     [Open Feedback Summary](introduction-to-feebak/dashboard/open-feedback-summary.md)\n   [Reports](introduction-to-feebak/report-generation/README.md)\n     [Survey Score Details Report](introduction-to-feebak/report-generation/survey-score-details-1.md)\n     [Survey Answer Details Report](introduction-to-feebak/report-generation/survey-answer-details.md)\n     [Report History](introduction-to-feebak/report-generation/report-history.md)\n     [Scheduled Reports](introduction-to-feebak/report-generation/scheduled-reports.md)\n   [Scoring Category](introduction-to-feebak/scoring-category.md)\n   [Prompts](introduction-to-feebak/prompts.md)\n   [Surveys](introduction-to-feebak/surveys/README.md)\n     [Data Action](introduction-to-feebak/surveys/data-action/README.md)\n       [Authorization](introduction-to-feebak/surveys/data-action/authorization.md)\n       [Performance Monitoring](introduction-to-feebak/surveys/data-action/performance-monitoring.md)\n     [Principal Agent](introduction-to-feebak/surveys/principal-agent.md)\n   [Outbound Campaigns (Agentless)](introduction-to-feebak/outbound-campaigns-agentless.md)\n   [Outbound Campaigns (With Agents)](introduction-to-feebak/outbound-campaigns-with-agents.md)\n   [Individual Outbound Calls](introduction-to-feebak/individual-outbound-calls.md)\n   [Queues](introduction-to-feebak/queues.md)\n   [Survey Assignments](introduction-to-feebak/survey-assignments.md)\n   [Custom Conversation Type](introduction-to-feebak/custom-conversation-type.md)\n   [Audit Logs](introduction-to-feebak/audit-logs.md)\n   [Gamification](introduction-to-feebak/gamification.md)\n [Administration](feebak-administration/README.md)\n   [Division Management](feebak-administration/division-management/README.md)\n     [Divisions overview](feebak-administration/division-management/divisions-overview.md)\n     [How to Integrate Division](feebak-administration/division-management/how-to-integrate-division.md)\n   [User Management](feebak-administration/user-management/README.md)\n     [How to Add Supervisor](feebak-administration/user-management/how-to-add-supervisor.md)\n     [How to Add Agent or User](feebak-administration/user-management/how-to-add-agent-or-user.md)\n     [Sync User Details From Genesys Cloud](feebak-administration/user-management/sync-user-details-from-genesys-cloud.md)\n   [Roles Management](feebak-administration/roles-management.md)\n   [Email Templates](feebak-administration/email-templates.md)\n   [Notification Templates](feebak-administration/notification-templates.md)\n   [Voice Feedback Setup](feebak-administration/voice-feedback-setup.md)\n   [Custom Email Sender](feebak-administration/custom-email-sender/README.md)\n     [Custom Domain](feebak-administration/custom-email-sender/custom-domain.md)\n     [Custom Email](feebak-administration/custom-email-sender/custom-email.md)\n   [IP Whitelisting](feebak-administration/ip-whitelisting.md)\n   [Sentiment Analysis](feebak-administration/sentiment-analysis.md)\n   [Survey Frequency Management](feebak-administration/survey-frequency-management.md)\n [Subscription](feebak-subscription/README.md)\n   [Named Licensing](feebak-subscription/named-licensing.md)\n   [Concurrent Licensing](feebak-subscription/concurrent-licensing.md)\n   [Assigning feebak Licences to users](feebak-subscription/assigning-feebak-licences-to-users.md)\n [Setup & Integration](setup-and-integration/README.md)\n   [Feebak Self Serve Experience](setup-and-integration/feebak-self-serve-experience.md)\n   [Premium Installation](setup-and-integration/premium-installation/README.md)\n     [Integrating the Application](setup-and-integration/premium-installation/integrating-the-application.md)\n     [Integrating Chat Notifications](setup-and-integration/premium-installation/integrating-chat-notifications.md)\n   [Non-Premium Installation](setup-and-integration/non-premium-installation/README.md)\n     [Generating OAuth Tokens](setup-and-integration/non-premium-installation/generating-oauth-tokens.md)\n     [Adding a Group](setup-and-integration/non-premium-installation/adding-a-group.md)\n     [Integrating the Application](setup-and-integration/non-premium-installation/integrating-the-application.md)\n     [Integrating Chat Notifications](setup-and-integration/non-premium-installation/integrating-chat-notifications.md)\n   [IP Whitelisting in Genesys Cloud (Optional)](setup-and-integration/ip-whitelisting-in-genesys-cloud-optional.md)\n   [Konnect Insights Integration](setup-and-integration/konnect-insights-integration.md)\n   [CRM Connectors Integration](setup-and-integration/crm-connectors-integration.md)\n   [Database Encryption](setup-and-integration/database-encryption.md)\n [Offering Surveys](offering-surveys/README.md)\n   [Offer Voice Surveys (Automatic)](offering-surveys/scripts-to-offer-survey-automatic.md)\n   [Offer Voice Surveys (Manual)](offering-surveys/call-transferring-survey.md)\n   [Offer Voice Survey using Set Post-Flow Action](offering-surveys/offer-voice-survey-using-set-post-flow-action.md)\n   [Offer Automated IVR Surveys](offering-surveys/offer-automated-ivr-surveys.md)\n   [Offer Surveys for Facebook Messenger](offering-surveys/offer-surveys-for-facebook-messenger.md)\n   [Offer Surveys for Instagram](offering-surveys/offer-surveys-for-instagram.md)\n   [Offer Surveys for Open Messaging Channels](offering-surveys/offer-surveys-for-open-messaging-channels.md)\n   [Offer Surveys for SMS Channel](offering-surveys/offer-surveys-for-sms-channel.md)\n   [Offer Surveys for Web Messaging Channel](offering-surveys/offer-surveys-for-web-messaging-channel.md)\n   [Offer Surveys for Whatsapp Channel](offering-surveys/offer-surveys-for-whatsapp-channel.md)\n   [Offer Surveys for Email Channel](offering-surveys/offer-surveys-for-email-channel.md)\n   [Offer Chat Surveys for Genesys Widgets (version 2)](offering-surveys/offering-chat-surveys-for-genesys-widgets-version-2.md)\n   [Offer Standalone Surveys](offering-surveys/offer-standalone-surveys.md)\n   [Offer Kiosk Surveys](offering-surveys/offer-kiosk-surveys.md)\n   [Offer Digital Surveys using JS SDK](offering-surveys/offering-digital-surveys-directly.md)\n [Survey Scripts Configuration](survey-scripts-configuration/README.md)\n   [Scripts Configuration for Voice and Web Messaging](survey-scripts-configuration/creating-scripts-manual.md)\n   [Conversational Flow Configuration for Facebook and Instagram](survey-scripts-configuration/conversational-flow-configuration-for-facebook-and-instagram.md)\n   [Script Configuration for Conversational Survey in Web Messaging and WhatsApp](survey-scripts-configuration/script-configuration-for-conversational-survey-in-web-messaging-and-whatsapp.md)\n   [Automated Surveys for Web Messaging](survey-scripts-configuration/automated-surveys-for-web-messaging.md)\n   [Creating Scripts Disconnect (Deprecated)](survey-scripts-configuration/creating-scripts-automatic-transfer.md)\n [Troubleshooting / FAQ](troubleshooting-faq/README.md)\n   [Adding Authorization Code Grant](troubleshooting-faq/auth-code-grant.md)"
  },
  {
    "instruction": "API",
    "input": "",
    "output": "[API Keys](api/api-keys.md)\n [Rate Limits](api/rate-limits.md)\n [Feebak Regions](api/feebak-regions.md)\n [Digital Survey API](api/digital-survey-api/README.md)\n   [Digital Survey API (version 1)](api/digital-survey-api/digital-survey-api-version-1.md)\n   [Digital Survey API (version 2)](api/digital-survey-api/digital-survey-api-version-2.md)\n   [Digital Survey API (version 3)](api/digital-survey-api/digital-survey-api-version-3.md)\n [Interaction Export API](api/interaction-export-api/README.md)\n   [Interaction Export API (version 1 - deprecated)](api/interaction-export-api/interaction-data-export.md)\n   [Migration Guide for Interaction Export API](api/interaction-export-api/migration-guide-for-interaction-export-api.md)\n   [Interaction Export API (version 2)](api/interaction-export-api/interaction-data-export-1.md)\n [Data Action Performance API](api/data-action-performance-api/README.md)\n   [Reference](api/data-action-performance-api/reference.md)\n [Survey API](api/survey-api/README.md)\n   [Reference](api/survey-api/reference.md)"
  },
  {
    "instruction": "Getting Started",
    "input": "",
    "output": "feebak makes creating, managing, and monitoring after-interaction surveys with your favorite Call Centre Software easy.&#x20;\n\nThis guide will help you to understand how to integrate feebak with your call center software, create surveys and start monitoring your surveys.\n\nCurrently, feebak provides integration with Genesys® Cloud, there will be more integrations available in the future.&#x20;\n\nfeebak offers a comprehensive survey solution for Genesys Cloud CX to collect valuable customer feedback across various customer journey touchpoints such as Contact Center Interactions, Websites, Mobile apps, Social Media, Email, Branch Visits, etc.\n\nKey highlights of Feebak includes:&#x20;\n\n MultiChannel Surveys - supports a variety of channels, including Inbound voice (with or without an agent), outbound IVR, messaging chat, chat widget version 2, email, social messaging, web forms, survey kiosks, and APIs.&#x20;\n Chatbot Mode Survey - Use rich messaging capabilities like buttons and quick replies to create conversational surveys for digital and social messaging platforms.&#x20;\n No Code Drag & Drop Survey Builder - Through a user-friendly interface, without an architect, you can create or edit surveys in minutes rather than days.&#x20;\n Dynamic Survey Trigger - based on customer/contextual information, queue/agent assignment, channel type, CRM ticket closure (API) events, etc.&#x20;\n Detailed Dashboards & Reporting - Rich filter settings, gamification mode agent scoreboards, intuitive real-time and historical dashboards, and BI integration APIs.\n\nSome features of Feebak include:\n\n Personalization - The ability to personalize/prioritize incoming client contacts is made possible by quick access to the most recent survey findings.&#x20;\n Auto survey assignment on Agent call disconnect - Provide surveys on agent disconnect without requiring agent transfer.&#x20;\n Build Once, Deploy Across - A single survey could be sent across multiple channels with the option to report unified or separately using channel filters.&#x20;\n Customizable Webform Survey - Apply your branding theme to the webform surveys.\n Personalized Dynamic Survey Flow - Next questions based on context info or previous question responses.&#x20;\n External Triggers - Based on responses, notifications can be sent via API integration to chat groups, emails, or SMS/CRM.&#x20;"
  },
  {
    "instruction": "Troubleshooting / FAQ",
    "input": "",
    "output": "### Frequently Asked Questions\n\n1. Authorization Token Expired / Invalid\\\n   Issue: Your authorization token has expired, impacting certain features like Voice Feedback Transcription.\\\n   Solution: Follow the steps outlined in the [Adding Authorization Code Grant](auth-code-grant.md) page to restore functionality."
  },
  {
    "instruction": "Adding Authorization Code Grant",
    "input": "",
    "output": "### Steps to Enable Authorization Code Grant\n\n1. Log in to Feebak\n    Use an account with Feebak Organization Admin role to log in.\n2. Navigate to Organization Settings\n    In the Administration Section, go to Organization Settings page.\n3. Add the Authorization Code Grant\n    In the Organization Settings, add the Authorization Code Grant to restore functionality.\n\n<figure><img src=\"../.gitbook/assets/image (430).png\" alt=\"\"><figcaption></figcaption></figure>\n\nThe user must have the necessary permissions to authorize the following scopes for the Authorization Code Grant:\n\n architect\n integrations\n routing\n telephony\n alerting\n authorization conversations\n notifications\n oauth\n organization:readonly\n outbound groups\n\n voicemail speech-and-text-analytics\n speech-and-text-analytics:readonly\n\n{% hint style=\"danger\" %}\nImportant Note\n\nIf the authorization token associated with your organization has expired due to changes in your account such as password change, you must reauthorize your account to restore full access. Failure to update the token will result in functional disruptions, such as the Voice Feedback Transcription feature being unavailable. We highly recommend completing this as soon as possible.\n{% endhint %}"
  },
  {
    "instruction": "Concurrent Licensing",
    "input": "",
    "output": "Concurrent licensing promotes a variant of consuming the licenses purchased from typical method. This type allows the usage of licenses, but calculates only the amount used.\n\nConcurrent license can be assigned to any number of users, the billing is based on peak usage over 30-minutes interval, overage will be charged according to the subscription.\n\n![](../.gitbook/assets/concurrent.png)\n\nTo know more about Assigning feebak Licenses, [click here](assigning-feebak-licences-to-users.md)."
  },
  {
    "instruction": "Named Licensing",
    "input": "",
    "output": "Named Licenses furnish the feature of typical allocation of licenses to desired agents.\\\nThis method enables the assignment of licenses to agents strictly based on the number of licenses purchased.\n\nTo know more about Assigning feebak Licenses, [click here](assigning-feebak-licences-to-users.md)."
  },
  {
    "instruction": "Assigning feebak Licences to users",
    "input": "",
    "output": "{% hint style=\"info\" %}\nNote: This guide only applies for Non Premium Installations.\n{% endhint %}\n\n1. Once you are logged in, different options can be viewed in the navigation bar in which License is present in the Organization section\n2.  Selecting License allows you to view details of all the purchased as well as remaining Licenses\\\n\n\n    ![](<../.gitbook/assets/image (115).png>)\n3.  To manage the User License, click on the Manage User License button at the top right corner of your screen\\\n\n\n    ![](<../.gitbook/assets/image (113).png>)\n4. You can assign new Licenses to Users as well as unassign the existing ones.\n5. Click the save button at the top right corner of the screen to save the changes made.&#x20;"
  },
  {
    "instruction": "Subscription",
    "input": "",
    "output": "To measure agent performance using Feebak surveys in a contact center, the appropriate subscriptions must be assigned to agents.\n\nFeebak offers multiple licensing options:\n\n Feebak 1 -  Provides survey functionality in Voice Channel. A subscription must be assigned to each agent.\n Feebak 2 - Provides survey functionality to both Voice and Digital Channels. A subscription must be assigned to each agent.\n Feebak Digital Addon - Adds digital channel capabilities to agents using Feebak 1.\n Feebak 2 Digital Only - Provides survey functionality in Digital Channels only.\n Feebak Agentless - Enables surveys in agentless scenarios such as Bots, Outbound Voice Campaigns, IVR, CRM, and more.\n Feebak Sentiment Analysis -  Enables sentiment analysis for voice feedback collected at the end of the survey. Billing is calculated per unit, with each unit representing 1,000 characters. For example, feedback containing 1,500 characters will be billed as 2 units.\n Feebak Voice Transcription - Enables transcription of voice feedback collected at the end of the survey. Billing is based on a per-unit basis, where each unit is defined as 60 seconds. For example, feedback lasting 90 seconds will be billed as 2 units.\n Feebak Quickstart Package - Mandatory for new customers onboarding with Feebak. It includes essential technical configurations within the customer's Genesys Cloud organization, along with a comprehensive training program for Feebak administrators and survey admins to ensure a smooth and effective setup.\n Feebak Hourly Service Rate - Covers additional Feebak technical configurations and/or admin training beyond the standard Feebak Quickstart Package. Hours are tracked and managed by Feebak.\n\n{% hint style=\"info\" %}\nNote: Organizations cannot combine Feebak 1 and Feebak 2 licenses. To extend Feebak 1 functionality, the Feebak Digital Add-on can be subscribed to.\n{% endhint %}\n\nFeebak's licensing models are aligned with Genesys Cloud and support:&#x20;\n\n Named Agents - Best suited for organizations with predictable agent staffing needs, offering cost-effective pricing.\n Concurrent Agents - Ideal for organizations with agents working across multiple shifts and peak times, providing flexibility for 24-hour operations."
  },
  {
    "instruction": "Prerequisites",
    "input": "",
    "output": "Feebak licenses are essential for an agent to gain access to their Feebak account. When licensed agent logs into their Genesys Cloud account, they are automatically logged into their Feebak account.\n\nIf an agent is facing an issue where no survey is offered after their interaction with the client, make sure that they are logged into their Feebak account. Even though the Feebak login is done automatically, sometimes agents get logged out after a period of inactivity."
  },
  {
    "instruction": "Subscription Assignment",
    "input": "",
    "output": "The steps to Assign Feebak Subscription to the agents will differ based on how Feebak is installed in the organization."
  },
  {
    "instruction": "**Premium Installation**",
    "input": "",
    "output": "For Premium installation, assign Genesys Cloud permissions as below\n\n<table><thead><tr><th width=\"153\">Subscription</th><th width=\"241.66666666666669\">Genesys Cloud Permission</th><th>Genesys Part</th></tr></thead><tbody><tr><td>Feebak 1</td><td>integration:feebakVoice:agent</td><td><strong>Named</strong><br>GCAFP-170-NV-FEEBAK1<br><strong>Concurrent</strong><br>GCAFP-170-NV-FEEBAK1C</td></tr><tr><td>Feebak 2</td><td>integration:feebakApps:view</td><td><strong>Named</strong><br>GCAFP-170-NV-FEEBAKSUR<br><strong>Concurrent</strong><br>GCAFP-170-NV-FEEBAKSURC</td></tr><tr><td>Feebak Digital Addon</td><td>integration:feebakDigital:agent</td><td><strong>Named</strong><br>GCAFP-170-NV-FEEBAKDGTLADD<br><strong>Concurrent</strong><br>GCAFP-170-NV-FEEBAKDGTLADDC</td></tr><tr><td>Feebak 2 Digital Only</td><td>integration:feebakDigital:agent</td><td><strong>Named</strong><br>GCAFP-170-NV-FEEBAK2DGTLONLY<br><strong>Concurrent</strong><br>GCAFP-170-NV-FEEBAK2DGTLONLYC</td></tr><tr><td>Feebak Agentless Metered</td><td>integration:feebakAgentless:view <br>(Only for users to manage Feebak)</td><td>GCAFP-170-NV-FEEBAKAGNTLSS</td></tr></tbody></table>\n\nTo assign permissions against roles in your organization, refer Genesys documentation for [Roles and Permissions.](https://help.mypurecloud.com/articles/about-roles-permissions/)"
  },
  {
    "instruction": "Additional Services",
    "input": "",
    "output": "Feebak Hourly Service Rate (GCAFS-170-NV-FEEBAKSURPHR) - Covers additional technical configurations and admin training beyond the Quickstart Package.\n Feebak Quickstart Package (GCAFS-170-NV-FEEBAKSURQSE) - Includes technical configurations and training for Feebak administrators and survey admins. Can be purchases along with Feebak 1, Feebak 2, Feebak Digital Addon and Feebak 2 Digital Only licenses."
  },
  {
    "instruction": "Non Premium Installation",
    "input": "",
    "output": "For Non-Premium installation, you can assign the licenses to the agents based on the below guide based on the type of licenses you purchased from Feebak.\n\n[Named Licenses](named-licensing.md) | [Concurrent Licenses](concurrent-licensing.md)"
  },
  {
    "instruction": "Optional Features",
    "input": "",
    "output": "These features can be enabled from the Feebak Organization Settings page, where organizations can choose whether to bill these services through Genesys or Feebak. For billing, please ensure the relevant part numbers are included in your order. The part numbers are listed below:\n\n Feebak Voice Transcription Metered (GCAFP-170-NV-FEEBAKVCTRNSCRPT) - Enables transcription of voice feedback collected at the end of the survey, billed on a per-unit basis, where a unit is defined as 60 seconds.\n Feebak Sentiment Analysis Metered (GCAFP-170-NV-FEEBAKSNTMNTMTRD) - Provides sentiment analysis for voice feedback collected at the end of the survey, also billed on a per-unit basis, where a unit is defined as 1000 characters.\n\nFor any other queries about licenses, contact [sales@feebak.com](mailto:sales@feebak.com)"
  },
  {
    "instruction": "Offer Automated IVR Surveys",
    "input": "",
    "output": "Automating surveys in Feebak involves three main steps to ensure a seamless transfer of calls to the survey flow without the need for agent intervention. This guide will walk you through the process of creating a standard survey, generating an inbound flow, and transferring calls within the IVR.\n\n1.  Create and Publish a Standard Survey in Feebak\n\n    Start by [creating a survey](../introduction-to-feebak/surveys/#add-survey) in Feebak. Ensure you configure all the questions and options according to your requirements. Once your survey is ready, publish it to make it available for use.\n2.  Feebak Generates an Inbound Flow in Genesys\n\n    After publishing your survey, Feebak will automatically generate an inbound flow within Genesys Cloud.\n3. Transfer Calls to the Survey Flow"
  },
  {
    "instruction": "Transfer to Flow Action Configuration",
    "input": "",
    "output": "Use the \"Transfer to Flow\" action to direct the call to the survey flow generated by Feebak, from the available Inbound Call Flows. Additionally, you can set Pre-Transfer and Failed Transfer audios.\n\nTo set up the transfer, follow these steps in Genesys Cloud Architect:\n\n1.  Create a Transfer to Flow Action\n\n     Navigate to the desired location within the Genesys Cloud Architect where the transfer should occur.\n     Add a \"Transfer to Flow\" action.\n     Select the Inbound Call Flow created by Feebak from the list of available flows.\n     Configure the Pre-Transfer Audio and Failed Transfer Audio settings as needed.\n\n    <figure><img src=\"../.gitbook/assets/image (330).png\" alt=\"\"><figcaption></figcaption></figure>\n2.  Configure the IVR Selection Task\n\n     In the IVR configuration, select the task that determines when the call should be transferred to the survey flow. This could be after a specific menu selection or at the end of the call.\n     Assign the \"Transfer to Flow\" task to a DTMF button within your IVR menu.\n\n    <figure><img src=\"../.gitbook/assets/image (329).png\" alt=\"\"><figcaption></figcaption></figure>\n\nBy following these steps, you can effectively automate the survey process, directing calls to the survey flow without any agent intervention, thus enhancing the efficiency of your survey collection efforts."
  },
  {
    "instruction": "Offer Surveys for Open Messaging Channels",
    "input": "",
    "output": "{% hint style=\"info\" %}\nPrerequisites\n\nThe following permissions should be assigned to the Feebak Client role:\n\n Conversation > Message > Create\n Conversation > ThreadingTimeline > View\n{% endhint %}"
  },
  {
    "instruction": "Create an Open Messaging Survey",
    "input": "",
    "output": "To offer a survey for Open Messaging channels, you should turn on Open Messaging in the survey types when creating the survey, as shown below.\n\n![](<../.gitbook/assets/Open Messaging Type.png>)\n\nNext, in the Open Messaging tab of the survey, you have to enter the Survey Prompt and Survey Link Timer.\n\n![](<../.gitbook/assets/Open Messaging Details.png>)\n\nHere, the Survey Link Timer is the delay after which the survey link is shared with the customer. It's recommended to keep its value the same as the threading timeline of the Open Messaging channel in GenesysCloud.\n\nNext, you have to fill in all the other details of the survey and publish it. For detailed documentation for adding a survey, please refer to [this article](../introduction-to-feebak/surveys/). Once the survey is published, you can assign it to a queue (refer to [this article](../introduction-to-feebak/survey-assignments.md) for survey assignments), and then the survey will be offered for conversations in all the open messaging channels.\n\n{% hint style=\"info\" %}\nNote:\n\nTo provide the same survey in different languages, use a script button to set the customer participant data by passing the Feebak_Language attribute with the selected language value.\n{% endhint %}"
  },
  {
    "instruction": "Offer Voice Survey using Set Post-Flow Action",
    "input": "",
    "output": "Set Post-Flow action to transfer an interaction to an inbound call flow rather than disconnecting the interaction. By using this action, we can transfer the survey to the customer while the agent disconnects the interaction using the native Genesys disconnect button.\n\nFor more information about the Set Post-Flow Action and configuring the action, please refer to the link:  [https://help.mypurecloud.com/articles/set-post-flow-action/](https://help.mypurecloud.com/articles/set-post-flow-action/)\n\n   Set the post-flow action before transferring to the queue in the inbound call flow, which is your main flow to transfer the interaction into Agents or in the in-queue call flow. You can choose where should be configured the action based on your requirements.\\\n\n\n    <figure><img src=\"../.gitbook/assets/image (62).png\" alt=\"\"><figcaption></figcaption></figure>\n\n\n In the action,\n   Select the Target as Caller(‘caller’).\n   Select the Inbound Call Flow as Feebak Transfer to Survey Address Flow.\n\n{% hint style=\"info\" %}\nTo ensure accuracy in reporting, please make sure you select the Inbound Call Flow as Feebak Transfer to Survey Address Flow and not any survey call flows published from Feebak. When transferred directly to the survey calls flows, only completed surveys will appear in Feebak dashboards and reports; abandoned surveys will not be displayed.\n{% endhint %}\n\n<figure><img src=\"../.gitbook/assets/image (63).png\" alt=\"\"><figcaption></figcaption></figure>\n\n{% hint style=\"info\" %}\nNote:\n\nTo provide the same survey in different languages, use a script button to set the customer participant data by passing the Feebak_Language attribute with the selected language value.\n{% endhint %}"
  },
  {
    "instruction": "Offer Surveys for Email Channel",
    "input": "",
    "output": "{% hint style=\"info\" %}\nPrerequisites\n\nThe following permissions should be assigned to the Feebak Client role.\n\n Conversation > Message > Create\n Conversation > ThreadingTimeline > View\n{% endhint %}"
  },
  {
    "instruction": "Create an Email Survey",
    "input": "",
    "output": "To offer a survey for Email channels, you should turn on Email in the survey types when creating the survey, as shown below.\n\n<figure><img src=\"../.gitbook/assets/Screenshot_20230206_123632.png\" alt=\"\"><figcaption></figcaption></figure>\n\nThe template must then be selected in the survey's Email tab. You must also fill out the fields for From name, Reply email, and survey timer.\n\n<figure><img src=\"../.gitbook/assets/Screenshot 2025-01-13 162744.png\" alt=\"\"><figcaption></figcaption></figure>\n\n<figure><img src=\"../.gitbook/assets/image (423).png\" alt=\"\"><figcaption></figcaption></figure>\n\nAddress: The domain name / email addres you want to send emails from. You have to configure this beforehand. [Learn more.](../feebak-administration/custom-email-sender/)\n\nFrom Name : From Name (Sender Name) which you want to show in the Survey Invitation Email. \\\neg. Feebak Support Feedback.\n\nReply Email : The email address you want the customers to reply to. If you want to skip this, set it as noreply@your-domain.com.\n\nSurvey Timer: It is the delay after which the survey link is shared with the customer after the agent disconnects.\n\nTemplate: A pre-defined Rich Email Layout in which you can include contents like images or text. You can create new templates as per your requirements.\n\nTo create a new template, Click on the template field.\n\n<figure><img src=\"../.gitbook/assets/Screenshot_20230206_130331.png\" alt=\"\"><figcaption></figcaption></figure>\n\nClick on the \"Add New\" option in the list. Then a new window will get open in which you can create new templates according to your requirement\n\n<figure><img src=\"../.gitbook/assets/Screenshot_20230206_130719.png\" alt=\"\"><figcaption></figcaption></figure>\n\nName: Give a name to your template\n\nType: Select a template type from the given list\n\nSubject: Mention the subject of the template\n\nMail Body: Provide what you want to be the content of the mail. You can also add images, files, etc. in the body.\n\n{% hint style=\"info\" %}\nNote:&#x20;\n\nTo provide the same survey in different languages, use a script button to set the customer participant data by passing the Feebak_Language attribute with the selected language value.\n{% endhint %}\n\nNext, you have to fill in all the other details of the survey and publish it. For detailed documentation on adding a survey, please refer to the article [Surveys](../introduction-to-feebak/surveys/). Once the survey is published, you can assign it to a queue (refer to the article [Survey Assignments](../introduction-to-feebak/survey-assignments.md)), and then the survey will be offered for conversations in the Email channel."
  },
  {
    "instruction": "Offer Surveys for SMS Channel",
    "input": "",
    "output": "{% hint style=\"info\" %}\nPrerequisites\n\nThe following permissions should be assigned to the Feebak Client role:\n\n Conversation > Message > Create\n Conversation > ThreadingTimeline > View\n{% endhint %}"
  },
  {
    "instruction": "Create a SMS Survey",
    "input": "",
    "output": "To offer a survey via SMS, you should turn on the SMS in the survey types when creating the survey as shown below:\n\n<figure><img src=\"../.gitbook/assets/image (358).png\" alt=\"\"><figcaption></figcaption></figure>\n\nNext, in the SMS tab of the survey, you have to enter the Survey Prompt and Survey Link Timer.\n\n<figure><img src=\"../.gitbook/assets/image (359).png\" alt=\"\"><figcaption></figcaption></figure>\n\nHere, the Survey Link Timer is the delay after which the survey link is shared with the customer. It's recommended to keep its value the same as the threading timeline of the SMS channel in Genesys Cloud.\n\nNext, you have to fill in all the other details of the survey and publish it. For detailed documentation for adding a survey, please refer to [this article](../introduction-to-feebak/surveys/). Once the survey is published, you can assign it to a queue (refer to [this article](../introduction-to-feebak/survey-assignments.md) for survey assignments), and then the survey will be offered for conversations in all the SMS channels.\n\n{% hint style=\"info\" %}\nNote:\n\nTo provide the same survey in different languages, use a script button to set the customer participant data by passing the Feebak_Language attribute with the selected language value.\n{% endhint %}"
  },
  {
    "instruction": "Offer Voice Surveys (Automatic)",
    "input": "",
    "output": "{% hint style=\"info\" %}\nPrerequisites\n\nThe following permissions should be assigned to the agents in Genesys Cloud in order to offer surveys:\n\n Integrations > Action > View\n Integrations > Action > Execute\n Routing > CallRoute > View\n{% endhint %}\n\nYou can use Genesys Cloud Post Flow Actions to automatically transfer surveys. To configure the [Post Flow Action](offer-voice-survey-using-set-post-flow-action.md), refer to the documentation.\n\nIf you are using a script-based transfer and want to automate it, then follow the steps below:\n\n1. Create the script for automatic survey transfer. See [Creating Scripts](../survey-scripts-configuration/creating-scripts-manual.md).\n2. Install the [Feebak Chrome extension](https://chromewebstore.google.com/detail/feebak-for-genesys-cloud/bhflhnnljknbclcejfcafmbadhiiflpn) or [Feebak Firefox extension](https://addons.mozilla.org/en-US/firefox/addon/feebak-for-genesys-cloud/?utm_source=addons.mozilla.org\\&utm_medium=referral\\&utm_content=search) in the browsers of all agents that need to offer the survey.\n\nOnce the extension is enabled, clicking Genesys Cloud in-built end-call button will directly transfer the call to the survey.\n\n{% hint style=\"info\" %}\nNOTE: Currentl&#x79;, automatic survey offering is supported on Chrome and Firefox browsers. If you need to support another browser, please let us know at support@feebak.com.&#x20;\n{% endhint %}"
  },
  {
    "instruction": "Offer Surveys for Instagram",
    "input": "",
    "output": "{% hint style=\"info\" %}\nPrerequisites\n\nThe following permissions should be assigned to the Feebak Client role:\n\n Message > Integration > View\n Conversation > ThreadingTimeline > View\n{% endhint %}"
  },
  {
    "instruction": "Add Instagram Platform to Feebak",
    "input": "",
    "output": "1. From the Administration section, click on Message Platforms\n\n![](<../.gitbook/assets/step 1.png>)\n\n2\\. Click on Add New Platform button\n\n3\\. Select Instagram for the Platform. This will list all the Instagram platforms in your GenesysCloud organization. Choose the integration you want to add and click on the Login with Instagram button. This will open the Instagram login prompt.\n\n<figure><img src=\"../.gitbook/assets/image (88).png\" alt=\"\"><figcaption></figcaption></figure>\n\n4\\. Login with Instagram and select the page you want to integrate. Once you complete the steps in the prompt, the button text will change to connected, as shown below.\n\n<figure><img src=\"../.gitbook/assets/image (89).png\" alt=\"\"><figcaption></figcaption></figure>\n\n5\\. Click on the Save button to save the platform."
  },
  {
    "instruction": "Create an Instagram Survey",
    "input": "",
    "output": "To offer a survey on Instagram, you should turn on Instagram in the survey types when creating the survey, as shown below.\n\n<figure><img src=\"../.gitbook/assets/image (90).png\" alt=\"\"><figcaption></figcaption></figure>\n\nNext, in the Instagram tab of the survey, you have to enter the Survey Prompt, Survey Link Label and Survey Link Timer.\n\n<figure><img src=\"../.gitbook/assets/image (92).png\" alt=\"\"><figcaption></figcaption></figure>\n\nHere, Survey Link Timer is the delay after which the survey link is shared with the customer. It's recommended to keep its value the same as the threading timeline of the Instagram Direct channel in GenesysCloud.\n\nNext, you have to fill in all the other details of the survey and publish it. For detailed documentation for adding a survey, please refer to [this article](../introduction-to-feebak/surveys/). Once the survey is published, you can assign it to a queue (refer to [this article](../introduction-to-feebak/survey-assignments.md) for survey assignments), and offer it to the customers."
  },
  {
    "instruction": "Create an Instagram Conversation Survey",
    "input": "",
    "output": "You can create conversational survey for Instagram by selecting the conversation option in the Instagram survey Page\n\n<figure><img src=\"../.gitbook/assets/image (301).png\" alt=\"\"><figcaption></figcaption></figure>\n\nFor this conversation survey to work, you have to configure a data action in the Genesys Cloud. Refer the documentation for data action configuration here: [Conversational Flow Configuration for Instagram](../survey-scripts-configuration/conversational-flow-configuration-for-facebook-and-instagram.md)\n\n{% hint style=\"info\" %}\nNote:\n\nTo provide the same survey in different languages, use a script button to set the customer participant data by passing the Feebak_Language attribute with the selected language value.\n{% endhint %}"
  },
  {
    "instruction": "Offer Digital Surveys using JS SDK",
    "input": "",
    "output": "### How it works\n\nYou can use Feebak's Javascript library to generate a survey link, and offer the survey to your customers."
  },
  {
    "instruction": "Code Snippet to consume Feebak API and generate survey link",
    "input": "",
    "output": "#### 1. Include feebak.chat.js script\n\nmarkup\n<script src=\"https://static.feebakcdn.com/js/feebak.chat.js\"></script>"
  },
  {
    "instruction": "2. Call the API to generate survey link",
    "input": "",
    "output": "javascript\nconst surveyLink = await feebakOfferSurvey({\n    SurveyID: 1234,\n    CustomerID: 'customer email or phone',\n    Language: 'en-us',\n    OrganisationID: 1234,\n    OrganisationKey: 'YOUR_FEEBAK_ORGKEY',\n    ReferenceID: 'SOME_UNIQUE_ID',\n    \"Offered\" : true\n});\n\n\n\n\nfeebakOfferSurvey function returns the survey link and requires the following parameters.\n\n1. SurveyID - ID of the survey you want to offer. It can be found from the Survey Page in Feebak.\n2. Language - language used on your website\n3. OrganisationKey - Every organisation has a unique key. It can be found on the Administration > Organisation Settings page in Feebak.\n4. OrganisationID - ID of your organisation\n5. ReferenceID - You can pass ID here which can be used to uniquely identify this survey in dashboard & reports.\n6. CustomerID (Optional) - Customer email or phone\n7. Offered - To mark the survey as offered. If you want to mark the survey as offered while creating the survey link, you can set the \"Offered\" variable as true.\n\nOnce to get the survey link, you can redirect the customer to the link to offer the survey"
  },
  {
    "instruction": "Offering Surveys",
    "input": "",
    "output": "### Different types of Surveys in Feebak:&#x20;\n\n [Voice Surveys (Automatic)](scripts-to-offer-survey-automatic.md)\n [Voice Surveys (Manual)](call-transferring-survey.md)\n [Voice Survey using Set Post-Flow Action](offer-voice-survey-using-set-post-flow-action.md)\n [Surveys for Facebook Messenger](offer-surveys-for-facebook-messenger.md)\n [Surveys for Instagram](offer-surveys-for-instagram.md)\n [Surveys for Open Messaging Channels](offer-surveys-for-open-messaging-channels.md)\n [Surveys for Web Messaging Channel](offer-surveys-for-web-messaging-channel.md)\n [Surveys for WhatsApp Channel](offer-surveys-for-whatsapp-channel.md)\n [Surveys for Email Channel](offer-surveys-for-email-channel.md)\n [Chat Surveys for Genesys Widgets (version 2)](offering-chat-surveys-for-genesys-widgets-version-2.md)\n [Digital Surveys using JS DSK](offering-digital-surveys-directly.md)"
  },
  {
    "instruction": "Offer Voice Surveys (Manual)",
    "input": "",
    "output": "{% hint style=\"info\" %}\nPrerequisites\n\nThe following permissions should be assigned to the agents in Genesys Cloud in order to offer survey\n\n Integrations > Action > View\n Integrations > Action > Execute\n Routing > CallRoute > View\n\nOffer Survey button should be added to the script. See [Creating Scripts](../survey-scripts-configuration/creating-scripts-manual.md).\n{% endhint %}\n\n![](<../.gitbook/assets/to edit.PNG>)\n\n1. The survey can be offered after receiving the call from customer\n2. Click on the Offer Survey button to disconnect the call and survey to be offered after meeting criteria of offering survey.\n\n{% hint style=\"info\" %}\nThe survey will not be offered if the call is disconnected normally. However, will be recorded and shown in the agent score dashboard.\n{% endhint %}\n\n![](<../.gitbook/assets/image (240).png>)\n\nIn Agent Score Dashboard, Unoffered Surveys are the number of times the agent disconnected the call without offering the survey. Supervisors can warn the agents based on this number.\n\n{% hint style=\"info\" %}\nNote:\n\nTo provide the same survey in different languages, use a script button to set the customer participant data by passing the Feebak_Language attribute with the selected language value.\n{% endhint %}"
  },
  {
    "instruction": "Offer Standalone Surveys",
    "input": "",
    "output": "Unlike other Feebak surveys, which are tied to specific Genesys Cloud interactions or unique customer details passed through API (making each survey link valid for only one submission), Standalone Surveys allow multiple people to respond using the same survey link. This feature is particularly useful when you want to distribute one survey link to a large audience without needing dynamic link generation for each respondent.\n\nWith Standalone Surveys, a single URL can be shared across channels, enabling multiple submissions from different respondents. This feature is ideal for collecting feedback from various individuals while keeping the survey accessible and simple to use.\n\nEnabling Standalone Surveys\n\nFollow the steps below to enable and configure Standalone Surveys in the Feebak Survey Builder:\n\n1. Enable Standalone Surveys:\\\n   In the Survey Builder, go to the General tab and enable Standalone Surveys under Advanced Settings.\n\n<figure><img src=\"../.gitbook/assets/image (8).png\" alt=\"\" width=\"252\"><figcaption></figcaption></figure>\n\n2. Customize Survey Link Identifier:\\\n   Once Standalone Surveys are enabled, Feebak will generate a survey link. You can customize the final part of the URL. The link identifier must be:\n    Unique to your organization.\n    Validated to avoid duplication.\n\n<figure><img src=\"../.gitbook/assets/Screenshot 2024-12-12 142724.png\" alt=\"\" width=\"255\"><figcaption></figcaption></figure>\n\n3. Select Default Survey Type:\\\n   You can select any type from your available conversation types. This will be the Interaction Type for the responses to the Standalone surveys in your Interaction dashboard. You can also create custom conversation types to record standalone surveys. [Learn more.](../introduction-to-feebak/custom-conversation-type.md)\n4. (Optional) Collect Customer Details:\\\n   Decide whether to collect customer details. You can decide each field to be either compulsory or optional and have fields like:\n    Name\n    Email\n    Mobile Number\n\n<figure><img src=\"../.gitbook/assets/image (395).png\" alt=\"\" width=\"254\"><figcaption></figcaption></figure>\n\n5.  Applying Changes\n\n    After configuring your survey, publish the survey and copy the link. The link is now ready to share for multiple responses."
  },
  {
    "instruction": "Offer Surveys for Web Messaging Channel",
    "input": "",
    "output": "## Create a Web Messaging Survey\n\nTo offer a survey for Web Messaging channels, you should turn on Web Messaging in the survey types when creating the survey as shown below.\n\n![](<../.gitbook/assets/Screenshot 2025-01-21 104631.png>)\n\nNext, in the Web Messaging tab of the survey, you have to enter the Survey Prompt.\n\n<figure><img src=\"../.gitbook/assets/image (428).png\" alt=\"\"><figcaption></figcaption></figure>\n\nNext, you have to fill in all the other details of the survey and publish it. For detailed documentation for adding a survey, please refer to [this article](../introduction-to-feebak/surveys/). Once the survey is published, you can assign it to a queue (refer to [this article](../introduction-to-feebak/survey-assignments.md) for survey assignments), and then the survey will be offered for conversations in all the open messaging channels.\n\n{% hint style=\"info\" %}\nNote:\\\nTo provide the same survey in different languages, use a script button to set the customer participant data by passing the Feebak_Language attribute with the selected language value.\n\nTo provide ensure survey delivery for Web Messaging Channel make sure you add https://static.feebakcdn.com to your Content Security Policy for the website and https://mestatic.feebakcdn.com if Feebak is installed in the ME region for your organisation.\n{% endhint %}"
  },
  {
    "instruction": "Create a Conversational Survey for Web Messaging.",
    "input": "",
    "output": "Select the Conversation option from the Web Messaging tab. Then provide Survey Prompt (optional) and Invalid Entry Prompt.\n\n<figure><img src=\"../.gitbook/assets/image (429).png\" alt=\"\"><figcaption></figcaption></figure>\n\nTo trigger surveys after a Web Messaging interaction, there are two approaches you can use:\n\n1. Script Button Configuration:\\\n   Configure a script button to manually trigger survey offerings at the end of the interaction, available for Webform and Conversation surveys. For detailed steps, refer to the documentation [Script Configuration for Conversational Survey in Web Messaging and WhatsApp](../survey-scripts-configuration/script-configuration-for-conversational-survey-in-web-messaging-and-whatsapp.md).\n2. Automated Survey Sending:\\\n   Set up automated surveys to be sent without agent intervention. In this approach, a survey popup is displayed directly within the Web Messaging plugin, allowing users redirect to the webform survey seamlessly. [Learn more](../survey-scripts-configuration/automated-surveys-for-web-messaging.md).\n\nChoose the approach that best fits your operational workflow and customer engagement strategy."
  },
  {
    "instruction": "Offer Surveys for Whatsapp Channel",
    "input": "",
    "output": "{% hint style=\"info\" %}\nPrerequisites\n\nThe following permissions should be assigned to the Feebak Client role:\n\n Conversation > Message > Create\n Conversation > ThreadingTimeline > View\n\nTo Integrate WhatsApp with Genesys Cloud, you have to create corresponding responses in your Genesys Cloud Account.&#x20;\n\nTo add response in Genesys Cloud, refer the link: [https://help.mypurecloud.com/articles/add-a-response/](https://help.mypurecloud.com/articles/add-a-response/)\n{% endhint %}\n\nTo offer a survey for WhatsApp channels, you should turn on WhatsApp in the survey types when creating the survey, as shown below.\n\n<figure><img src=\"../.gitbook/assets/image (328).png\" alt=\"\"><figcaption></figcaption></figure>\n\nIn WhatsApp, there are two approaches to offering a survey:&#x20;\n\n Web Form Survey&#x20;\n Conversational Survey&#x20;"
  },
  {
    "instruction": "Create a WhatsApp Web Form Survey",
    "input": "",
    "output": "In the WhatsApp tab of the survey, select the type Webform. Then you have to enter all the required fields.\n\n<figure><img src=\"../.gitbook/assets/image (326).png\" alt=\"\"><figcaption></figcaption></figure>\n\nLink Substitution: It is an id or name of the message template that you want to send in the survey. Link Substitution can be accessed via your Genesys Cloud Account. Refer the link to know how the Link Substitution can be obtained [https://help.mypurecloud.com/articles/add-a-substitution/](https://help.mypurecloud.com/articles/add-a-substitution/)\n\n<figure><img src=\"../.gitbook/assets/image (273).png\" alt=\"\"><figcaption></figcaption></figure>\n\nTemplate Id: It is the ID of the template using for survey and it can get from the URL of the Canned Response created in Genesys Cloud\n\n<figure><img src=\"../.gitbook/assets/image (190).png\" alt=\"\"><figcaption></figcaption></figure>\n\nSurvey Timer: It is the delay after which the survey link is shared with the customer.\n\n\n\nNext, you have to fill in all the other details of the survey and publish it. For detailed documentation on adding a survey, please refer to the article [Surveys](../introduction-to-feebak/surveys/). Once the survey is published, you can assign it to a queue (refer to the article on [Survey Assignments](../introduction-to-feebak/survey-assignments.md)), and then the survey will be offered for conversations in the WhatsApp channel."
  },
  {
    "instruction": "Create a WhatsApp Conversational Survey",
    "input": "",
    "output": "In the WhatsApp tab, you can see the type \"Conversation\".&#x20;\n\n<figure><img src=\"../.gitbook/assets/image (65).png\" alt=\"\"><figcaption></figcaption></figure>\n\n{% hint style=\"info\" %}\nNote:\n\nTo provide the same survey in different languages, use a script button to set the customer participant data by passing the Feebak_Language attribute with the selected language value.\n{% endhint %}\n\nTo offer a conversation survey after a WhatsApp interaction, you need to configure a script button. You can refer to the documentation [Script configuration for Conversational Survey in Web Messaging and WhatsApp ](../survey-scripts-configuration/script-configuration-for-conversational-survey-in-web-messaging-and-whatsapp.md)for instructions on configuring the script button."
  },
  {
    "instruction": "Offer Kiosk Surveys",
    "input": "",
    "output": "Kiosk Surveys allow customers to submit feedback anonymously using static survey links, making them ideal for environments like retail stores where multiple customers can respond using the same device. These surveys are particularly useful for collecting feedback at various stages of the customer journey.\n\nEnabling Kiosk Surveys\n\nTo set up Kiosk Surveys in Feebak, follow these steps:\n\n1. Enable Static Surveys:\\\n   In the Survey Builder, enable Static Surveys. This generates a reusable survey link that allows multiple users to submit responses.\\\n\n\n<figure><img src=\"../.gitbook/assets/image (9).png\" alt=\"\" width=\"252\"><figcaption></figcaption></figure>\n\n2. Customize Survey Link Identifier:\\\n   Once Kiosk Surveys are enabled, Feebak will generate a survey link. You can customize the final part of the URL. The link identifier must be:\n    Unique to your organization.\n    Validated to avoid duplication.\\\n\n\n<figure><img src=\"../.gitbook/assets/Screenshot 2024-12-12 143041.png\" alt=\"\" width=\"253\"><figcaption></figcaption></figure>\n\n3. Configure Reload Timer:\\\n   Set a Reload Time to automatically restart the survey if a user leaves it incomplete. This ensures that the survey resets for the next participant after the set period of inactivity.\n4. Select Default Survey Type:\\\n   You can select any type from your available conversation types. This will be the Interaction Type for the responses to the Kiosk surveys in your Interaction dashboard. You can also create custom conversation types to record Kiosk surveys. [Learn more.](../introduction-to-feebak/custom-conversation-type.md)\n5.  (Optional) Collect Customer Details:\\\n    Decide whether to collect customer details. You can decide each field to be either compulsory or optional and have fields like:\n\n     Name\n     Email\n     Mobile Number\n\n    <figure><img src=\"../.gitbook/assets/image (395).png\" alt=\"\"><figcaption></figcaption></figure>\n6. Publish and Share the Survey:\\\n   After configuring the survey, publish the survey and copy the link. You can then deploy the survey on kiosks in your physical locations for continuous customer feedback collection."
  },
  {
    "instruction": "Offer Chat Surveys for Genesys Widgets (version 2)",
    "input": "",
    "output": "## Prerequisites\n\nTo follow along with this article, you should already have a Genesys Widget (version 2) deployed on your website. Follow [this article](https://developer.mypurecloud.com/api/webchat/widget-version2.html) if you haven't set up that."
  },
  {
    "instruction": "How it works",
    "input": "",
    "output": "From the Genesys Widget deployment code, you need to call the Feebak API to generate a chat survey link, and then you can offer the survey to the customer by injecting the survey link into the chat or opening the survey link in a new window."
  },
  {
    "instruction": "Code Snippet to consume Feebak API and generate survey link",
    "input": "",
    "output": "1. Include feebak.chat.js script\n\nhtml\n<script src=\"https://static.feebakcdn.com/js/feebak.chat.js\"></script>\n\n\nMake sure you include the Genesys Cloud widget script after changing the regional domain and any necessary settings for your specific environment.\n\n2. Save the conversation to Feebak in the chat start event\n\njavascript\ncustomPlugin.subscribe('WebChatService.started', function (e) {\n        feebakSaveConversation(e);\n});\n\n\n3. Generate the survey link in the chat start\n\njavascript\ncustomPlugin.subscribe('WebChatService.ended', async function (e) {      \n    // Generate chat survey link by calling Feebak API\n    const surveyLink = await feebakOfferSurvey({\n        CustomerID: 'customer email or phone',\n        Language: 'en-us',\n        OrganisationID: 1234,\n        OrganisationKey: 'YOUR_FEEBAK_ORGKEY_HERE',\n    });\n});\n\n\nfeebakOfferSurvey function returns the survey link and requires the following parameters.\n\n Language - language used on your website\n OrganisationKey - Every organisation has a unique key. It can be found on the Administration > Organisation Settings page in Feebak.\n OrganisationID - ID of your organisation\n CustomerID (Optional) - Customer email or phone\n\nOnce you get the survey link you can offer the survey to the customer by injecting the survey link into the chat or opening the survey link in a new window.\n\n4. Inject the Survey into the Conversation\n\nOnce you get the survey link, you can offer the survey to the customer by injecting the survey link into the chat. This can be done in two ways: by sending a message with a link or by embedding the survey in an iframe.\n\n Passing a Link in the Conversation:\n\njavascript\nif (surveyLink) {\n    customPlugin\n        .command('WebChat.injectMessage', {\n            type: 'html',\n            name: 'Feebak',\n            text: Please click on this <a href=\"${surveyLink}\">link</a> to take part in a survey,\n            custom: false,\n            bubble: {\n                fill: '#3168d2',\n                radius: '4px',\n                time: false,\n                name: false,\n                direction: 'right',\n                avatar: {\n                    custom: '<div>word</div>',\n                    icon: 'email',\n                },\n            },\n        })\n        .done(function (e) {\n            // WebChat injected a message successfully\n            // e.data == The message HTML reference (jQuery wrapped set)\n        })\n        .fail(function (e) {\n            // WebChat isn't open or no active chat\n        });\n}\n\n\n\n Embedding an Iframe in the Conversation:\n\njavascript\nif (surveyLink) {\n    customPlugin\n      .command('WebChat.injectMessage', {\n        type: 'html',\n        name: 'Feebak',\n        text: <iframe id=\"chat-survey-frame\" src=\"${surveyLink}\" ></iframe>,\n        // text: <div id=\"chat-survey\"><div class=\"question-box\"><p class=\"question\">${surveyData.GreetingTexts['en-us']}</p><button class=\"sb\" onclick=\"nextQuestion(0)\">Start</button></div></div>,\n        custom: false,\n        bubble: {\n          fill: 'transparent',\n          radius: '4px',\n          time: false,\n          name: false,\n          direction: 'right',\n          avatar: {\n            custom: '<div>word</div>',\n            icon: 'email',\n          },\n        },\n      })\n      .done(function (e) { \n         // WebChat injected an iframe successfully\n      })\n      .fail(function (e) {\n        // WebChat isn't open or no active chat\n      });\n    console.log(document.getElementById('chat-survey-frame').innerHTML);\n  }"
  },
  {
    "instruction": "Code sample for offering Feebak chat surveys using Genesys Widgets (version 2)",
    "input": "",
    "output": "The following is code for a simple web page with the Genesys WebChat widget which will offer a Feebak chat survey when the agent disconnects.&#x20;\n\nmarkup\n<script\n    src=\"https://apps.mypurecloud.com/widgets/9.0/cxbus.min.js\"\n    onload=\"javascript:CXBus.configure({debug:false,pluginsPath:'https://apps.mypurecloud.com/widgets/9.0/plugins/'}); CXBus.loadPlugin('widgets-core');\"\n></script>\n<script src=\"https://static.feebakcdn.com/js/feebak.chat.js\"></script>\n\n<script>\n    window._genesys = {\n        widgets: {\n            webchat: {\n                transport: {\n                    type: 'purecloud-v2-sockets',\n                    dataURL: 'https://api.mypurecloud.com',\n                    deploymentKey: 'YOUR_DEPLOYMENTKEY_HERE',\n                    orgGuid: 'YOUR_ORGGUID_HERE',\n                    interactionData: {\n                        routing: {\n                            targetType: 'QUEUE',\n                            targetAddress: 'YOUR_QUEUENAME_HERE',\n                            priority: 2,\n                        },\n                    },\n                },\n                userData: {\n                    addressStreet: '',\n                    addressCity: '',\n                    addressPostalCode: '',\n                    addressState: '',\n                    phoneNumber: '',\n                    customField1Label: '',\n                    customField1: '',\n                    customField2Label: '',\n                    customField2: '',\n                    customField3Label: '',\n                    customField3: '',\n                },\n            },\n        },\n    };\n\n    function getAdvancedConfig() {\n        return {\n            form: {\n                autoSubmit: false,\n                firstname: '',\n                lastname: '',\n                email: '',\n                subject: '',\n            },\n            formJSON: {\n                wrapper: '<table></table>',\n                inputs: [\n                    {\n                        id: 'cx_webchat_form_firstname',\n                        name: 'firstname',\n                        maxlength: '100',\n                        placeholder: 'Required',\n                        label: 'First Name',\n                    },\n                    {\n                        id: 'cx_webchat_form_lastname',\n                        name: 'lastname',\n                        maxlength: '100',\n                        placeholder: 'Required',\n                        label: 'Last Name',\n                    },\n                    {\n                        id: 'cx_webchat_form_email',\n                        name: 'email',\n                        maxlength: '100',\n                        placeholder: 'Optional',\n                        label: 'Email',\n                    },\n                    {\n                        id: 'cx_webchat_form_subject',\n                        name: 'subject',\n                        maxlength: '100',\n                        placeholder: 'Optional',\n                        label: 'Subject',\n                    },\n                ],\n            },\n        };\n    }\n\n    const customPlugin = CXBus.registerPlugin('Custom');\n\n    // Save the conversation in Feebak when chat starts\n    customPlugin.subscribe('WebChatService.started', function (e) {\n        feebakSaveConversation(e);\n    });\n\n    // Offer survey when the chat ends\n    customPlugin.subscribe('WebChatService.ended', async function (e) {\n        \n        // Generate chat survey link by calling Feebak API\n        const surveyLink = await feebakOfferSurvey({\n            CustomerID: 'customer email or phone',\n            Language: 'en-us',\n            OrganisationID: 1234,\n            OrganisationKey: 'YOUR_FEEBAK_ORGKEY_HERE',\n        });\n\n        // Inject the link as a message if it's not null\n        if (surveyLink) {\n            customPlugin\n                .command('WebChat.injectMessage', {\n                    type: 'html',\n                    name: 'Feebak',\n                    text: Please click on this <a href=\"${surveyLink}\">link</a> to take part in a survey,\n                    custom: false,\n                    bubble: {\n                        fill: '#3168d2',\n                        radius: '4px',\n                        time: false,\n                        name: false,\n                        direction: 'right',\n                        avatar: {\n                            custom: '<div>word</div>',\n                            icon: 'email',\n                        },\n                    },\n                })\n                .done(function (e) {\n                    // WebChat injected a message successfully\n                    // e.data == The message HTML reference (jQuery wrapped set)\n                })\n                .fail(function (e) {\n                    // WebChat isn't open or no active chat\n                });\n        }\n    });\n</script>\n\n<button\n    type=\"button\"\n    id=\"chat-button\"\n    onclick=\"customPlugin.command('WebChat.open', getAdvancedConfig());\"\n>\n    Start Chat\n</button>"
  },
  {
    "instruction": "Offer Surveys for Facebook Messenger",
    "input": "",
    "output": "{% hint style=\"info\" %}\nPrerequisites\n\nThe following permissions should be assigned to the Feebak Client role.\n\n Message > Integration > View\n Conversation > ThreadingTimeline > View\n{% endhint %}"
  },
  {
    "instruction": "Add Facebook Platform to Feebak",
    "input": "",
    "output": "1. From the Administration section, click on Message Platforms\n\n![](<../.gitbook/assets/step 1.png>)\n\n2\\. Click on Add New Platform button\n\n3\\. Select Facebook for the Platform. This will list all the Facebook platforms from your GenesysCloud organization. Choose the integration you want to add and click on Connect to Facebook button. This will open Facebook login prompt.\n\n![](<../.gitbook/assets/image (188).png>)\n\n4\\. Login with Facebook and select the page you want to integrate. Once you complete the steps in the prompt, the button text will change to connected as shown below.\n\n![](<../.gitbook/assets/image (207).png>)\n\n5\\. Click on the Save button to save the platform."
  },
  {
    "instruction": "Create a Facebook Survey",
    "input": "",
    "output": "To offer a survey in Facebook, you should turn on Facebook Messenger in the survey types when creating the survey as shown below.\n\n\n\n![](<../.gitbook/assets/image (138).png>)\n\nNext, in the FB Messenger tab of the survey, you have to enter the Survey Prompt, Survey Link Label and Survey Link Timer.\n\n![](<../.gitbook/assets/FB Messenger settigns (1).png>)\n\nHere, Survey Link Timer is the delay after which the survey link is shared with the customer. It's recommended to keep its value the same as the threading timeline of the Facebook Messenger channel in GenesysCloud.\n\nNext, you have to fill in all the other details of the survey and publish it. For detailed documentation for adding a survey, please refer to [this article](../introduction-to-feebak/surveys/). Once the survey is published, you can assign it to a queue (refer to [this article](../introduction-to-feebak/survey-assignments.md) for survey assignments), and offer it to the customers.\n\n{% hint style=\"info\" %}\nNote:\n\nTo provide the same survey in different languages, use a script button to set the customer participant data by passing the Feebak_Language attribute with the selected language value.\n{% endhint %}"
  },
  {
    "instruction": "Create Facebook Conversational Survey",
    "input": "",
    "output": "You can create conversational survey for Facebook by selecting the conversation option in the Facebook survey Page\n\n<figure><img src=\"../.gitbook/assets/image (300).png\" alt=\"\"><figcaption></figcaption></figure>\n\nFor this conversation survey to work, you have to configure a data action in the Genesys Cloud. Refer the documentation for data action configuration here: [Conversational Flow Configuration for Facebook](../survey-scripts-configuration/conversational-flow-configuration-for-facebook-and-instagram.md)"
  },
  {
    "instruction": "Feebak Self Serve Experience",
    "input": "",
    "output": "Feebak provides organizations with a seamless way to integrate and use customer survey tools within Genesys Cloud. This guide covers key features, from installation to survey setup, enabling efficient feedback collection and analysis. As a Genesys AppFoundry partner, Feebak offers easy integration, customizable survey tools, and powerful insights to enhance customer experience initiatives."
  },
  {
    "instruction": "Integrating the Application",
    "input": "",
    "output": "Integrating Feebak with Genesys Cloud is a straightforward process, involving the installation and activation of the application. In this section, you’ll be guided through installing Feebak from the Genesys AppFoundry, configuring group access, and activating the app to ensure it functions properly within your environment. By following these steps, you'll be set up to start gathering valuable customer feedback quickly and efficiently."
  },
  {
    "instruction": "Install Feebak <a href=\"#install-and-activate-feebak\" id=\"install-and-activate-feebak\"></a>",
    "input": "",
    "output": "1. Ensure Admin Access: Make sure you are logged in with a Master Admin account that has access to all divisions in Genesys Cloud for a smooth installation process.\n2. Navigate to Admin Settings: Once logged in, navigate to the Admin section from the navigation bar.\n3. Access Integrations: In the Admin section, select Integrations. Then, click on the Add Integration button located in the top right corner of the screen to add a new app.\n4. Search for Feebak: In the search bar, type Feebak and select it from the list of results.\n5. Install Feebak Application: Click the Install button to begin the installation process for Feebak.\n\n<figure><img src=\"../.gitbook/assets/image (226).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "Activate Feebak <a href=\"#activate-the-feebak-application\" id=\"activate-the-feebak-application\"></a>",
    "input": "",
    "output": "1. Open Feebak Integration: After installing, the Feebak integration page should open automatically. If it doesn’t, go to Admin > Integrations and select Feebak from the list of installed integrations.\n\n<figure><img src=\"../.gitbook/assets/Screenshot_20221229_165519.png\" alt=\"\"><figcaption></figcaption></figure>\n\n2. Activate Feebak: In the Feebak integration page, toggle the Activate button and then click Save to enable the application.\n3. Configure Group Access: If you want to limit access to Feebak based on user groups, go to the Configuration tab, and in Properties, navigate to Group Filtering. Select the groups you want grant access to Feebak. If no group is selected, all users will be able to view Feebak.\n4. Navigate to Feebak in Apps: After activation, go to the Apps section in the navigation bar and select Feebak.\n5. Start Feebak: A welcome screen will appear. Pick your preferred data hosting region (Europe & Middle East) and click on Start to begin the setup process.\n\n<figure><img src=\"../.gitbook/assets/image (2) (1).png\" alt=\"\"><figcaption></figcaption></figure>\n\n6. Set Timezone and Install: Choose your organization's timezone and click Install. The installation process will take around 30 seconds.\n\n<figure><img src=\"../.gitbook/assets/Screenshot_20221228_181514.png\" alt=\"\"><figcaption></figcaption></figure>\n\n7. Access Feebak: Once installation is complete, the Feebak Survey Activity Timeline page will open, and the system will be ready for use.\n\n![](<../.gitbook/assets/Screenshot 2024-10-07 200151.png>)"
  },
  {
    "instruction": "Creating surveys",
    "input": "",
    "output": "Surveys in Feebak are a core tool for gathering customer feedback and insights. Setting up a survey involves several key terminologies and features that allow you to tailor the experience to your organization's needs."
  },
  {
    "instruction": "Survey Components",
    "input": "",
    "output": "Surveys: A survey is a set of structured questions aimed at gathering customer feedback after an interaction. Surveys can be customized with various question types, from simple yes/no to detailed multiple choice questions.\n Survey Channels: Feebak supports various interaction types, including voice, digital (Chat, Email, WhatsApp, Instagram, Facebook Messenger), and agentless scenarios (such as Outbound IVR or Bot interactions). Surveys can be deployed across these multiple channels, ensuring omnichannel feedback collection. This approach allows businesses to effectively reach their customers wherever they interact, optimizing the collection of valuable insights.\n Prompts: Prompts are the specific instructions or messages displayed or played to the participant during the survey. These can be customized to guide respondents effectively through the survey process.\n Score Categories: Score categories help break down the feedback into meaningful insights, such as customer satisfaction (CSAT) or Net Promoter Score (NPS). Feebak also allows you to define custom score categories to suit your organization’s needs."
  },
  {
    "instruction": "Create a Survey",
    "input": "",
    "output": "Creating a survey in Feebak is a straightforward process that enables organizations to gather valuable customer feedback effectively. Here are the key steps involved in creating a survey:\n\n1. Access the Survey Builder: Navigate to the survey builder (Administration > Surveys > Add New Survey) within Feebak to start building your survey.\n2. Define Survey Objectives: Clearly outline the purpose of your survey to guide your question selection and design. Then provide a clear title and a brief description for your survey.\n3. Select Languages: Choose all the supported languages of your survey, and the default language.\n4. Select Interaction Types: Choose from the available interaction channels, like Voice, Chat, FB Messenger, Email, Web Messaging, Open Messaging, WhatsApp, Instagram, and SMS.\n5. Select Score Category: Choose a score category for your survey, such as CSAT (Customer Satisfaction) or NPS (Net Promoter Score), to effectively categorize and analyze the feedback you collect.\n6. Select division: Choose from the available divisions under Publish Settings tab.\n7. Customize Questions: Add questions to your survey, ensuring they are concise and relevant to the feedback you wish to gather. You can also customize prompts to guide respondents through the survey effectively. Make sure you select score categories for individual questions as well.\n8. Select Question Types: Choose from various question types such as multiple-choice, yes/no, NPS, CSAT, Numeric Range, Multiple Choice, or open-ended questions. This allows you to collect diverse feedback based on your specific needs.\n9. Preview the Survey: Before finalizing, preview the survey to see how it will appear to respondents. This allows you to make any necessary adjustments for clarity and engagement.\n10. Save and Publish: Once you've configured everything within the survey, save your changes and click publish it to push it to Genesys Cloud.&#x20;\n\nFor detailed instructions on adding a survey, refer to our [documentation](../introduction-to-feebak/surveys/#add-survey)."
  },
  {
    "instruction": "Import Divisions, Queues and Users",
    "input": "",
    "output": "To streamline your survey setup, you can import existing divisions, queues, and users into Feebak. This simplifies the process of assigning surveys and ensures accurate data management. Here’s how to import:"
  },
  {
    "instruction": "**Import Divisions**",
    "input": "",
    "output": "1. Navigate to the Division Page: Access the divisions page within Feebak (Administration > Divisions).\n2. Select Import Divisions: Click on the Import Divisions button.\n3. Select Required Divisions: Choose all the divisions you want to import from the list.\n4. Click Import Selected: After making your selections, click on the Import Selected button.\n\n{% hint style=\"info\" %}\nNote: Before importing divisions to Feebak, ensure the Feebak OAuth Client has access to the necessary divisions to facilitate importing queues from them. In Genesys Cloud, go to Admin > OAuth, search for Feebak OAuth Client, navigate to the Roles tab, and verify that the required divisions are assigned to the Feebak Client role.\n{% endhint %}\n\n<figure><img src=\"../.gitbook/assets/image (364).png\" alt=\"\"><figcaption></figcaption></figure>\n\nFor detailed instructions on importing divisions, refer to our [documentation.](../feebak-administration/division-management/#import-genesys-cloud-users)"
  },
  {
    "instruction": "**Import Queues**",
    "input": "",
    "output": "1. Navigate to the Queue Page: Access the queues page within Feebak (Administration > Queues).\n2. Select Import Queues: Click on the Import Queues button.\n3. Select Required Queues: Choose all the queues you wish to import by selecting them from the list.\n4. Click Import Selected: After selecting the desired queues, click on the Import Selected button.\n\n{% hint style=\"info\" %}\nNote: If a queue isn’t visible in the list, it’s likely due to missing division access in the client token. Verify that all required divisions are selected for the Feebak OAuth Client in Genesys Cloud.\n{% endhint %}\n\n<figure><img src=\"../.gitbook/assets/image (30).png\" alt=\"\"><figcaption></figcaption></figure>\n\nFor detailed instructions on importing queues, refer to our [documentation](../introduction-to-feebak/queues.md#import-queues)."
  },
  {
    "instruction": "**Import Users**",
    "input": "",
    "output": "1. Navigate to the User Page: Access the users page within Feebak (Administration > Users).\n2. Select Import Users: Click on the Import Users button.\n3. Select Required Users: Choose the users you wish to import from the list.\n4. Click Import Selected: After selecting the necessary users, click on the Import Selected button.\n\n{% hint style=\"info\" %}\nNote: Agents will only be able to offer surveys for interactions if they are imported into Feebak.\n{% endhint %}\n\n![](<../.gitbook/assets/Screenshot 2024-08-29 170333.png>)\n\nFor detailed instructions on importing users, refer to our [documentation.](../feebak-administration/user-management/#import-users)"
  },
  {
    "instruction": "Set Survey Assignment",
    "input": "",
    "output": "Survey Assignments dictate when and to whom surveys are sent by assigning them to specific queues based on various conditions. These conditions include the date, participant data, interaction type, queue priority, and agent. This ensures that feedback is collected from the most relevant touchpoints, optimizing the effectiveness of your surveys.\n\n1. Navigate to Survey Assignments Page: Access the survey assignments page within Feebak to configure your survey distribution settings.\n2. Select Queue and Activate Subscription: From the dropdown menu, select the specific queue to which you want to assign the survey. Then, turn on the queue subscription to enable survey delivery for that queue.\n3. Activate the Survey: Once the queue is selected, add a survey for that queue to ensure it will be sent out to the designated participants.\n4. Provide Survey Preferences: If applicable, set any additional survey preferences or conditions, such as specific dates, participant data or user, to further refine when and to whom the survey will be sent. This ensures that feedback is collected from the most relevant touchpoints.\n\n![](<../.gitbook/assets/image (276).png>)\n\nFor detailed instructions on setting survey assignment, refer to our [documentation.](../introduction-to-feebak/survey-assignments.md#add-survey-activation)"
  },
  {
    "instruction": "Routing Interactions to Surveys",
    "input": "",
    "output": "Once the survey assignments are configured in Feebak, Genesys Cloud interactions can be routed to surveys. The routing method varies depending on the interaction channel:\n\n1. Voice Interactions: You can offer surveys through either the Set Post-Flow Action in Architect or a Script Button. For more details, refer to the [Post-Flow Action Guide](../offering-surveys/offer-voice-survey-using-set-post-flow-action.md) or the [Voice Survey Script Configuration Guide.](../survey-scripts-configuration/creating-scripts-manual.md#offer-voice-interaction-survey)\n2. Web Messaging & WhatsApp: Surveys are delivered using the Script Button for conversational surveys in WhatsApp and for all surveys in Web Messaging. For WhatsApp webform surveys, the surveys are automatically sent if survey assignment is set for the queue. For more details, refer to the [Script Configuration for Conversational Surveys Guide ](../survey-scripts-configuration/script-configuration-for-conversational-survey-in-web-messaging-and-whatsapp.md)and for webform approach in Web Messaging, refer to [Script Configuration for Web Messaging Guide.](../survey-scripts-configuration/creating-scripts-manual.md#offer-web-messaging-survey)\n3. Other Channels (Email, SMS, Facebook Messenger, Instagram, Open Messaging and Chat): Surveys are automatically offered once the agent disconnects from the interaction, if survey assignment is set for the queue. For details about using conversational flow configuration for FB Messenger and Instagram, refer to [Conversational Flow Configuration for Facebook and Instagram Guide.](../survey-scripts-configuration/conversational-flow-configuration-for-facebook-and-instagram.md)\n\n<div align=\"center\"><figure><img src=\"../.gitbook/assets/conversational (1).jpg\" alt=\"\" width=\"188\"><figcaption><p>Conversational Survey in WhatsApp</p></figcaption></figure> <figure><img src=\"../.gitbook/assets/webform (1).jpg\" alt=\"\" width=\"375\"><figcaption><p>Webform Survey in WhatsApp</p></figcaption></figure></div>\n\nConversational Surveys occur within the same messaging conversation, such as WhatsApp, where survey questions and responses appear as interactive message options or buttons within the chat. This keeps the survey experience consistent and convenient for customers within the original conversation.\n\nWebform (Link-Based) Surveys are delivered as a link sent to the customer. Upon clicking, the link opens a webform in a browser, where the customer can complete the survey."
  },
  {
    "instruction": "Best Practices for Effective Survey Implementation",
    "input": "",
    "output": "To maximize the impact of your surveys, consider the following best practices:\n\n Define Clear Objectives: Establish clear goals for what you want to achieve with each survey, ensuring that questions align with these objectives.\n Keep It Short and Engaging: Design surveys that are concise and engaging, encouraging higher response rates while still gathering meaningful insights.\n Utilize Post-Survey and Timeout Actions: Take advantage of [Post Survey Actions](../introduction-to-feebak/surveys/#post-survey-actions) to follow up with respondents, thanking them for their feedback or use data action to push customer data to your CRM. [Survey Timeout Actions](../introduction-to-feebak/surveys/#survey-timeout-settings) will also allow you to handle incomplete surveys effectively, ensuring you get as much feedback as possible.\n Leverage Rules and Actions: Close the feedback loop by setting up [Rules and Actions](../introduction-to-feebak/surveys/#rules-and-actions) to trigger automatic follow-ups or tasks based on specific conditions or responses. This ensures that important feedback doesn’t slip through the cracks and timely action can be taken.\n Enable Voice and Digital Open Feedback: For more comprehensive insights, use Voice Feedback to collect open-ended responses from users in voice surveys. Similarly, enable Feedback Notes for digital channels like WhatsApp, and Instagram to capture users' detailed thoughts and suggestions in their own words. This helps you gain a deeper understanding of customer sentiment."
  },
  {
    "instruction": "Analyzing Feedback",
    "input": "",
    "output": "Once surveys are deployed and responses collected, analyzing the feedback is crucial for gaining actionable insights. Feebak offers several features to facilitate this:\n\n Dashboards: Access real-time dashboards that visualize survey responses, making it easy to track trends and performance metrics. Feebak offers a variety of dashboards, including the [Interaction Dashboard](../introduction-to-feebak/dashboard/interactions.md), [Survey Activity Timeline](../introduction-to-feebak/dashboard/supervisor-dashboard-1-1-1-1-1.md), [Survey Score Summary](../introduction-to-feebak/dashboard/survey-score-summary.md), [Survey Response Summary](../introduction-to-feebak/dashboard/survey-response-summary.md), [NPS Summary](../introduction-to-feebak/dashboard/nps-summary.md), [CSAT Summary](../introduction-to-feebak/dashboard/csat-summary.md), [Agent Scoreboard](../introduction-to-feebak/dashboard/agent-scoreboard.md) and [Agent Dashboard.](../introduction-to-feebak/dashboard/agent-dashboard-1.md)\n Reports: Feebak provides comprehensive reporting tools to analyze survey data in detail. Generate reports such as the [Survey Score Details Report](../introduction-to-feebak/report-generation/survey-score-details-1.md) and [Survey Answer Details Report](../introduction-to-feebak/report-generation/survey-answer-details.md), offering insights into both survey performance and individual responses. You can also schedule these reports to receive them via email, ensuring you have timely access to the latest data for informed decision-making."
  },
  {
    "instruction": "Getting Support",
    "input": "",
    "output": "If you have questions or need assistance while using Feebak, our support team is here to help:\n\n Help Center: For quick access to personalized support, visit our Help Center at [support.feebak.com.](http://support.feebak.com) You can submit your inquiries or report any issues directly through the portal, and our team will respond promptly.\n Email Support: Alternatively, you can contact us by emailing support@feebak.com for assistance.\n\nFeebak empowers organizations to efficiently capture and act on customer feedback, turning insights into tangible improvements. Seamlessly integrated with Genesys Cloud, Feebak offers flexible survey customization, real-time analytics, and powerful reporting tools that help you elevate your customer experience initiatives. With Feebak, you can proactively address customer needs, optimize service delivery, and drive lasting improvements across your products and services."
  },
  {
    "instruction": "Konnect Insights Integration",
    "input": "",
    "output": "Integrating Feebak with Konnect Insights enables seamless data exchange between the two platforms, enhancing your ability to manage customer feedback efficiently. This guide walks you through the process of obtaining the required credentials from Feebak, including your Organization ID, Organization Key, Client ID, and Client Secret. These details are essential for setting up the integration and ensuring secure communication between Feebak and Konnect Insights. Follow the detailed steps below to gather these credentials and complete the integration process.\n\nStep 1: Find Your Organization ID and Organization Key\n\n1. Log in to Feebak using your administrator credentials.\n2. Navigate to the Administration section in the sidebar.\n3. Go to the Organisation Settings:\n    In the Organisation section, find and click on Organisation Settings.\n4. Find Organization ID & Organization Key:\n    Your Organization ID & Organization Key will be displayed under the Organization Details tab.\n\nStep 2: Generate Client ID and Client Secret\n\n1. Navigate to API Keys:\n    Once logged in, Navigate to the Administration section in the sidebar. In the Organisation section, find and click on API Keys.\n2. Generate a New API Key:\n    In the API Keys page, click on the \"Generate Key\" button. This action will open a pop-out window displaying your newly generated Key ID and Key Secret.\n3. Save the Credentials:\n    Important: Securely save the Key ID and Key Secret as these details will not be displayed again. Remember, do not share your API Key Secret with anyone or on any public platform.\n4. View and Manage API Keys:\n    After generating the key, you can see the Key ID, Key Secret, and the creation date listed on the API Keys page. This is your Client ID and Client Secret.\n\nStep 3: Input the Details into Konnect Insights\n\n1. Open Konnect Insights.\n2. Add Feebak Details:\n    Fill in the Profile Name (optional).\n    Enter the Feebak Organization ID and Organization Key obtained in Step 1.\n    Enter the Client ID and Client Secret from Step 2.\n3. Save the Configuration:\n    Click Save to complete the integration process.\n\n<figure><img src=\"../.gitbook/assets/konnectInsights.png\" alt=\"\"><figcaption></figcaption></figure>\n\nWith these steps, your Feebak account should be successfully integrated with KonnectInsights, allowing for seamless data sharing and enhanced insights."
  },
  {
    "instruction": "IP Whitelisting in Genesys Cloud (Optional)",
    "input": "",
    "output": "IP whitelisting serves as an optional security measure to control access to your Genesys Cloud organization This documentation outlines the steps to whitelist Feebak's IP address along with relevant IPs from Google Cloud and Microsoft Azure to ensure seamless functionality of Feebak within your Genesys Cloud account. IP whitelisting becomes necessary only if the organization restricts access from selected IP addresses.\n\nPrerequisites: Before proceeding with IP whitelisting, ensure that you have administrative access to your Genesys Cloud organization settings.\n\nSteps to Whitelist IP Addresses:\n\n1. Check Organization Settings:\n    Log in to your Genesys Cloud account.\n    Navigate to Organization Settings.\n    Confirm whether IP whitelisting is enabled by checking the \"Allowed IP Address\" settings.\n2. Add Feebak's IP Address:\n    If IP whitelisting is enabled, add Feebak's IP address: 35.204.147.48/32.\n3. Google Cloud IPs:\\\n   Whitelist the outbound IP ranges for Google Cloud, where Feebak is hosted, if you have added IP address restrictions. These IP ranges are subject to change and should be periodically updated.\n4. Microsoft Azure IPs:\\\n   Additionally, if your organization utilizes Microsoft Azure, whitelist the outbound IP addresses for Azure functionalities used by Feebak.\n5.  Periodic Updates:\n\n    As Google Cloud and Microsoft Azure update their IP ranges, it's essential to periodically update the allowed IP addresses in your Genesys Cloud organization settings.\n\n    \\\n    Access the following URLs for up-to-date IP range information:\n\n     Google Cloud: [https://www.gstatic.com/ipranges/cloud.json](https://www.gstatic.com/ipranges/cloud.json)\n     Google APIs: [https://www.gstatic.com/ipranges/goog.json](https://www.gstatic.com/ipranges/goog.json)\n     Microsoft Azure: [https://www.microsoft.com/en-us/download/details.aspx?id=56519](https://www.microsoft.com/en-us/download/details.aspx?id=56519)\n\nNote: Ensure to follow the provided URLs for the most current IP range information, as cloud vendors frequently update their IP addresses."
  },
  {
    "instruction": "Setup & Integration",
    "input": "",
    "output": "Feebak can be installed in two ways:\n\n1. [Premium Installation](premium-installation/) (Billing is done by Genesys Cloud. For knowing more about Genesys billing, refer to the link [https://appfoundry.genesys.com/docs/billing](https://appfoundry.genesys.com/docs/billing))\n2. [Non-Premium Installation](non-premium-installation/) (Billing is done by Feebak)"
  },
  {
    "instruction": "Prerequisites",
    "input": "",
    "output": "Genesys Cloud CX Admin Account\n [Feebak Subscription](../feebak-subscription/)\n [IP Whitelisting for Feebak in Genesys Cloud](ip-whitelisting-in-genesys-cloud-optional.md) ( Optional - Only if the organization restricts access from selected IP addresses)."
  },
  {
    "instruction": "CRM Connectors Integration",
    "input": "",
    "output": "Integrating Feebak with CRM connectors for Salesforce or SAP C4C allows for automatic transfers to surveys at the end of interactions, enhancing customer feedback collection directly within the CRM. Follow the steps below to install and configure the extension, ensuring it operates smoothly within your organization’s existing setup."
  },
  {
    "instruction": "**Step 1: Install the Connector Extension**",
    "input": "",
    "output": "To begin the integration, you’ll need to install the Feebak extension:\n\n1. Visit the [Chrome Web Store link](https://chrome.google.com/webstore/detail/feebak-for-salesforce/klpfkakgklajfdmgoacjfnbdboobekcn).\n2. Click on Add to Chrome (or Edge if you are using Microsoft Edge) and follow the prompts to install the extension."
  },
  {
    "instruction": "Step 2: Configure the Connector",
    "input": "",
    "output": "Once the extension is installed, you need to configure the connector by creating and applying the appropriate registry settings for your browser. Follow the steps below to create, and run the configuration file.\n\nStep 2.1: Create the Configuration File\n\nDepending on your browser, use the correct registry configuration:\n\n   Google Chrome Configuration:\n\n     Open a text editor (e.g., Notepad or VS Code).\n     Copy and paste the following content into the file:\n\n    bash\n    Windows Registry Editor Version 5.00\n    [HKEY_LOCAL_MACHINE\\Software\\Policies\\Google\\Chrome\\3rdparty\\extensions\\klpfkakgklajfdmgoacjfnbdboobekcn\\policy]\n    \"orgkey\"=\"YOUR_ORGANISATION_KEY_HERE\"\n    \"orgid\"=\"YOUR_ORGANISATION_ID_HERE\"\n    \n\n     Save the file as chrome.reg.\n   Microsoft Edge Configuration:\n\n     Open a text editor (e.g., Notepad or VS Code).\n     Copy and paste the following content into the file:\n\n    bash\n    Windows Registry Editor Version 5.00\n    [HKEY_LOCAL_MACHINE\\Software\\Policies\\Microsoft\\Edge\\3rdparty\\extensions\\klpfkakgklajfdmgoacjfnbdboobekcn\\policy]\n    \"orgkey\"=\"YOUR_ORGANISATION_KEY_HERE\"\n    \"orgid\"=\"YOUR_ORGANISATION_ID_HERE\"\n    \n\n     Save the file as edge.reg.\n\nYou need to replace the placeholders with your actual Organization Key and Organization ID. Use the below steps to find them.\n\nStep 2.2: Find Your Organization ID and Organization Key\n\nTo find the necessary credentials:\n\n1. Log in to Feebak using your administrator credentials.\n2. Navigate to the Administration section in the sidebar.\n3. Go to Organisation Settings under the Organisation section.\n4. Your Organization ID and Organization Key will be displayed under the Organization Details tab.\n\nStep 2.3: Apply the Configuration\n\nAfter updating the configuration file:\n\n1. Save the file after replacing the placeholders.\n2. Run the file by double-clicking it to apply the changes to your system registry.\n3. Deployment options:\n    You can either run the registry file individually on each agent’s device.\n    Alternatively, you can deploy it as a group policy across all agent devices in your organization for streamlined configuration.\n\n{% hint style=\"info\" %}\nNote: Administrator permissions are required to run the registry file.\n{% endhint %}\n\n4. Restart your browser to ensure the changes take effect."
  },
  {
    "instruction": "**Step 3: Verify the Extension**",
    "input": "",
    "output": "To ensure the integration is working correctly:\n\n1. Open your browser and navigate to the Extensions settings.\n2. In the Feebak for CRM connectors extension settings, select Extension Options to verify the configuration."
  },
  {
    "instruction": "**Step 4: Test the Integration**",
    "input": "",
    "output": "Once configured, test the extension:\n\n1. Open Salesforce, or SAP C4C.\n2. Make a test call and disconnect the call.\n3. Verify that the Feebak extension prompts a survey automatically at the end of the interaction.\n\n<figure><img src=\"../.gitbook/assets/feebak-salesforce-extension (1).jpg\" alt=\"\"><figcaption></figcaption></figure>\n\nWith these steps completed, your Feebak for CRM connectors integration should be fully operational, enabling automatic surveys to be triggered at the end of customer interactions. This integration ensures seamless feedback collection directly within your CRM environment."
  },
  {
    "instruction": "Database Encryption",
    "input": "",
    "output": "{% hint style=\"info\" %}\nPrequisites:\n\nThe following permissions in Feebak are required to manage encryption keys in AWS KMS:\n\n EncryptionKeyManagement\\_View\n EncryptionKeyManagement\\_Add\n EncryptionKeyManagement\\_Delete\n\nTo enable database encryption for your organization, please contact Feebak Support at [support@feebak.com](mailto:support@feebak.com)\n{% endhint %}\n\nFeebak uses symmetric keys managed by AWS Key Management Service (KMS) to securely encrypt and decrypt sensitive data stored within the application. Feebak generates data keys from a primary symmetric key in KMS for encryption, and authorized decryption requests to AWS KMS provide secure access to this data. This guide covers prerequisites for AWS KMS encryption, setup steps, and key configuration for Feebak."
  },
  {
    "instruction": "Steps to Set Up Encryption with AWS KMS",
    "input": "",
    "output": "Step 1: Create AWS KMS Key\n\n1. Create a Customer-Managed Key\\\n   In your AWS account, create a customer-managed key specifically for Feebak.\n2. Set Key Type to Symmetric\\\n   Configure the key type as symmetric.\n3. Configure Key Usage\\\n   Set the key usage to \"Encrypt and decrypt\" to allow data encryption and decryption operations.\n4. Edit Key Policy & Specify Conditions\\\n   Update the Key Policy Statement to grant Feebak access using account ID 002525011236, to allow only requests originating from Feebak and to ensure that your Genesys Cloud Organization GUID is added.&#x20;\n\nSample Key Policy Statement:\n\njson\n{\n    \"Sid\": \"Allow key usage\",\n    \"Effect\": \"Allow\",\n    \"Principal\": {\n        \"AWS\": \"arn:aws:iam::002525011236:root\"\n    },\n    \"Action\": [\n        \"kms:Encrypt\",\n        \"kms:Decrypt\",\n        \"kms:GenerateDataKey\"\n    ],\n    \"Resource\": \"\",\n    \"Condition\": {\n        \"StringEquals\": {\n            \"kms:EncryptionContext:genesys-cloud-organization-id\": \"YOUR-GENESYS-ORGANIZATION-GUID\"\n        }\n    }\n},\n{\n    \"Sid\": \"Allow key validation\",\n    \"Effect\": \"Allow\",\n    \"Principal\": {\n        \"AWS\": \"arn:aws:iam::002525011236:root\"\n    },\n    \"Action\": \"kms:DescribeKey\",\n    \"Resource\": \"\"\n}\n\n\nStep 2: Configure AWS KMS Key Alias in Feebak\n\n1. Access Organization Settings\\\n   Navigate to the Feebak Organization Settings page.\n2. Select Key Configuration Type\\\n   In the Encryption Key Configuration section, set the Key Configuration Type to AWS KMS Symmetric.\n3. Enter ARN of KMS Key Alias\\\n   In the field labeled ARN of the AWS KMS key alias, for generating Customer Secured Data key, enter the ARN associated with your KMS key alias. The ARN format is arn:aws:kms:{region}:{accountId}:alias/{your-alias}, which can be located in the AWS KMS console.\n4. Save Configuration\\\n   Click Save to apply your encryption configuration.\n\n<figure><img src=\"../.gitbook/assets/encryption-ss.png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "Disabling Encryption",
    "input": "",
    "output": "1. Access Organization Settings\\\n   Navigate to the Feebak Organization Settings page.\n2. Select Key Configuration Type\\\n   In the Encryption Key Configuration section, set the Key Configuration Type to None.\n3. Save Configuration\\\n   Click Save to stop encrypting sensitive conversation data. This action removes removes your encryption key configuration for conversations.\n\n{% hint style=\"info\" %}\nNote: Save the old key for at least 31 days in AWS, so that existing customer secured conversation data can continue to be decrypted. Not doing so may result in permanent data loss, as data encrypted by these keys cannot be recovered without them.\n{% endhint %}"
  },
  {
    "instruction": "Important Notes",
    "input": "",
    "output": "_Critical_ - Impact of Lost Key Access: If access to an encryption key added in Feebak is lost, Feebak’s functionality will be disrupted until access is restored. This disruption can affect various operations within Feebak.\n Risk of Data Loss: If access to an encryption key is lost after it has been removed from Feebak and before the specified 31-day period, the data associated with that key will be permanently lost. Please ensure that key management is handled with care to avoid irreversible consequences.\n Search Functionality: If encryption is enabled for your organization, Feedback Text Search filter in the Interaction Dashboard may not function properly.\n Key Rotation: When rotating to a new encryption key, note that it takes up to 24 hours for the new key to take effect in Feebak.\n\nBy following these guidelines, you can ensure that sensitive data within Feebak is securely encrypted, in line with the highest data protection standards."
  },
  {
    "instruction": "Integrating Chat Notifications",
    "input": "",
    "output": "1. Once you are logged in, navigate to Admin in the navigation bar\n2. Select Integrations from the Integrations section\n3. Click the integrations button at the right corner to add a new app to Genesys Cloud\n\n![](../../.gitbook/assets/integration.PNG)\n\n4. Install the Generic Chat Notifications by clicking install in the respective section\n\n![](../../.gitbook/assets/install.png)\n\n5. Click Back to installed to view all the installed applications\n6. Select the installed application to view the Details and add Mappings\n\n![](<../../.gitbook/assets/integ- details.PNG>)\n\n7. Select Mappings to view the existing mappings and add a new mapping, click on the New Mapping button&#x20;\n\n![](<../../.gitbook/assets/New Mapping.PNG>)\n\n8. Enter the name and chat targets.\n\n{% hint style=\"info\" %}\nYou can find groups among the already-added ones or add new ones.\n{% endhint %}\n\nTo add a new group, refer to the following:\n\n{% content-ref url=\"../non-premium-installation/adding-a-group.md\" %}\n[adding-a-group.md](../non-premium-installation/adding-a-group.md)\n{% endcontent-ref %}\n\n9. Click the Delete button in the Filter section, as you don't have to add Filters to Mappings\n10. Finally, tap the Save button at the bottom to add the mapping"
  },
  {
    "instruction": "Premium Installation",
    "input": "",
    "output": "Feebak is a premium Appfoundry Partner and the integration with Genesys Cloud makes the Installation process seamless.\n\nTo embed the Feebak application inside the Genesys Cloud, follow the guide below:\n\n{% content-ref url=\"integrating-the-application.md\" %}\n[integrating-the-application.md](integrating-the-application.md)\n{% endcontent-ref %}\n\nTo send automated notifications to Genesys Cloud group chat rooms from Feebak, follow the guide below:\n\n{% content-ref url=\"integrating-chat-notifications.md\" %}\n[integrating-chat-notifications.md](integrating-chat-notifications.md)\n{% endcontent-ref %}"
  },
  {
    "instruction": "Integrating the Application",
    "input": "",
    "output": "### Install and Activate Feebak\n\n1. Ensure Admin Access\\\n   Make sure you are logged in with a Master Admin account that has access to all divisions in Genesys Cloud for a smooth installation process.\n2. Navigate to Admin Settings\\\n   Once logged in, navigate to the Admin section from the navigation bar.\n3. Access Integrations\\\n   In the Admin section, select Integrations. Then, click on the Add Integration button located in the top right corner of the screen to add a new app.\n4. Search for Feebak\\\n   In the search bar, type Feebak and select it from the list of results.\n5. Install Feebak Application\\\n   Click the Install button to begin the installation process for Feebak.\n\n<figure><img src=\"../../.gitbook/assets/image (226).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "Activate the Feebak Application",
    "input": "",
    "output": "1. Open Feebak Integration\\\n   After installing, the Feebak integration page should open automatically. If it doesn’t, go to Admin > Integrations and select Feebak from the list of installed integrations.\n\n<figure><img src=\"../../.gitbook/assets/Screenshot_20221229_165519.png\" alt=\"\"><figcaption></figcaption></figure>\n\n2. Activate Feebak\\\n   In the Feebak integration page, toggle the Activate button and then click Save to enable the application.\n3. Navigate to Feebak in Apps\\\n   After activation, go to the Apps section in the navigation bar and select Feebak.\n4. Start Feebak\\\n   A welcome screen will appear. Click on Start to begin the setup process.\n\n<figure><img src=\"../../.gitbook/assets/Screenshot_20221228_181505.png\" alt=\"\"><figcaption></figcaption></figure>\n\n5. Set Timezone and Install\\\n   Choose your organization's timezone and click Install. The installation process will take around 30 seconds.\n\n<figure><img src=\"../../.gitbook/assets/Screenshot_20221228_181514.png\" alt=\"\"><figcaption></figcaption></figure>\n\n6. Access Feebak Home\\\n   Once installation is complete, the Feebak home page will open, and the system will be ready for use.\n\n![](<../../.gitbook/assets/Screenshot 2024-10-07 200151.png>)"
  },
  {
    "instruction": "Integrating Chat Notifications",
    "input": "",
    "output": "1. Once you are logged in, navigate to Admin in the navigation bar\n2. Select Integrations from the Integrations section\n3. Click the Integrations button at the right corner to add a new app to Genesys Cloud\n\n![](../../.gitbook/assets/integration.PNG)\n\n4. Install the Generic Chat Notifications by clicking Install in the respective section\n\n![](../../.gitbook/assets/install.png)\n\n5. Click Back to Installed to view all the installed applications\n6. Select the installed application to view the Details and add Mappings\n\n![](<../../.gitbook/assets/integ- details.PNG>)\n\n7. Select Mappings to view the existing mappings and to add new mapping, click on the New Mapping button&#x20;\n\n![](<../../.gitbook/assets/New Mapping.PNG>)\n\n8. Enter the name and chat targets.\n\n{% hint style=\"info\" %}\nYou can find groups among the already-added ones or add new ones.\n{% endhint %}\n\n{% content-ref url=\"adding-a-group.md\" %}\n[adding-a-group.md](adding-a-group.md)\n{% endcontent-ref %}\n\nTo add a new group, refer to the following:\n\n9. Click the Delete button in the Filter section as you don't have to add Filters to Mappings\n10. Finally, tap the Save button at the bottom to add the mapping"
  },
  {
    "instruction": "Adding a Group",
    "input": "",
    "output": "1. Once you are logged in, navigate to Admin in the navigation bar\n2. Select Groups from the Directory section\n\n![](<../../.gitbook/assets/add group.PNG>)\n\n3. Click on the Add Group button and enter Name, Type and Visibility of the new group.\n4. Click on the Save Button at the bottom to save the New Group."
  },
  {
    "instruction": "Non-Premium Installation",
    "input": "",
    "output": "In a non-premium installation, all the configuration steps should be done manually by the admin.\n\nTo embed the Feebak application inside the Genesys Cloud CX, follow the guide below:\n\n{% content-ref url=\"generating-oauth-tokens.md\" %}\n[generating-oauth-tokens.md](generating-oauth-tokens.md)\n{% endcontent-ref %}\n\n{% content-ref url=\"adding-a-group.md\" %}\n[adding-a-group.md](adding-a-group.md)\n{% endcontent-ref %}\n\n{% content-ref url=\"integrating-the-application.md\" %}\n[integrating-the-application.md](integrating-the-application.md)\n{% endcontent-ref %}\n\nTo send automated notifications to Genesys Cloud group chat rooms from Feebak, follow the guide below:\n\n{% content-ref url=\"integrating-chat-notifications.md\" %}\n[integrating-chat-notifications.md](integrating-chat-notifications.md)\n{% endcontent-ref %}"
  },
  {
    "instruction": "Generating OAuth Tokens",
    "input": "",
    "output": "{% hint style=\"info\" %}\nThese steps are only required for non-premium organizations. For premium organizations, these will be added automatically during the installation\n{% endhint %}"
  },
  {
    "instruction": "Generating Client Credentials",
    "input": "",
    "output": "Before adding an OAuth client we need to add a role. This role we will later be assigned to the OAuth client.\n\n1\\. Once you are logged in, click on Admin in the navigation bar.\n\n2\\. Select Roles / Permissions from the People & Permissions section.\n\n![PureCloud Admin Portal](../../.gitbook/assets/OAuth.PNG)\n\n3\\. Click on Add Role button to add a new role.\n\n![](../../.gitbook/assets/add_role.PNG)\n\n4\\. Name the role as \"Feebak Client\".\n\n![](../../.gitbook/assets/role_permissions.PNG)\n\n5\\. Click on the Permissions tab and assign the permissions listed below:\n\n Architect > flow > Delete\n Architect > userPrompt > Add\n Architect > userPrompt > Delete\n Architect > userPrompt > Edit\n integrations > action > Add\n integrations > integration > Add\n integrations > integration > Edit\n Routing > Queue > Add\n Routing > Queue > View\n Routing > Wrap-up Code > Add\n Routing > CallRoute > View\n Routing > Call Route > Delete\n Telephony > Plugin > All\n Voicemail > All\n Outbound > Contact List > Add\n Outbound > Contact List > View\n Outbound > Contact List > Search\n Outbound > Response Set > All Permissions\n Conversation > Communication > All Permissions\n\n6\\. Click Save, the new roles will be seen added to the roles list.\n\n7\\.  To edit, click on the newly created \"Feebak Client\" role from the list.\n\n8\\. Click on the Change Membership button, assign it to the currently logged-in user and click Save.\\\n(User needs this role in order to assign it to the OAuth client)\n\n![](../../.gitbook/assets/assigntouser.PNG)\n\nNow, the role that should be assigned to the OAuth client is successfully setup. The next step is to add the actual OAuth client.\n\n1\\. Navigate to the Admin section in the navigation bar.\n\n2\\. Select OAuth from the Integrations section.\n\n3\\. Click on Add Client to add a new client.\n\n![](<../../.gitbook/assets/image (169).png>)\n\n4\\. Fill the form with the below data:\n\n| Field Name     | Field Data                       | Required? |\n| -------------- | -------------------------------- | --------- |\n| App Name       | Feebak OAuth Token               | Required  |\n| Description    | OAuth token to Connect to Feebak | Optional  |\n| Token Duration | 172800                           | Required  |\n| Grant Types    | Client Credentials               | Required  |\n| Roles          | Feebak Client                | Required  |\n\n![](../../.gitbook/assets/client.png)\n\n![](../../.gitbook/assets/client-roles.png)\n\nWhen \"Acknowledgement\" is prompted, Accept it.\n\nOn Sav&#x65;, Client ID and Client Secret are generated. These keys are used by feebak to communicate with Genesys Cloud.&#x20;"
  },
  {
    "instruction": "Generating Token Implicit Grant",
    "input": "",
    "output": "This token is required for login with Genesys Cloud feature inside feebak.\n\nLike the previous guide, click Add Client button from the OAuth list in Integrations\n\nFill the form with the following data\n\n| Field Name      | Field Data                                                                                                                                                                                                                                                                                                                                                                           | Required? |\n| --------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ | --------- |\n| App Name        | Implicit Grant                                                                                                                                                                                                                                                                                                                                                                       | Required  |\n| Description     | Token for Genesys Cloud Login with Feebak                                                                                                                                                                                                                                                                                                                                            | Optional  |\n| Token Duration  | 172800                                                                                                                                                                                                                                                                                                                                                                               | Required  |\n| Grant Types     | Token Implicit Grant (Browser)                                                                                                                                                                                                                                                                                                                                                       | Required  |\n| Authorized URLs | <p> <a href=\"https://app.feebak.com/login\">https://app.feebak.com/login</a></p><p> <a href=\"https://app.feebak.com/login/agent\">https://app.feebak.com/login/agent</a></p><p> <a href=\"https://app.feebak.com/register\">https://app.feebak.com/register</a></p><p> <a href=\"https://app.feebak.com/static/scriptwidget.html\">https://app.feebak.com/static/scriptwidget.html</a></p> | Required  |\n| Scope           | <p>architect</p><p>integrations</p><p>routing</p><p>telephony<br>alerting<br>authorization </p><p>conversations </p><p>notifications<br>oauth </p><p>organization:readonly<br>outbound</p><p>groups<br>voicemail</p>                                                                                                                                                                 | Required  |\n\nOn Sav&#x65;, Client ID and Client Secret are generated. We will use these keys in a later stage during the integration of feebak inside the Genesys Cloud interface.\n\n![](../../.gitbook/assets/grant.png)"
  },
  {
    "instruction": "Integrating the Application",
    "input": "",
    "output": "There are two ways to embed feebak inside Genesys Cloud interface\n\n1. Once you are logged in, navigate to Admin in the navigation bar\n2. Select Integrations from the Integrations section\n3. Click the integrations button at the right corner to add a new app to Genesys Cloud\n\n As a standalone application (which can be accessed from performance menu in Genesys Cloud)\n As a widget, which can be accessed from left side-bar menu, inside the Apps section\n\nBefore integrating the application as a standalone or widget, ensure that you have added the group (Users who have access to the application). To know more about adding a group, refer to the guide below:\n\n{% content-ref url=\"adding-a-group.md\" %}\n[adding-a-group.md](adding-a-group.md)\n{% endcontent-ref %}\n\n![](../../.gitbook/assets/integration.PNG)\n\n4. Install the Custom Client Application by clicking install in the respective section\n\n![](<../../.gitbook/assets/image (271).png>)\n\n5. Click Back to installed to view all the installed applications"
  },
  {
    "instruction": "Integrating as Standalone Application",
    "input": "",
    "output": "![](<../../.gitbook/assets/image (159).png>)\n\n1. Select the installed application and enter the Details and Configuration\n\ni) Enter the details and tap save\n\nii) Configuration section allows you to add properties\n\nAdd properties with the below details:\n\n| Property Name          | Value                                                                                                                                                                                      | Remarks                                                                                                                                                                                      |\n| ---------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |\n| Application URL        | https://app.feebak.com/clientapp/\\{{pcEnvironment\\}}/Replace_With_Implicit_Grant_Id or https://app.feebak.me/clientapp/\\{{pcEnvironment\\}}/Replace_With_Implicit_Grant_Id  | The address of application to be embed. Note: make sure you use .com if you want to install Feebak in the EU region and .me if you want to install Feebak in the Middle East region. |\n| Application Type       | Standalone                                                                                                                                                                                 | Dictates the way application will appear                                                                                                                                                     |\n| Application Category   | (optional) contactCenterInsights                                                                                                                                                           | To decide from where you want to access feebak (contactCenterInsights-displays the app inside performance section)                                                                           |\n| Iframe Sandbox Options | allow-scripts,allow-same-origin-allow-forms,allow-modals,allow-popups,allow-downloads                                                                                                      | All the Iframe permission required to run feebak efficiently                                                                                                                                 |\n| Group Filtering        | Select the groups of users for whom access to the application is granted                                                                                                                   | only accounts from the selected Groups will see the Application in their PureCloud interface                                                                                                 |\n\nIn the Application URL, Replace_With_Implicit_Grant_Id should be replaced with Implicit Grant Token generated before. To know more about generating Tokens, Follow the  guide below:\n\n{% content-ref url=\"generating-oauth-tokens.md\" %}\n[generating-oauth-tokens.md](generating-oauth-tokens.md)\n{% endcontent-ref %}\n\n&#x20;\n\n{% hint style=\"info\" %}\nThe application can be accessed from the performance menu&#x20;\n{% endhint %}\n\n![](<../../.gitbook/assets/image (154).png>)"
  },
  {
    "instruction": "Integrating as Widget Application",
    "input": "",
    "output": "![](<../../.gitbook/assets/image (193).png>)\n\n1. Select the installed application and enter the Details and Configuration\n\ni) Enter the details and tap save\n\nii) Configuration section allows you to add properties\n\nAdd properties with the below details:\n\n| Property Name          | Value                                                                                                                                                                                      | Remarks                                                                                                                                                                                      |\n| ---------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |\n| Application URL        | https://app.feebak.com/clientapp/\\{{pcEnvironment\\}}/Replace_With_Implicit_Grant_Id or https://app.feebak.me/clientapp/\\{{pcEnvironment\\}}/Replace_With_Implicit_Grant_Id  | The address of application to be embed. Note: make sure you use .com if you want to install Feebak in the EU region and .me if you want to install Feebak in the Middle East region. |\n| Application Type       | Widget                                                                                                                                                                                     | Dictates the way application will appear                                                                                                                                                     |\n| Application Category   | (optional)                                                                                                                                                                                 | To decide from where you want to access feebak                                                                                                                                               |\n| Iframe Sandbox Options | allow-scripts,allow-same-origin-allow-forms,allow-modals                                                                                                                                   | All the Iframe permission required to run feebak efficiently                                                                                                                                 |\n| Group Filtering        | Select the groups of users for whom access to the application is granted                                                                                                                   | Only accounts from the selected Groups will see the Application in their Genesys Cloud interface                                                                                             |\n|                        |                                                                                                                                                                                            |                                                                                                                                                                                              |\n\nIn the Application URL, Replace_With_Implicit_Grant_Id should be replaced with Implicit Grant Token generated before. To know more about generating Tokens, Follow the  guide below:\n\n{% content-ref url=\"generating-oauth-tokens.md\" %}\n[generating-oauth-tokens.md](generating-oauth-tokens.md)\n{% endcontent-ref %}\n\n{% hint style=\"info\" %}\nClick the app's icon on the sidebar to access the application\n{% endhint %}\n\n![](<../../.gitbook/assets/image (105).png>)"
  },
  {
    "instruction": "API Keys",
    "input": "",
    "output": "API Keys are essential for authenticating requests when using the Interaction Data Export API. Each API Key is a combination of the Key Id and Key Secret, which is required for secure API requests. Organizations can generate and manage upto 100 API Keys."
  },
  {
    "instruction": "Generate API Keys",
    "input": "",
    "output": "1. After logging in, navigate to API Keys under the Organisation section in the navigation bar. You will be directed to the API Keys page where existing keys are listed. You can view API Key details including the Name, Key Id, Creation Date and Status here.\n2. On the API Keys page, click the Add New button. Then enter the Name and Description for the API key.\n3. Click Save.\n4. To view the Key Id and Key Secret, click on the Edit button on the key. You can copy the Key Id and Key Secret from here.\n\n{% hint style=\"warning\" %}\nWarning:\\\nDo not share your API Key Secret with anyone or post it on public platforms to avoid security risks.\n{% endhint %}"
  },
  {
    "instruction": "**Regenerate API Key Secret**",
    "input": "",
    "output": "If you need to regenerate an API Key Secret:\n\n1. After logging in, naviagte to API Keys under the Organisation section in the navigation bar. You will be directed to the API Keys page where existing keys are listed.\n2. On the API Keys page, identify the API Key for which you want to regenerate the Key Secret.\n3.  Click the Edit button next to the relevant API Key. Then click on the Regenerate Key button next to the Key Secret. A new Key Secret will be generated, but the Key Id remains the same.\\\n\n\n    <figure><img src=\"../.gitbook/assets/api-keys.png\" alt=\"\"><figcaption></figcaption></figure>\n4. Copy and store the new Key Secret securely. Once regenerated, the old key will no longer be valid."
  },
  {
    "instruction": "Delete API Keys",
    "input": "",
    "output": "If you need to delete an API Key:\n\n1. After logging in, go to API Keys under the Organisation section in the navigation bar. You will be directed to the API Keys page where existing keys are listed.\n2. On the API Keys page, identify the API Key you want to delete.\n3.  Click the Delete button next to the relevant API Key. Confirm the deletion prompt to permanently remove the API Key from your organization.\\\n\n\n    <figure><img src=\"../.gitbook/assets/image (379).png\" alt=\"\"><figcaption></figcaption></figure>\n4. Once deleted, the API Key will no longer be valid for making API requests. Ensure that any applications using the deleted key are updated with an active key. However, if you want to restore this key, you can do that within 7 days after deleting."
  },
  {
    "instruction": "Restore API Keys",
    "input": "",
    "output": "{% hint style=\"info\" %}\nNote: You can only restore an API Key within 7 days after the deletion is initiated.\n{% endhint %}\n\nIf you need to restore a previously deleted API Key:\n\n1. After logging in, go to API Keys under the Organisation section in the navigation bar. You will be directed to the API Keys page where existing keys are listed.\n2. On the API Keys page, identify the Deleted Key you want to restore.\n3.  Click the Edit button next to the relevant API Key. Click the Restore button from the popup. The key will be reactivated, allowing it to be used for API requests again.\\\n\n\n    <figure><img src=\"../.gitbook/assets/api-keys (1).png\" alt=\"\"><figcaption></figcaption></figure>\n4. Ensure you copy and securely store the restored Key Secret, as it will be needed for future requests."
  },
  {
    "instruction": "Rate Limits",
    "input": "",
    "output": "### Understanding Rate Limits\n\nRate limits are thresholds set on API endpoints to control the volume of requests that a client can make within a specified period, typically measured in requests per minute (RPM) or requests per second (RPS). These limits help maintain the performance, availability, and reliability of Feebak's services, ensuring that all users have fair access. By defining the maximum number of requests allowed, rate limits prevent abuse, manage unexpected traffic patterns, and encourage efficient use of resources, ensuring accuracy and responsiveness. They also promote more thoughtful interaction with the API, as users are encouraged to consider how they utilize the available resources. While most rate limits are published and regularly updated alongside new features, Feebak reserves the right to adjust existing limits, introduce new ones, or enforce undocumented limits when deemed necessary to protect the system and its users. This approach allows Feebak to maintain flexibility in managing its services and ensures that the platform remains robust and equitable for all users."
  },
  {
    "instruction": "Handling Rate Limits Gracefully",
    "input": "",
    "output": "To handle rate limits effectively, it's crucial to implement strategies that ensure your application behaves correctly when limits are reached. The most basic approach is to monitor for HTTP 429 status codes, which indicate that the rate limit has been exceeded, and implement a retry mechanism. Identifying rate-limited requests is straightforward: the HTTP response code will be 429, and the response will contain a Retry-After header. It's important to note that while HTTP headers are case-insensitive, many SDKs store them in case-sensitive map structures, so accessing the Retry-After header may require using the exact case.\n\nThe retry mechanism should follow an exponential backoff schedule, which gradually increases the time between retries. Adding randomness to the backoff schedule can help prevent a thundering herd effect, where multiple retries occur simultaneously, potentially overwhelming the server. When implementing retries, applications should respect the Retry-After header, which indicates the number of seconds to wait before making another request. This header is included in all 429 responses and certain 5xx-level responses under specific circumstances. Requests without this header should not be automatically retried.\n\nOptimizing individual requests can only go so far. A more advanced approach is to manage request traffic at a global level by throttling requests when substantial rate limiting is detected. One effective method is implementing a token bucket rate limiting algorithm on the client-side. This algorithm allows requests to be made at a controlled rate and slows down or pauses when limits are reached. Ready-made implementations for token bucket algorithms are available in most programming languages. By combining these strategies - monitoring for 429 responses, respecting the Retry-After header, implementing exponential backoff with jitter, and using client-side rate limiting - applications can handle rate limits gracefully and maintain efficient interaction with the API."
  },
  {
    "instruction": "Available Limits",
    "input": "",
    "output": "| API Route                                                                                      | Rate Limit | Interval |\n| ---------------------------------------------------------------------------------------------- | ---------- | -------- |\n| [Digital Survey API (version 1)](digital-survey-api/digital-survey-api-version-1.md)           | 300        | 1 minute |\n| [Digital Survey API (version 2)](digital-survey-api/digital-survey-api-version-2.md)           | 300        | 1 minute |\n| [Digital Survey API (version 3)](digital-survey-api/digital-survey-api-version-3.md)           | 300        | 1 minute |\n| [Interaction Export API (version 1)](interaction-export-api/interaction-data-export.md)        | 100        | 1 minute |\n| [Interaction Export API (version 2)](interaction-export-api/interaction-data-export-1.md)      | 100        | 1 minute |\n| [Get Survey Form API](survey-api/reference.md#version-surveys-forms-linkid)                    | 300        | 1 minute |\n| [Submit Survey Form Answers API](survey-api/reference.md#version-surveys-forms-linkid-answers) | 300        | 1 minute |\n| [Submit Survey Answer API](survey-api/reference.md#version-surveys-forms-linkid-answer)        | 300        | 1 minute |\n| [Execute Questions Rules API](survey-api/reference.md#version-surveys-forms-linkid-action)     | 20         | 1 hour   |"
  },
  {
    "instruction": "Feebak Regions",
    "input": "",
    "output": "### What is a Feebak Region?\n\nA Feebak region refers to the specific geographical area where Feebak's servers are located and where your data is processed when using Feebak's services. The region determines the Base URL you should use for API requests to ensure optimal performance and compliance with regional data regulations. Feebak currently operates in two regions: the Middle East and Europe."
  },
  {
    "instruction": "What’s the Difference Between Feebak Region and Genesys Cloud Region?",
    "input": "",
    "output": "While Feebak regions are specific to where Feebak’s infrastructure is hosted, Genesys Cloud regions refer to the data centers where Genesys Cloud services are provided. Your Genesys Cloud region determines where your customer interaction data is stored and processed within the Genesys platform. However, the Feebak region is independent of your Genesys Cloud region and is solely related to where Feebak handles survey-related data. It’s important to correctly identify your Feebak region to ensure that your API requests are routed properly."
  },
  {
    "instruction": "How to Get Your Base URL",
    "input": "",
    "output": "To determine your Feebak Base URL, follow these steps in Genesys Cloud:\n\n1. Log in to Genesys Cloud.\n2. Navigate to the Admin Section.\n3. Go to Integrations.\n4. Search for Feebak: Use the search bar to find the \"Feebak\" integration.\n5. Select Feebak and Open the Configuration Tab.\n6. Find Application URL: Locate the Application URL in Configuration Properties.\n7. Identify Your Base URL: Determine whether the URL is app.feebak.com (Europe) or app.feebak.me (Middle East). This is the Base URL to use for your API requests.\n\n<figure><img src=\"../.gitbook/assets/image (363).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "Reference",
    "input": "",
    "output": "{% swagger src=\"../../.gitbook/assets/sample (22).yaml\" path=\"/{version}/surveys/forms/{linkId}\" method=\"get\" %}\n[sample (22).yaml](<../../.gitbook/assets/sample (22).yaml>)\n{% endswagger %}\n\n{% swagger src=\"../../.gitbook/assets/sample (22).yaml\" path=\"/{version}/surveys/forms/{linkId}/answers\" method=\"put\" %}\n[sample (22).yaml](<../../.gitbook/assets/sample (22).yaml>)\n{% endswagger %}\n\n{% swagger src=\"../../.gitbook/assets/sample (22).yaml\" path=\"/{version}/surveys/forms/{linkId}/answer\" method=\"put\" %}\n[sample (22).yaml](<../../.gitbook/assets/sample (22).yaml>)\n{% endswagger %}\n\n{% swagger src=\"../../.gitbook/assets/sample (22).yaml\" path=\"/{version}/surveys/forms/{linkId}/action\" method=\"post\" %}\n[sample (22).yaml](<../../.gitbook/assets/sample (22).yaml>)\n{% endswagger %}"
  },
  {
    "instruction": "Survey API",
    "input": "",
    "output": "The Survey API can be used to create a fully custom survey interface by providing APIs to submit responses, execute preset rules, and retrieve survey details for surveys already created on Feebak."
  },
  {
    "instruction": "Features",
    "input": "",
    "output": "The Survey API provides the following features:\n\n1. Get Survey Form: Retrieve a comprehensive overview of the survey form's structure, including its theme, questions, answer options, rules, and actions.\n2. Submit Survey Form: Save the answers of all the questions in the survey form.\n3. Submit Survey Answer: Save the answer of a single question in the survey form.\n4. Execute Questions Rules: Execute rules based on the survey questions and responses, allowing actions such as Send Email, Send Notification, and Invoke API actions."
  },
  {
    "instruction": "Query Parameters",
    "input": "",
    "output": "version: This is the version of Survey API you want to use. Current version is v1.0\n   linkId: A unique identifier generated for each survey. It can be obtained from the survey URL generated by [Digital Survey API](../digital-survey-api/). This parameter is required for calling all the API listed below.\n\n    For example, in the below sample survey URL, \"faqC8Q2\" is the linkId\n\n\nhttps://app.feebak.com/survey/fqgC8Q2"
  },
  {
    "instruction": "Rate Limiting",
    "input": "",
    "output": "Below are the rate limits for each of the API routes, measured in requests per minute:\n\n| API Route                      | Rate Limit (Requests per Minute) |\n| ------------------------------ | -------------------------------- |\n| Get Survey Form API            | 300                              |\n| Submit Survey Form Answers API | 300                              |\n| Submit Survey Answer API       | 300                              |\n| Execute Questions Rules API    | 20                               |\n\nFor more details about rate limits, [click here](../rate-limits.md)."
  },
  {
    "instruction": "Base URLs",
    "input": "",
    "output": "We have two base URLs for different regions:\n\n| Region      | Base URL                 |\n| ----------- | ------------------------ |\n| Middle East | https://app.feebak.me  |\n| Europe      | https://app.feebak.com |\n\nFor more details about your Feebak base URL, [click here](../feebak-regions.md)."
  },
  {
    "instruction": "Errors",
    "input": "",
    "output": "The Survey API uses conventional HTTP response codes to indicate the success or failure of an API request. In general, codes in the 2xx range indicate success, codes in the 4xx range indicate an error that failed given the information provided (e.g., a required parameter was omitted, a resource does not exist, etc.), and codes in the 5xx range indicate an error with Feebak's servers.\n\nFor detailed information on all the endpoints, please refer to the [Reference ](reference.md)page."
  },
  {
    "instruction": "Reference",
    "input": "",
    "output": "{% swagger src=\"../../.gitbook/assets/data-action-performance-api.yaml\" path=\"/v1/dataexport/dataactions/log?organisationid=\" method=\"post\" %}\n[data-action-performance-api.yaml](../../.gitbook/assets/data-action-performance-api.yaml)\n{% endswagger %}"
  },
  {
    "instruction": "Data Action Performance API",
    "input": "",
    "output": "The Data Action Performance API provides access to detailed logs for monitoring and analyzing the performance of your data actions. It uses the POST method, accepts JSON-encoded request bodies, returns JSON-encoded responses, and follows standard HTTP response codes.\n\nNone of the data action logs will be received if there are validation errors or an API failure. Ensure all validation errors are resolved to access the logs."
  },
  {
    "instruction": "Features",
    "input": "",
    "output": "The Data Action Performance API offers the following capabilities:\n\n Monitor Data Actions: Access detailed logs to track data action performance.\n Error Analysis: Diagnose and resolve validation errors by reviewing API responses.\n Performance Optimization: Use logs to identify areas for improving data action efficiency."
  },
  {
    "instruction": "Base URLs",
    "input": "",
    "output": "We have two base URLs for different regions:\n\n| Region      | Base URL                 |\n| ----------- | ------------------------ |\n| Middle East | https://app.feebak.me  |\n| Europe      | https://app.feebak.com |\n\nFor more details about your Feebak base URL, [click here](../feebak-regions.md)."
  },
  {
    "instruction": "Authentication - API Keys",
    "input": "",
    "output": "The Interaction Data Export  API uses API Keys to authenticate requests. For more details about Feebak API Keys, [click here](../api-keys.md)."
  },
  {
    "instruction": "Interaction Export API (version 1 - deprecated)",
    "input": "",
    "output": "{% hint style=\"warning\" %}\nEffective from March 17 2025, Interaction Export API (version 1) will be deprecated. Please use [Interaction Export API (version 2)](interaction-data-export-1.md) for exporting interaction data from Feebak. See Migration Guide.\n{% endhint %}\n\nThe Interaction Data Export  API uses [Feebak API Keys](./#authentication-api-keys) to authenticate requests."
  },
  {
    "instruction": "Interaction Export API V1",
    "input": "",
    "output": "The Interaction data export API uses POST method, it accepts [JSON-encoded](http://www.json.org/) request bodies, returns [JSON-encoded](http://www.json.org/) responses, and uses standard HTTP response codes.\n\nHTTP Method: POST\n\nURL: [/v1/dataexport/interactions?organisationId=](https://app.feebak.com/v1/dataexport/interactions?organisationId=){YOUR\\__ORG\\__&#x49;D}"
  },
  {
    "instruction": "Authorization",
    "input": "",
    "output": "The Interaction data export API uses Basic Authorization. The Basic authorization requires APIKey Id and APIKey Secret"
  },
  {
    "instruction": "Body",
    "input": "",
    "output": "json\n{\n    \"FromDate\": \"2023-06-23\",\n    \"ToDate\": \"2023-06-25\",\n    \"Start\": 0,\n    \"Limit\": 50,\n    \"SurveyState\": [\"Answered (Completed)\"],\n    \"Queueidentifiers\": [\"df317db4-c0c7-4af5-ba44-20d444596810\"],\n    \"Divisionidentifiers\": [\"b9836c7e-d925-49f5-a1d1-f640678d498f\"],\n    \"HasAnswers\": true\n}\n\n\n FromDate - (Date) Start date of the interactions to export ( Expected Timezone is UTC )\n ToDate - (Date) End date of the interactions to export ( Expected Timezone is UTC )\n Start - (Integer) Pagination start count (less than Total Count property in the response)\n Limit - (Integer) Pagination records per request (maximum 100 records per request, less than Total Count property in the response)\n Completed - (Deprecated) This variable is deprecated. Use \"SurveyState\" instead.\n SurveyState - (String) List of state of the surveys to be returned. Accepts the following values:\n   Answered (Completed): Surveys where all questions or required sections are answered.\n   Answered (Partial): Surveys with at least one answered question, but not fully completed.\n   Abandoned: Surveys that remain incomplete due to the customer not fully answering them (partial survey feature disabled).\n   Expired: Surveys not accessed before the specified expiration time.\n   Active: Surveys not yet opened and not expired.\n Queueidentifiers - (Optional) List of GUID of queues in Genesys Cloud (Admin > Queues). Select a particular queue. Queue Id will be in the URL of the page. [https://apps.mypurecloud.com/directory/#/admin/organization/queues/{queueidentifer}/general](https://apps.aps1.pure.cloud/directory/#/admin/organization/queues/e6ad3e55-b127-4665-8b2a-2335786ed766/general)\n Divisionidentifiers - (Optional) List of GUID of divisions in Genesys Cloud (Admin > Divisions). Select a particular division. Division Id will be in the URL of the page.\\\n  [https://apps.aps1.pure.cloud/directory/#/admin/people-and-permissions/divisions/{divisionIdentifier}/members](https://apps.aps1.pure.cloud/directory/#/admin/people-and-permissions/divisions/c31d545d-6bda-4181-943a-7eed9dcb8b89/members)\n HasAnswers - (Optional, Boolean) The HasAnswers variable determines whether the API should return surveys with at least one answer. If the set to true, the API will return surveys with at least one answer. Conversely, if it set to false, the API will return surveys with no answers. This variable is dependent on the Completed variable; when Completed is true, and HasAnswers is false, no surveys will be listed. When Completed is true, and HasAnswers is true, there will be no effect. When the Completed variable is set to false and the HasAnswers variable is set to false, only unanswered surveys will be listed. Conversely, when the Completed variable is false and the HasAnswers variable is set to true, only partially answered surveys will be listed.\\\n  (Please note: To utilize this feature effectively, ensure that the partial survey functionality is enabled.)"
  },
  {
    "instruction": "Example Response",
    "input": "",
    "output": "python\n{\n    \"LocalizedMessage\": \"Success\",\n    \"TotalCount\": 1,\n    \"Success\": false,\n    \"Data\": [\n        {\n            \"ConversationID\": \"7793a7ce-d900-49f4-8059-d8414beeb1d8\",\n            \"CallerAddress\": null,\n            \"LanguageText\": \"English\",\n            \"SurveyScore\": 68.0,\n            \"AgentScore\": 75.0,\n            \"VoiceFeedbackText\": null,\n            \"FeedbackText\": null,\n            \"Answered\": true,\n            \"Type\": \"Facebook\",\n            \"Direction\": \"inbound\",\n            \"SurveyStartTime\": \"2023-06-24T06:52:19.264594\",\n            \"SurveyEndTime\": \"2023-06-24T06:52:51.466418\",\n            \"AddedDate\": \"2023-06-24T06:51:42.301\",\n            \"CustomerID\": \"5804404099638645\",\n            \"CustomerName\": \"അർജുൻ മോഹൻദാസ്\",\n            \"AgentName\": \"Farshan\",\n            \"AgentEmail\": \"farshan@fantacode.com\",\n            \"QueueName\": \"Feebak_Helpdesk - A name change\",\n            \"SurveyName\": \"Multiple type multiple language staging\",\n            \"SurveyScoreLevel\": \"Good\",\n            \"SurveyID\": 1266,\n            \"AgentIdentifier\": null,\n            \"QueueIdentifier\": \"df317db4-c0c7-4af5-ba44-20d444596810\",\n            \"QueueWaitingTime\": 2,\n            \"IvrHandleTime\": null,\n            \"AgentConnectedTime\": 7,\n            \"Answers\": [\n                {\n                    \"QuestionID\": 2380,\n                    \"Answer\": \"4\",\n                    \"AnswerScore\": 75,\n                    \"AnswerID\": 3772131,\n                    \"QuestionName\": \"CSAT\",\n                    \"LevelName\": \"Satisfied\",\n                    \"SurveyID\": 0,\n                    \"SurveyName\": null,\n                    \"AddedDate\": \"2023-06-24T06:52:51.466861\",\n                    \"CallDate\": \"2023-06-24T06:51:42.301\",\n                    \"CallID\": 15280327\n                },\n                {\n                    \"QuestionID\": 2381,\n                    \"Answer\": \"6\",\n                    \"AnswerScore\": 60,\n                    \"AnswerID\": 3772132,\n                    \"QuestionName\": \"NPS\",\n                    \"LevelName\": \"Detractors\",\n                    \"SurveyID\": 0,\n                    \"SurveyName\": null,\n                    \"AddedDate\": \"2023-06-24T06:52:51.466861\",\n                    \"CallDate\": \"2023-06-24T06:51:42.301\",\n                    \"CallID\": 15280327\n                }\n            ],\n            \"ExternalContactId\": \"c0896b29-d9a5-4e18-b374-da17c8c54ed2\",\n            \"CustomConversationID\": null,\n            \"LinkId\": \"Q0z14ej\",\n            \"SurveyState\": \"Answered (Completed)\"\n        }\n    ],\n    \"Status\": \"EXA100\",\n    \"Message\": \"Success\"\n}"
  },
  {
    "instruction": "Errors",
    "input": "",
    "output": "Interaction data export API uses conventional HTTP response codes to indicate the success or failure of an API request. In general: Codes in the 2xx range indicate success. Codes in the 4xx range indicate an error that failed given the information provided (e.g., a required parameter was omitted, a charge failed, etc.).&#x20;\n\n<table data-header-hidden><thead><tr><th width=\"248\"></th><th></th></tr></thead><tbody><tr><td><strong>Status code</strong></td><td><strong>Implication</strong></td></tr><tr><td>200 - OK</td><td>Everything worked as expected</td></tr><tr><td>400 - Bad Request</td><td>The request was unacceptable, often due to missing a required parameter.</td></tr><tr><td>401 - Unauthorized</td><td>Unable to authenticate user</td></tr><tr><td>402 - Request Failed</td><td>The parameters were valid but the request failed.</td></tr><tr><td>403 - Forbidden</td><td>The API key doesn't have permissions to perform the request.</td></tr><tr><td>404 - Not Found</td><td>The requested resource doesn't exist.</td></tr></tbody></table>"
  },
  {
    "instruction": "Migration Guide for Interaction Export API",
    "input": "",
    "output": "### Migration Steps"
  },
  {
    "instruction": "Step 1: Identify Your Current Usage",
    "input": "",
    "output": "Review your current implementation of the Interaction Export API (Version 1).\n Locate where you are using the \"Start\" and \"Limit\" parameters to control data retrieval."
  },
  {
    "instruction": "Step 2: Update to Version 2",
    "input": "",
    "output": "Update your API calls to use the Interaction Export API (Version 2) endpoint: /v2/dataexport/interactions?organisationId={your\\_org\\_id}\\&limit={limit}\n Set the limit for the number of records you want to retrieve. (max: 100)"
  },
  {
    "instruction": "Step 3: Implement Cursor Pagination",
    "input": "",
    "output": "After making the initial API call, check the response for the PreviousLink and NextLink.\n Use these links to navigate through the records. Continue using the NextLink until it returns null, indicating that all records have been retrieved."
  },
  {
    "instruction": "Step 4: Test and Validate",
    "input": "",
    "output": "Thoroughly test your updated implementation to ensure seamless data retrieval."
  },
  {
    "instruction": "Summary of Changes",
    "input": "",
    "output": "Version 1: Relied on \"Start\" and \"Limit\" parameters for pagination. You would manually adjust the \"Start\" value using the TotalCount in the response to navigate through pages of data. This approach required calculating offsets to fetch subsequent records.\n Version 2: Introduces cursor pagination with PreviousLink and NextLink. Instead of manually setting start values, you navigate through records using these links. The API returns the first set of records along with NextLink and PreviousLink. You use these links to fetch the next or previous set of records, streamlining the process and reducing the complexity of data retrieval."
  },
  {
    "instruction": "Interaction Export API",
    "input": "",
    "output": "The Interaction Export API can be used to extract the interaction data. It accepts JSON-encoded request bodies, returns JSON-encoded responses, and uses standard HTTP response codes. The API can only be used to retrieve the interactions of the last 60 days from the current date."
  },
  {
    "instruction": "Features",
    "input": "",
    "output": "The Interaction Export API provides the following features:\n\n1. Extract Interaction Data: Retrieve interaction data for the past 60 days.\n2. Flexible Filtering: Filter interactions by date range, survey state, queue identifiers, and conversation IDs.\n3. Comprehensive Interaction Details: Access a wide range of information about each interaction, including customer and agent details, survey responses, and timing data.\n4. Survey Insights: Gather information about survey scores, states, and individual question responses.\n5. Pagination Support: Handle large datasets with cursor-based pagination.\n6. Multi-channel Support: Extract data from various interaction types such as email, chat, and voice."
  },
  {
    "instruction": "Versions",
    "input": "",
    "output": "These are the available versions of Interaction Export API:\n\n [Interaction Export API (version 1)](interaction-data-export.md)\n [Interaction Export API (version 2)](interaction-data-export-1.md)\\"
  },
  {
    "instruction": "Rate Limiting",
    "input": "",
    "output": "Rate limits for the Interaction Export API routes, measured in requests per minute:\n\n| API Route                                                          | Rate Limit (Requests per Minute) |\n| ------------------------------------------------------------------ | -------------------------------- |\n| [Interaction Export API (version 1)](interaction-data-export.md)   | 100                              |\n| [Interaction Export API (version 2)](interaction-data-export-1.md) | 100                              |\n\nFor more details about rate limits, [click here](../rate-limits.md)."
  },
  {
    "instruction": "Base URLs",
    "input": "",
    "output": "We have two base URLs for different regions:\n\n| Region      | Base URL                 |\n| ----------- | ------------------------ |\n| Middle East | https://app.feebak.me  |\n| Europe      | https://app.feebak.com |\n\nFor more details about your Feebak base URL, [click here](../feebak-regions.md)."
  },
  {
    "instruction": "Authentication - API Keys",
    "input": "",
    "output": "The Interaction Data Export  API uses API Keys to authenticate requests. For more details about Feebak API Keys, [click here](../api-keys.md)."
  },
  {
    "instruction": "Interaction Export API (version 2)",
    "input": "",
    "output": "The Interaction Data Export API uses [Feebak API Keys](./#authentication-api-keys) to authenticate requests."
  },
  {
    "instruction": "Interaction Export API V2",
    "input": "",
    "output": "The Interaction data export API uses POST method, it accepts [JSON-encoded](http://www.json.org/) request bodies, returns [JSON-encoded](http://www.json.org/) responses, and uses standard HTTP response codes. The API can only be used to retrieve the interactions of the last 60 days from the current date.\n\nHTTP Method: POST\n\nURL: [/v2/dataexport/interactions?organisationId=](https://app.feebak.com/v1/dataexport/interactions?organisationId=){your\\__org\\_id_}\\&limit={limit}"
  },
  {
    "instruction": "**Query Params**",
    "input": "",
    "output": "1. OrganisationID - ID of your Organisation&#x20;\n2. Limit - (Integer) Pagination records per request (maximum 100 records per request)"
  },
  {
    "instruction": "Pagination",
    "input": "",
    "output": "This API uses Cursor pagination. When you call the API with a limit of 100, it will return the first 100 records, along with a PreviousLink and a NextLink along with it. You can call the API again using the NextLink to retrieve the next 100 records, and you can keep repeating this until NextLink returned is null (which means there are no more records to retrieve). Similarly, you can use the PreviousLink to retrieve the previous 100 records."
  },
  {
    "instruction": "Authorization",
    "input": "",
    "output": "The Interaction data export API uses Basic Authorization. The Basic authorization requires APIKey Id and APIKey Secret"
  },
  {
    "instruction": "Body",
    "input": "",
    "output": "json\n{\n    \"FromDate\": \"2024-06-21\",\n    \"ToDate\": \"2024-06-25\",\n    \"SurveyState\": [\"Active\", \"Answered (Completed)\"],\n    \"Queueidentifiers\": [\"df317db4-c0c7-4af5-ba44-20d444596810\"],\n    \"Divisionidentifiers\": [\"b9836c7e-d925-49f5-a1d1-f640678d498f\"],\n    \"ConversationIds\": [\"b2fe5fef-344a-408c-a69a-11377bd49692\", \"54505908-7390-4455-8827-6aa8305b5440\"]\n}\n\n\n FromDate - (Date) Start date of the interactions to export ( Expected Timezone is UTC )\n ToDate - (Date) End date of the interactions to export ( Expected Timezone is UTC )\n Completed - (Deprecated) This variable is deprecated. Use \"SurveyState\" instead.\n SurveyState - (String, optional) List of state of the surveys to be returned. Accepts the following values:\n   Answered (Completed): Surveys where all questions or required sections are answered.\n   Answered (Partial): Surveys with at least one answered question, but not fully completed.\n   Abandoned: Surveys that remain incomplete due to the customer not fully answering them (partial survey feature disabled).\n   Expired: Surveys not accessed before the specified expiration time.\n   Active: Surveys not yet opened and not expired.\n Queueidentifiers - (Optional) List of GUID of queues in Genesys Cloud (Admin > Queues). Select a particular queue. Queue Id will be in the URL of the page.\\\n  [https://apps.mypurecloud.com/directory/#/admin/organization/queues/{queueidentifer}/general](https://apps.aps1.pure.cloud/directory/#/admin/organization/queues/e6ad3e55-b127-4665-8b2a-2335786ed766/general)\n Divisionidentifiers - (Optional) List of GUID of divisions in Genesys Cloud (Admin > Divisions). Select a particular division. Division Id will be in the URL of the page. \\\n  https://apps.aps1.pure.cloud/directory/#/admin/people-and-permissions/divisions/{divisionIdentifier}/members\n ConversationIds - (Optional) Filter by using the Genesys ConversationIDs (InteractionIDs)."
  },
  {
    "instruction": "Example Response",
    "input": "",
    "output": "python\n{\n    \"LocalizedMessage\": \"Success\",\n    \"Success\": false,\n    \"NextCursor\": null,\n    \"PreviousCursor\": \"NDU5MzYwNDQ=\",\n    \"NextLink\": null,\n    \"PreviousLink\": \"https://app.feebak.com/v2/dataexport/interactions?organisationId=21&limit=100&cursor=NDU5MzYwNDQ=&cursorDirection=previous\",\n    \"Data\": [\n        {\n            \"ConversationID\": \"b2fe5fef-344a-408c-a69a-11377bd49692\",\n            \"CallerAddress\": null,\n            \"LanguageText\": \"English\",\n            \"SurveyScore\": null,\n            \"AgentScore\": null,\n            \"VoiceFeedbackText\": null,\n            \"FeedbackText\": null,\n            \"Answered\": false,\n            \"Type\": \"Email\",\n            \"Direction\": \"inbound\",\n            \"SurveyStartTime\": null,\n            \"SurveyEndTime\": null,\n            \"AddedDate\": \"2024-06-24T05:22:07.593\",\n            \"CustomerID\": \"swetha.s@fantacode.com\",\n            \"CustomerName\": \"Swetha Sadanandan\",\n            \"AgentName\": \"Arshina\",\n            \"AgentEmail\": \"arshina.p@fantacode.com\",\n            \"QueueName\": \"Feebak_Helpdesk\",\n            \"SurveyName\": \"Prod Conversation Survey\",\n            \"SurveyScoreLevel\": null,\n            \"SurveyID\": 1508,\n            \"AgentIdentifier\": null,\n            \"QueueIdentifier\": \"df317db4-c0c7-4af5-ba44-20d444596810\",\n            \"QueueWaitingTime\": 4,\n            \"IvrHandleTime\": null,\n            \"AgentConnectedTime\": 6,\n            \"Answers\": [],\n            \"ExternalContactId\": \"12cc484d-da9f-413d-b22b-4904336701e0\",\n            \"CustomConversationID\": null,\n            \"LinkId\": \"kSm2GUP\",\n            \"SurveyState\": \"Active\"\n        },\n        {\n            \"ConversationID\": \"54505908-7390-4455-8827-6aa8305b5440\",\n            \"CallerAddress\": null,\n            \"LanguageText\": \"English\",\n            \"SurveyScore\": 78.0,\n            \"AgentScore\": 75.0,\n            \"VoiceFeedbackText\": null,\n            \"FeedbackText\": \"conversation survey\",\n            \"Answered\": true,\n            \"Type\": \"Whatsapp\",\n            \"Direction\": \"inbound\",\n            \"SurveyStartTime\": \"2024-06-22T10:58:59.396726\",\n            \"SurveyEndTime\": \"2024-06-22T10:59:34.40375\",\n            \"AddedDate\": \"2024-06-22T10:58:32.46\",\n            \"CustomerID\": \"919747148246\",\n            \"CustomerName\": \"arjunmohandas\",\n            \"AgentName\": \"Arshina\",\n            \"AgentEmail\": \"arshina.p@fantacode.com\",\n            \"QueueName\": \"Feebak_Helpdesk\",\n            \"SurveyName\": \"Prod Conversation Survey\",\n            \"SurveyScoreLevel\": \"Good\",\n            \"SurveyID\": 1508,\n            \"AgentIdentifier\": null,\n            \"QueueIdentifier\": \"df317db4-c0c7-4af5-ba44-20d444596810\",\n            \"QueueWaitingTime\": 2,\n            \"IvrHandleTime\": null,\n            \"AgentConnectedTime\": 10,\n            \"Answers\": [\n                {\n                    \"QuestionID\": 3543,\n                    \"Answer\": \"8\",\n                    \"AnswerScore\": 80,\n                    \"AnswerID\": 11271666,\n                    \"QuestionName\": \"Recent Purchase\",\n                    \"LevelName\": \"Passives\",\n                    \"SurveyID\": 0,\n                    \"SurveyName\": null,\n                    \"AddedDate\": \"2024-06-22T10:59:07.974628\",\n                    \"CallDate\": \"2024-06-22T10:58:32.46\",\n                    \"CallID\": 45819533\n                },\n                {\n                    \"QuestionID\": 3544,\n                    \"Answer\": \"4\",\n                    \"AnswerScore\": 75,\n                    \"AnswerID\": 11271667,\n                    \"QuestionName\": \"Company\",\n                    \"LevelName\": \"Satisfied\",\n                    \"SurveyID\": 0,\n                    \"SurveyName\": null,\n                    \"AddedDate\": \"2024-06-22T10:59:14.915093\",\n                    \"CallDate\": \"2024-06-22T10:58:32.46\",\n                    \"CallID\": 45819533\n                }\n            ],\n            \"ExternalContactId\": \"a3784ae3-6b51-42ac-960d-f0ad18f159cc\",\n            \"CustomConversationID\": null,\n            \"LinkId\": null,\n            \"SurveyState\": \"Answered (Completed)\"\n        }\n    ],\n    \"Status\": \"EXA100\",\n    \"Message\": \"Success\"\n}"
  },
  {
    "instruction": "Errors",
    "input": "",
    "output": "Interaction data export API uses conventional HTTP response codes to indicate the success or failure of an API request. In general: Codes in the 2xx range indicate success. Codes in the 4xx range indicate an error that failed given the information provided (e.g., a required parameter was omitted, a charge failed, etc.).&#x20;\n\n| Status code      | Implication                                                          |\n| -------------------- | ------------------------------------------------------------------------ |\n| 200 - OK             | Everything worked as expected                                            |\n| 400 - Bad Request    | The request was unacceptable, often due to missing a required parameter. |\n| 401 - Unauthorized   | Unable to authenticate user                                              |\n| 402 - Request Failed | The parameters were valid but the request failed.                        |\n| 403 - Forbidden      | The API key doesn't have permissions to perform the request.             |\n| 404 - Not Found      | The requested resource doesn't exist.                                    |"
  },
  {
    "instruction": "Digital Survey API (version 3)",
    "input": "",
    "output": "HTTP Method: POST\n\nURL: [/v3/websurvey/link](https://app.feebak.com/v2/websurvey/link)\n\nBody\n\nTo offer a survey for a Genesys conversation, you should pass the Genesys Conversation ID in the body, as shown below. In this case, the Survey offered will be the one assigned in the Survey Assignments.\n\njavascript\n{\n    \"ConversationID\": \"Genesys Conversation ID\",\n    \"SurveyID\": \"Feebak Survey ID\",\n    \"CustomerID\": \"customer email, phone or username\",\n    \"Language\": \"en-us\",\n    \"OrganisationID\": 1234,\n    \"OrganisationKey\": \"YOUR_FEEBAK_ORGKEY\",\n    \"ConversationType\": \"Webmessaging\",\n    \"Offered\" : true,\n    \"AgentIdentifier\" : \"Genesys Agent ID\",\n    \"QueueIdentifier\" : \"Genesys Queue ID\"\n}\n\n\n1. ConversationID: Genesys Conversation ID\n2. SurveyID(Optional): ID of the survey you want to offer. It can be found on the Survey Page in Feebak. Note that, SurveyID will only be considered for agentless interactions. The details such as ConversationID, and customer details will be stored with survey results.\n3. Language:&#x6C;anguage in which the survey should be offered.\n4. OrganisationKey: Every organisation has a unique key. It can be found on the [Administration > Organisation Settings Page](https://app.feebak.com/administration/organisationsettings) in Feebak.\n5. OrganisationID: ID of your organisation\n6. CustomerID(Optional): Customer email, phone, or username\n7. ConversationType (Optional): Possible values are _Call, Chat, Email, Webmessaging, Facebook,_ and _Whatsapp._ If no values are given, then Chat will be taken as the conversation type. To generate a survey link for call interactions, you must have at least one digital survey type enabled in the voice survey configuration.\n8. Offered (Optional, default: true) : Indicates whether to mark the survey as Offered when creating the survey link.\n9. AgentIdentifier (Optional): GUID of agent in Genesys Cloud (Admin > People). Select a particular agent. Agent Id will be in the URL of the page [https://apps.aps1.pure.cloud/directory/#/admin/directory/peopleV2/{agentidentifier}/roles](https://apps.aps1.pure.cloud/directory/#/admin/directory/peopleV2/4bced821-2f98-4d65-ac52-0f9ca5671065/roles). If there are multiple agents in an interaction, you can use this agent identifier to generate a survey link for a specific agent. We will store the history of agents involved in a conversation and validate whether the agent specified through the API was part of the conversation. If the validation succeeds, the survey will be offered. If the validation fails, an error will be thrown: 'Agent doesn't belong to the conversation'.\n10. QueueIdentifier (Optional): GUID of queue in Genesys Cloud (Admin > Queues). Select a particular queue. Queue Id will be there in the URL of the page. [https://apps.mypurecloud.com/directory/#/admin/organization/queues/{queueidentifer}/general](https://apps.aps1.pure.cloud/directory/#/admin/organization/queues/e6ad3e55-b127-4665-8b2a-2335786ed766/general). If there are multiple queues in an interaction, you can use this queue identifier to generate a survey link for a specific queue. We will store the history of queue involved in a conversation and validate whether the queue specified through the API was part of the conversation. If the validation succeeds, the survey will be offered. If the validation fails, an error will be thrown: 'Queue doesn't belong to the conversation'.\n\nResponse Messages\n\n| HTTP Status Code | Response            | Reason                             |\n| ---------------- | ------------------- | ---------------------------------- |\n| 200              | Success             | Survey Link Generated successfully |\n| 400              | BadRequest          | Invalid Input                      |\n| 401              | Unauthorized        | Unable to authenticate user        |\n| 404              | NotFound            | Invalid Organisation ID            |\n| 500              | InternalServerError | Unknown error occurred user        |"
  },
  {
    "instruction": "Offering Surveys without a Genesys Conversation",
    "input": "",
    "output": "To offer a survey directly without a Genesys conversation, you should pass the ID of the survey to be offered and a Reference ID to uniquely identify the interaction in dashboards and reference reports.\n\njson\n{\n    \"SurveyID\": 1234,\n    \"ReferenceID\": \"SOME_UNIQUE_ID\",\n    \"CustomerID\": \"customer email, phone or username\",\n    \"Language\": \"en-us\",\n    \"OrganisationID\": 1234,\n    \"OrganisationKey\": \"YOUR_FEEBAK_ORGKEY\",\n    \"ConversationType\": \"Whatsapp\",\n    \"Offered\" : true\n}\n\n\n1. SurveyID - ID of the survey you want to offer. It can be found on the Survey Page in Feebak.\n2. ReferenceID - You can pass an ID here, which can be used to uniquely identify this survey in dashboards and reports.\n\nResponse\n\njavascript\n{\n    \"Success\": true,\n    \"Data\": {\n        \"SurveyLink\": \"https://fantacode.feebak.com/survey/ccG8a68\",\n        \"DefaultPrompt\": \"Greetings, would you mind taking a survey?\"\n    },\n    \"Message\": null\n}\n\n\nIn the response, the survey link and the default prompt will be returned inside the Data as shown above.\n\n1. SurveyLink - Generated link to the digital survey\n2. DefaultPrompt - prompt added in the Default Text field while creating the survey.\\\n   ![](<../../.gitbook/assets/image (250).png>)\n\nResponse Messages\n\n| HTTP Status Code | Response            | Reason                             |\n| ---------------- | ------------------- | ---------------------------------- |\n| 200              | Success             | Survey Link Generated successfully |\n| 400              | BadRequest          | Invalid Input                      |\n| 401              | Unauthorized        | Unable to authenticate user        |\n| 404              | NotFound            | Invalid Organisation ID            |\n| 500              | InternalServerError | Unknown error occurred user        |"
  },
  {
    "instruction": "Mark Survey as Offered",
    "input": "",
    "output": "If you want to mark an already generated survey link as offered, you can use the following API:\n\nHTTP Method: PUT\n\nURL: [https://app.feebak.com/v3/websurvey/link/state?linkid=\"LINKID\"\\&surveystatus=Offered](https://app.feebak.com/v2/websurvey/link/state?linkid=%22LINKID%22\\&surveystatus=Offered)\\\n\\\nQuery parameters to be send in URL\n\nLinkID: A unique id generated against a survey, can be obtained from survey URL generated.\n\neg: Below is a sample survey URL\n\njson\nhttps://app.feebak.com/survey/fqgC8Q2\n\n\nIn above survey URL, \"fqgC8Q2\" is the link ID which you can use in the above API to mark this survey as offered.\n\nSurveyStatus - status of the survey which you want to update about the survey. Use the value \"Offered\" to mark the survey as offered. \\\n\\\nBody\n\njson\n{\n    \"OrganisationID\": 1234,\n    \"OrganisationKey\": \"YOUR_FEEBAK_ORGKEY\"\n}"
  },
  {
    "instruction": "Custom Conversation Type",
    "input": "",
    "output": "{% hint style=\"info\" %}\nNote: Custom conversation types can only be used to create a survey link without a Genesys conversation\n{% endhint %}\n\nCustom conversation types can be used when generating surveys via the Feebak Digital Survey API. When using the Feebak Digital Survey API for survey generation, specify the custom conversation type that corresponds to the interaction.\n\njson\n{\n    \"SurveyID\": 1234,\n    \"ReferenceID\": \"SOME_UNIQUE_ID\",\n    \"CustomerID\": \"customer email, phone or username\",\n    \"Language\": \"en-us\",\n    \"OrganisationID\": 1234,\n    \"OrganisationKey\": \"YOUR_FEEBAK_ORGKEY\",\n    \"ConversationType\": \"Kiosk\"\n}\n\n\nThis ensures that the survey is categorized correctly.\n\nYou can also add Custom Attributes (key, value) in the API request.&#x20;\n\n{% hint style=\"info\" %}\nNote: You can use these custom attributes only when using custom conversation types\n{% endhint %}\n\njson\n{\n    \"SurveyID\": 1234,\n    \"ReferenceID\": \"SOME_UNIQUE_ID\",\n    \"CustomerID\": \"customer email, phone or username\",\n    \"Language\": \"en-us\",\n    \"OrganisationID\": 1234,\n    \"OrganisationKey\": \"YOUR_FEEBAK_ORGKEY\",\n    \"ConversationType\": \"Kiosk\",\n    \"CustomAttributes\" : {\n        \"key\" : \"value\",\n        \"key1\" : \"value1\"\n        }\n}\n\n\nThe Custom Attributes will show in the interaction details\n\n<figure><img src=\"../../.gitbook/assets/image (292).png\" alt=\"\"><figcaption></figcaption></figure>\n\nYou can refer the document to know how to add Custom Conversation Types here: [Custom Conversation Type](../../introduction-to-feebak/custom-conversation-type.md)"
  },
  {
    "instruction": "Digital Survey API (version 2)",
    "input": "",
    "output": "HTTP Method: POST\n\nURL: [/v2/websurvey/link](https://app.feebak.com/v2/websurvey/link)\n\nBody\n\nTo offer a survey for a Genesys conversation, you should pass the Genesys Conversation ID in the body, as shown below. In this case, the Survey offered will be the one assigned in the Survey Assignments.\n\njavascript\n{\n    \"ConversationID\": \"Genesys Conversation ID\",\n    \"SurveyID\": \"Feebak Survey ID\",\n    \"CustomerID\": \"customer email, phone or username\",\n    \"Language\": \"en-us\",\n    \"OrganisationID\": 1234,\n    \"OrganisationKey\": \"YOUR_FEEBAK_ORGKEY\",\n    \"ConversationType\": \"Webmessaging\",\n    \"Offered\" : true,\n    \"AgentIdentifier\" : \"Genesys Agent ID\",\n    \"QueueIdentifier\" : \"Genesys Queue ID\"\n}\n\n\n1. ConversationID: Genesys Conversation ID\n2. SurveyID(Optional): ID of the survey you want to offer. It can be found on the Survey Page in Feebak. Note that, SurveyID will only be considered for agentless interactions. The details such as ConversationID, and customer details will be stored with survey results.\n3. Language:&#x6C;anguage in which the survey should be offered.\n4. OrganisationKey: Every organisation has a unique key. It can be found on the [Administration > Organisation Settings Page](https://app.feebak.com/administration/organisationsettings) in Feebak.\n5. OrganisationID: ID of your organisation\n6. CustomerID(Optional): Customer email, phone, or username\n7. ConversationType (Optional): Possible values are _Call, Chat, Email, Webmessaging, Facebook,_ and _Whatsapp._ If no values are given, then Chat will be taken as the conversation type. To generate a survey link for call interactions, you must have at least one digital survey type enabled in the voice survey configuration.\n8. Offered (Optional, default: true) : Indicates whether to mark the survey as Offered when creating the survey link.\n9. AgentIdentifier (Optional): GUID of agent in Genesys Cloud (Admin > People). Select a particular agent. Agent Id will be in the URL of the page [https://apps.aps1.pure.cloud/directory/#/admin/directory/peopleV2/{agentidentifier}/roles](https://apps.aps1.pure.cloud/directory/#/admin/directory/peopleV2/4bced821-2f98-4d65-ac52-0f9ca5671065/roles). If there are multiple agents in an interaction, you can use this agent identifier to generate a survey link for a specific agent. We will store the history of agents involved in a conversation and validate whether the agent specified through the API was part of the conversation. If the validation succeeds, the survey will be offered. If the validation fails, an error will be thrown: 'Agent doesn't belong to the conversation'.\n10. QueueIdentifier (Optional): GUID of queue in Genesys Cloud (Admin > Queues). Select a particular queue. Queue Id will be there in the URL of the page. [https://apps.mypurecloud.com/directory/#/admin/organization/queues/{queueidentifer}/general](https://apps.aps1.pure.cloud/directory/#/admin/organization/queues/e6ad3e55-b127-4665-8b2a-2335786ed766/general). If there are multiple queues in an interaction, you can use this queue identifier to generate a survey link for a specific queue. We will store the history of queue involved in a conversation and validate whether the queue specified through the API was part of the conversation. If the validation succeeds, the survey will be offered. If the validation fails, an error will be thrown: 'Queue doesn't belong to the conversation'."
  },
  {
    "instruction": "Offering Surveys without a Genesys Conversation",
    "input": "",
    "output": "To offer a survey directly without a Genesys conversation, you should pass the ID of the survey to be offered and a Reference ID to uniquely identify the interaction in dashboards and reference reports.\n\njson\n{\n    \"SurveyID\": 1234,\n    \"ReferenceID\": \"SOME_UNIQUE_ID\",\n    \"CustomerID\": \"customer email, phone or username\",\n    \"Language\": \"en-us\",\n    \"OrganisationID\": 1234,\n    \"OrganisationKey\": \"YOUR_FEEBAK_ORGKEY\",\n    \"ConversationType\": \"Whatsapp\",\n    \"Offered\" : true\n}\n\n\n1. SurveyID - ID of the survey you want to offer. It can be found on the Survey Page in Feebak.\n2. ReferenceID - You can pass an ID here, which can be used to uniquely identify this survey in dashboards and reports.\n\nResponse\n\njavascript\n{\n    \"Success\": true,\n    \"Data\": {\n        \"SurveyLink\": \"https://fantacode.feebak.com/survey/ccG8a68\",\n        \"DefaultPrompt\": \"Greetings, would you mind taking a survey?\"\n    },\n    \"Message\": null\n}\n\n\nIn the response, the survey link and the default prompt will be returned inside the Data as shown above.\n\n1. SurveyLink - Generated link to the digital survey\n2. DefaultPrompt - prompt added in the Default Text field while creating the survey.\\\n   ![](<../../.gitbook/assets/image (250).png>)\n\nIf there are any errors, they will be returned in the Message."
  },
  {
    "instruction": "Mark Survey as Offered",
    "input": "",
    "output": "If you want to mark an already generated survey link as offered, you can use the following API:\n\nHTTP Method: PUT\n\nURL: [/v2/websurvey/link/state?linkid=\"LINKID\"\\&surveystatus=Offered](https://app.feebak.com/v2/websurvey/link/state?linkid=%22LINKID%22\\&surveystatus=Offered)\\\n\\\nQuery parameters to be send in URL\n\nLinkID: A unique id generated against a survey, can be obtained from survey URL generated.\n\neg: Below is a sample survey URL\n\njson\nhttps://app.feebak.com/survey/fqgC8Q2\n\n\nIn above survey URL, \"fqgC8Q2\" is the link ID which you can use in the above API to mark this survey as offered.\n\nSurveyStatus - status of the survey which you want to update about the survey. Use the value \"Offered\" to mark the survey as offered. \\\n\\\nBody\n\njson\n{\n    \"OrganisationID\": 1234,\n    \"OrganisationKey\": \"YOUR_FEEBAK_ORGKEY\"\n}"
  },
  {
    "instruction": "Custom Conversation Type",
    "input": "",
    "output": "{% hint style=\"info\" %}\nNote: Custom conversation types can only be used to create a survey link without a Genesys conversation\n{% endhint %}\n\nCustom conversation types can be used when generating surveys via the Feebak Digital Survey API. When using the Feebak Digital Survey API for survey generation, specify the custom conversation type that corresponds to the interaction.\n\njson\n{\n    \"SurveyID\": 1234,\n    \"ReferenceID\": \"SOME_UNIQUE_ID\",\n    \"CustomerID\": \"customer email, phone or username\",\n    \"Language\": \"en-us\",\n    \"OrganisationID\": 1234,\n    \"OrganisationKey\": \"YOUR_FEEBAK_ORGKEY\",\n    \"ConversationType\": \"Kiosk\"\n}\n\n\nThis ensures that the survey is categorized correctly.\n\nYou can also add Custom Attributes (key, value) in the API request.&#x20;\n\n{% hint style=\"info\" %}\nNote: You can use these custom attributes only when using custom conversation types\n{% endhint %}\n\njson\n{\n    \"SurveyID\": 1234,\n    \"ReferenceID\": \"SOME_UNIQUE_ID\",\n    \"CustomerID\": \"customer email, phone or username\",\n    \"Language\": \"en-us\",\n    \"OrganisationID\": 1234,\n    \"OrganisationKey\": \"YOUR_FEEBAK_ORGKEY\",\n    \"ConversationType\": \"Kiosk\",\n    \"CustomAttributes\" : {\n        \"key\" : \"value\",\n        \"key1\" : \"value1\"\n        }\n}\n\n\nThe Custom Attributes will show in the interaction details\n\n<figure><img src=\"../../.gitbook/assets/image (292).png\" alt=\"\"><figcaption></figcaption></figure>\n\nYou can refer the document to know how to add Custom Conversation Types here: [Custom Conversation Type](../../introduction-to-feebak/custom-conversation-type.md)"
  },
  {
    "instruction": "Digital Survey API",
    "input": "",
    "output": "The Digital Survey API can be used to generate a digital survey. It returns the link to the generated survey. It uses the POST method, accepts [JSON-encoded](http://www.json.org/) request bodies, returns [JSON-encoded](http://www.json.org/) responses, and uses standard HTTP response codes.&#x20;"
  },
  {
    "instruction": "Features",
    "input": "",
    "output": "The Digital Survey API provides the following features:\n\n1. Generate Survey Link: Create a unique link for a digital survey based on the provided parameters.\n2. Customize Survey Parameters: Allow customization of survey attributes through the API request.\n3. Offer Surveys with Genesys Conversation: Ability to offer surveys for Genesys Conversations.\n4. Offer Surveys without Genesys Conversation: Option to generate and distribute surveys independently of Genesys Conversation, using SurveyId."
  },
  {
    "instruction": "Versions",
    "input": "",
    "output": "These are the available versions of Digital Survey API:\n\n [Digital Survey API (version 1)](digital-survey-api-version-1.md)\n [Digital Survey API (version 2)](digital-survey-api-version-2.md)\n [Digital Survey API (version 3)](digital-survey-api-version-3.md)"
  },
  {
    "instruction": "Rate Limiting",
    "input": "",
    "output": "Rate limits for the Digital Survey API routes, measured in requests per minute:\n\n| API Route                                                         | Rate Limit (Requests per Minute) |\n| ----------------------------------------------------------------- | -------------------------------- |\n| [Digital Survey API (version 1)](digital-survey-api-version-1.md) | 300                              |\n| [Digital Survey API (version 2)](digital-survey-api-version-2.md) | 300                              |\n| [Digital Survey API (version 3)](digital-survey-api-version-3.md) | 300                              |\n\nFor more details about rate limits, [click here](../rate-limits.md)."
  },
  {
    "instruction": "Base URLs",
    "input": "",
    "output": "We have two base URLs for different regions:\n\n| Region      | Base URL                 |\n| ----------- | ------------------------ |\n| Middle East | https://app.feebak.me  |\n| Europe      | https://app.feebak.com |\n\nFor more details about your Feebak base URL, [click here](../feebak-regions.md)."
  },
  {
    "instruction": "Digital Survey API (version 1)",
    "input": "",
    "output": "HTTP Method: POST\n\nURL: [/v1/websurvey/link](https://app.feebak.com/v1/websurvey/link)&#x20;\n\nBody\n\nTo offer a survey for a Genesys conversation, you should pass the Genesys Conversation ID in the body as shown below. In this case, the Survey offered will be the one assigned in the Survey Assignments.\n\njavascript\n{\n    \"ConversationID\": \"Genesys Conversation ID\",\n    \"CustomerID\": \"customer email, phone or username\",\n    \"Language\": \"en-us\",\n    \"OrganisationID\": 1234,\n    \"OrganisationKey\": \"YOUR_FEEBAK_ORGKEY\",\n    \"Offered\" : true\n}\n\n\n1. ConversationID - Genesys Conversation ID\n2. Language - Language in which the survey should be offered\n3. OrganisationKey - Every organisation has a unique key. It can be found on the [_Administration > Organisation Settings Page_](https://app.feebak.com/administration/organisationsettings) in Feebak.\n4. OrganisationID - ID of your organisation\n5. CustomerID (Optional) - Customer email, phone or username\n6. Offered - (Optional, default: true) Indicates whether to mark the survey as Offered when creating the survey link."
  },
  {
    "instruction": "Offering Surveys without a Genesys Conversation",
    "input": "",
    "output": "To offer a survey directly without a Genesys conversation, you should pass the ID of the survey to be offered and a Reference ID to uniquely identify the interaction in dashboards & reports.\n\njavascript\n{\n    \"SurveyID\": 1234,\n    \"ReferenceID\": \"SOME_UNIQUE_ID\",\n    \"CustomerID\": \"customer email, phone or username\",\n    \"Language\": \"en-us\",\n    \"OrganisationID\": 1234,\n    \"OrganisationKey\": \"YOUR_FEEBAK_ORGKEY\",\n    \"Offered\" : true\n}\n\n\n\n\n1. SurveyID - ID of the survey you want to offer. It can be found on the Survey Page in Feebak.\n2. ReferenceID - You can pass an ID here, which can be used to uniquely identify this survey in dashboards & reports.\n\nResponse\n\njavascript\n{\n    \"Success\": true,\n    \"Data\": \"https://fantacode.feebak.com/survey/WgiIMaJ\",\n    \"Message\": null\n}\n\n\nThe survey link will be returned as the Data of response object as shown above. If there are any errors, they will be returned in the Message."
  },
  {
    "instruction": "Roles Management",
    "input": "",
    "output": "## View Roles\n\n1. Once you are logged in, different options can be viewed in the Navigation bar in which, Roles is present in the User Administration section.\n2. Selecting Roles will list all the added roles along with the Default Role.\n3. You can also search Roles by name.\n4. Edit and Delete buttons are present along with the Roles.\n\n![](<../.gitbook/assets/Screenshot 2024-08-29 194109.png>)"
  },
  {
    "instruction": "Add Roles",
    "input": "",
    "output": "1. Navigate to User Administration > Roles.\n2. Selecting Roles will list all the added Roles.\n3. To add a new Role, click on the Add New Role Button at the top right corner of the screen.\n4. It displays Role Details in which Role Name and Description are to be filled.\n\n![](<../.gitbook/assets/Screenshot 2024-08-29 193018.png>)\n\n5. To Add more Permissions to the Role, click On Add More Permissions.\n6. It displays a list of Permissions from which one more permissions can be selected.\n\n![](<../.gitbook/assets/Screenshot 2024-08-29 193308.png>)\n\n7. Tap on Save button at the top right corner after adding all the above mentioned details.\n\n{% hint style=\"info\" %}\nMake sure you have added the role before accessing the application\n{% endhint %}"
  },
  {
    "instruction": "Edit Roles",
    "input": "",
    "output": "1. Navigate to User Administration > User.\n2. Selecting Roles will list all the added roles along with the Default Role.\n3. Edit and Delete buttons are present along with the Roles.\n\n{% hint style=\"info\" %}\nPlease note that, the role listed as Default Role cannot be edited.\n{% endhint %}\n\n4. Click on the Edit Button respective to the role to be edited.&#x20;\n\n![](<../.gitbook/assets/Screenshot 2024-08-29 193715.png>)\n\n5. In the Role Details section, You can edit the Role Name along with the Description.\n6. To add more permissions to the role, Click on Add More Permissions.\n\n![](<../.gitbook/assets/Screenshot 2024-08-29 193738.png>)\n\n7. Select one or more Permissions to be added to the respective Role.  An added permission can be deleted by clicking on it.\n8. You can also assign this role and divisions to multiple users by clicking on the Assign Users button and making changes in the window.\n\n<figure><img src=\"../.gitbook/assets/image (38).png\" alt=\"\"><figcaption></figcaption></figure>\n\n9. Finally, Tap on the save button at the top right corner to save the changes made."
  },
  {
    "instruction": "Delete Roles",
    "input": "",
    "output": "1. Navigate to User Administration > User.\n2. Selecting Roles will list all the added roles along with the Default Role.\n3. Edit and Delete buttons are present along with the Roles.\n\n{% hint style=\"info\" %}\nPlease note that, the role listed as Default Role cannot be deleted\n{% endhint %}\n\n4. Click on the delete button respective to the role to be deleted.\n\n![](<../.gitbook/assets/Screenshot 2024-08-29 194151.png>)\n\n5. To Delete Multiple Users,\n    Click on Delete Multiple Roles at the top of the screen and select the users to be deleted.\n    Click on the delete Roles button at the top right corner of your screen after selecting the Roles to be deleted.\n    Click on Yes, Delete Role button in order to delete the selected Roles."
  },
  {
    "instruction": "IP Whitelisting",
    "input": "",
    "output": "IP Whitelisting is a security measure used to allow access to your business information, such as data stored on a server, only from specific IP addresses. By creating a list of trusted IP addresses, you can ensure that only authorized users can access sensitive information.\n\nA Static IP Address is an IP address that remains constant over time. Unlike dynamic IP addresses, which can change with each connection, a static IP address does not change, making it ideal for use in IP whitelisting. This stability ensures that authorized access remains consistently secure.\n\nWhen managing organization settings, the Allowed IP Addresses section enables you to limit access to Feebak to only those users visiting from a specified IP address.\n\n{% hint style=\"info\" %}\nWarning: If you use dynamic IPs, do not enable this functionality.\n{% endhint %}"
  },
  {
    "instruction": "Add IP Addresses in Feebak",
    "input": "",
    "output": "1. In the Administration tab, click on Organisation Settings.\n2.  In the IP Addresses box, enter the list of CIDR (Classless Inter-Domain Routing) blocks you would like to allow. You can enter one CIDR per line (for example, 192.168.100.0/22) up to a maximum of 150 CIDRs.\\\n\n\n    <figure><img src=\"../.gitbook/assets/image (319).png\" alt=\"\"><figcaption></figcaption></figure>\n\nWhen creating or updating an allowlist, ensure that your IP is added to or already in the allowlist.\n\n3. Click Save.\n\n{% hint style=\"info\" %}\nNotes:\n\n You can convert IP addresses using this [CIDR Utility Tool](http://www.ipaddressguide.com/cidr). If you do not enter anything in the Allowed IP Addresses field, all IP addresses can access Feebak.\n If you incorrectly enter a formatted CIDR, the system hides the Save button until you correct the format. Feebak checks to ensure that the IP address of the user who configures the allowlist appears in one of the CIDR entries.\n Only IPv4 addresses are supported.\n Do not include private IP addresses in your list.\n{% endhint %}"
  },
  {
    "instruction": "Email Templates",
    "input": "",
    "output": "Email templates in Feebak enable you to streamline communication by sending survey links to customers and email notifications to supervisors or relevant stakeholders. Customize templates to ensure consistent messaging and align with your branding."
  },
  {
    "instruction": "View Email-Templates",
    "input": "",
    "output": "1.Once you are logged in, different options can be viewed in the Navigation bar in which, Email Templates is present in the Templates section\n\n2.Selecting Email Templates will list all the added templates\n\n3\\. You can also search Templates by name\n\n4\\. Edit and Delete buttons are present along with the Template Name\n\n![](<../.gitbook/assets/image (219).png>)"
  },
  {
    "instruction": "&#x20;Add Email Templates",
    "input": "",
    "output": "1.Navigate to Templates > Email Templates\n\n2\\. Selecting Email Templates will list all the added templates\n\n3\\. To add a new Template, click on the Add New button at the top right corner of the screen\n\n![](<../.gitbook/assets/image (255).png>)\n\n4\\. Enter the Name, Subject and the mail body. You can also add Available variables to the mail body\n\n5\\. Click Add New on the top of the screen to add new languages from the given list\n\n6\\. Finally, tap on the save button at the top of the screen"
  },
  {
    "instruction": "Edit Email Templates",
    "input": "",
    "output": "1.Navigate to Templates > Email Templates\n\n2\\. Selecting Email Templates will list all the added templates\n\n3\\. Edit and Delete buttons are present along with the Template Name\n\n4\\. Click on the Edit button to edit the respective Email Template\n\n![](<../.gitbook/assets/image (194).png>)\n\n5\\. Template Name, Subject and Mail Body can be edited\n\n6\\. Click on the Add New at the top of your screen to add New languages\n\n7\\. Click on the save button at the top right corner of your screen  to save the changes made"
  },
  {
    "instruction": "Delete Email Template",
    "input": "",
    "output": "1.Navigate to Templates > Email Templates\n\n2\\. Selecting Email Templates will list all the added templates\n\n3\\. Edit and Delete buttons are present along with the Template Name\n\n4\\. Click on the Delete button to delete the respective Email Template\n\n![](<../.gitbook/assets/image (192).png>)\n\n5\\. Click Yes, Delete Template to confirm the Deletion\n\n\n\n\n\n\n\n\n\n\n\n&#x20; &#x20;"
  },
  {
    "instruction": "Sentiment Analysis",
    "input": "",
    "output": "Sentiment analysis in Feebak is a feature that calculates sentiment scores for open feedback responses from surveys. Sentiment scores is available for both digital and voice interactions.\n\n{% hint style=\"info\" %}\nNote: Premium organizations seeking to utilize sentiment analysis need to purchase the \"Feebak Sentiment Analysis Metered for Genesys Cloud\" license from Genesys. For non-premium organisations, sentiment analysis costs will be included in their monthly invoices.\n{% endhint %}"
  },
  {
    "instruction": "To enable Sentiment Score Analysis",
    "input": "",
    "output": "1. Navigate to Administration > Organisation Settings.\n2. In the Organisation Settings page, enable the Calculate Sentiment Score option.\n\n<figure><img src=\"../.gitbook/assets/image (59).png\" alt=\"\"><figcaption></figcaption></figure>\n\nFeebak utilizes the Google Cloud Natural Language API to perform sentiment analysis. Additionally, sentiment scores for Genesys Cloud conversations are available on the interaction detail page and are accessible to every organization by default."
  },
  {
    "instruction": "**Viewing Sentiment Scores**",
    "input": "",
    "output": "Sentiment scores are displayed on the [Survey Activity Timeline](../introduction-to-feebak/dashboard/survey-activity-timeline.md), [Sentiment Score Summary](../introduction-to-feebak/dashboard/sentiment-score-summary.md), and [Interaction Details](../introduction-to-feebak/dashboard/interactions.md) page include the following metrics:\n\n Feedback Sentiment Score: Indicates the sentiment score of the feedback text.\n Feedback Sentiment Score Value: The sentiment value determined based on the sentiment score.\n   Positive\n   Neutral\n   Negative\n Feedback Sentiment Magnitude: Indicates the overall strength of emotion (both positive and negative) within the text.\n\n{% hint style=\"info\" %}\nNote:\n\n Conversation Sentiment Score will only be available after 4 minutes of the interaction.\n Conversation Sentiment Score will only be available for interactions that have a sentiment score in Genesys.\n For more details, refer to the [Genesys Cloud Understand Sentiment Analysis](https://help.mypurecloud.com/articles/understand-sentiment-analysis/) documentation.\n{% endhint %}\n\n<figure><img src=\"../.gitbook/assets/image (51).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "Sentiment Score, Feedback Word Cloud & Sentiment Trend",
    "input": "",
    "output": "These charts in the Survey Activity Timeline display sentiment score, frequent words as a wordcloud, and the trend of sentiment score, all based on the open feedback left by customers.&#x20;\n\n<figure><img src=\"../.gitbook/assets/image (383).png\" alt=\"\"><figcaption></figcaption></figure>\n\n<figure><img src=\"../.gitbook/assets/image (382).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "**Filtering Interactions**",
    "input": "",
    "output": "Users can filter interactions using the Sentiment Score Filter available on the interaction dashboard. This filter helps in analyzing interactions based on their sentiment scores, enhancing the ability to gain insights from the customer feedback.\n\n<figure><img src=\"../.gitbook/assets/image (61).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "Configure words in the wordcloud",
    "input": "",
    "output": "To configure the words displayed in the Wordcloud in the Survey Activity Timeline, follow these steps:\n\n1. Navigate to Administration > Dashboard Settings.\n2. Click on Add New.\n3. Enter the desired word and its associated Sentiment Marker Colour.\n4. After adding the words, only the configured words will appear in the Wordcloud. If not configured, the top 100 words are shown.\n\n<figure><img src=\"../.gitbook/assets/image (22).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "Voice Feedback Setup",
    "input": "",
    "output": "## Selecting a Transcription Provider\n\nFeebak provides multiple options for voice transcription. You can configure the preferred method by selecting a transcript type from the dropdown in Administration > Organisation Settings > Voice Transcript Settings."
  },
  {
    "instruction": "Available Transcription Options:",
    "input": "",
    "output": "Feebak Managed Transcript – Feebak directly processes and transcribes voice feedback. For premium organizations, _Feebak Voice Transcription Metered for Genesys Cloud_ license is required, and billing is handled by Genesys. For non-premium organizations, Feebak manages transcription and billing directly. Read more: [subscriptions](../feebak-subscription/).\n Genesys Cloud Transcript – Uses Genesys Cloud's transcription capabilities for processing voice feedback. This option is subject to fair-use policy limits and overage billing. Read more: [Genesys Cloud Fair-Use Policy](https://help.mypurecloud.com/articles/genesys-cloud-fair-use-policy/)\n Google Cloud Transcript - Uses Google Cloud Speech-to-Text API. Requires setting up a Google Cloud project and configuring API credentials. See the Google Speech-to-Text [Pricing Table](https://cloud.google.com/speech-to-text/pricing#pricing_table) to compare the cost. Pricing will also depend on whether [Data Logging](https://cloud.google.com/speech-to-text/docs/data-logging) is enabled."
  },
  {
    "instruction": "Setting up Google Speech to Text API",
    "input": "",
    "output": "If you choose Google Cloud Transcript using Google Speech-to-Text API, follow these steps to enable transcription:\n\n![](<../.gitbook/assets/image (239).png>)"
  },
  {
    "instruction": "Step 1: Configure Google Speech-to-Text API",
    "input": "",
    "output": "1. Go to the Google Speech-to-Text API Quickstart page.\n2. Click Set up a project, enter a project name, and click Next.\n    This will create a Google Cloud project, enable the Speech-to-Text API, and generate a service account.\n3. Click DOWNLOAD PRIVATE KEY AS JSON.\n    This key will be required for Feebak to access the Speech-to-Text API."
  },
  {
    "instruction": "Step 2: Upload the Private Key to Feebak",
    "input": "",
    "output": "1. Navigate to Administration > Organisation Settings in Feebak.\n2. Locate the Google Speech API Credentials field.\n3. Upload the JSON Private Key file downloaded from Google Cloud.\n\n<figure><img src=\"../.gitbook/assets/image (2).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "Survey Frequency Management",
    "input": "",
    "output": "The Survey Frequency Manager is a powerful tool designed to help organizations regulate the frequency of survey offerings. By configuring specific rules, it ensures customers are not overwhelmed with surveys while still gathering valuable feedback at the right intervals. This feature empowers organizations to balance customer engagement with data collection, reducing survey fatigue and enhancing both customer satisfaction and the quality of insights gathered.\n\nWith the Survey Frequency Manager, you can control survey offerings based on two key frequency control options: interaction limits and time-based frequency limits. Interaction limits regulate survey offerings based on the number of customer interactions, while frequency limits allow you to define the maximum number of surveys offered within a specified time frame. These options provide flexibility to tailor survey offerings according to your business needs.\n\nAdditionally, the Survey Frequency Manager offers a high level of customization with its scope settings. Rules can be applied universally across all channels and queues or narrowed down to specific combinations of channels and queues. This granularity allows you to create multiple configurations that align with your organizational requirements, ensuring surveys are offered at the right moment and to the right audience."
  },
  {
    "instruction": "**Create Rules Based on Interaction Count**",
    "input": "",
    "output": "1. Navigate to Surveys > Survey Frequency Manager.\n\n<figure><img src=\"../.gitbook/assets/image (10).png\" alt=\"\"><figcaption></figcaption></figure>\n\n2. Click on Add Frequency Rule.\n3. Enter a name for the rule.\n\n<figure><img src=\"../.gitbook/assets/image (11).png\" alt=\"\"><figcaption></figcaption></figure>\n\n4. Click on Add Rule, and choose Rule Based on Interaction Count.\n5. Enter the count of Minimum Interactions Before Survey.\n\n<figure><img src=\"../.gitbook/assets/image (12).png\" alt=\"\" width=\"369\"><figcaption></figcaption></figure>\n\n6. Click on Add Scope, then choose Interaction Type and Queue.\n7. Select the appropriate options from the dropdown for Queues and Interaction Type.\n8. Finally click Save."
  },
  {
    "instruction": "**Create Rules Based on Survey Frequency**",
    "input": "",
    "output": "1. Navigate to Surveys > Survey Frequency Manager.\n\n<figure><img src=\"../.gitbook/assets/image (10).png\" alt=\"\"><figcaption></figcaption></figure>\n\n2. Click on Add Frequency Rule.\n3. Enter a name for the rule.\n\n<figure><img src=\"../.gitbook/assets/image (11).png\" alt=\"\"><figcaption></figcaption></figure>\n\n4. Click on Add Rule, and choose Rule Based on Survey Frequency.\n5. Enter the count of Maximum Surveys to Offer.\n6. Select the timeframe by entering the number of Days/Hours/Months.\n\n<figure><img src=\"../.gitbook/assets/image (13).png\" alt=\"\" width=\"369\"><figcaption></figcaption></figure>\n\n7. Click on Add Scope, then choose Interaction Type and Queue.\n8. Select the appropriate options from the dropdown for Queues and Interaction Type.\n9. Finally click Save."
  },
  {
    "instruction": "Notification Templates",
    "input": "",
    "output": "## View Notification Templates\n\n1.Once you are logged in, different options can be viewed in the Navigation bar in which, Notification Templates is present in the Templates section\n\n2\\. Selecting Notification Templates will list all the added Templates.\n\n3\\. You can also search Templates by name\n\n4\\. Edit and Delete buttons are present along with the Templates\n\n![](<../.gitbook/assets/image (199).png>)"
  },
  {
    "instruction": "Add Notification Templates",
    "input": "",
    "output": "1.Navigate to Templates > Notification Templates\n\n2\\. Selecting Notification Templates will list all the added templates\n\n3\\. To add a new Template, click on the Add New button at the top right corner of the screen\n\n![](<../.gitbook/assets/image (218).png>)\n\n4\\. Enter Name and the mail body. You can also add Available variables to the mail body\n\n5\\. Click Add New on the top of the screen to add new languages from the given list\n\n6\\. Finally, tap on the save button at the top of the screen"
  },
  {
    "instruction": "Edit Notification Templates",
    "input": "",
    "output": "1.Navigate to Templates > Notification Templates\n\n2\\. Selecting Notification Templates will list all the added templates\n\n4\\. Edit and Delete buttons are present along with the Templates\n\n5\\. Click on the Edit button to edit the respective template\n\n![](<../.gitbook/assets/image (197).png>)\n\n6\\. Template Name and Mail Body can be edited\n\n7\\. Click on Add New at the top of your screen to add New languages\n\n8\\. Click on the save button at the top right corner of your screen  to save the changes made"
  },
  {
    "instruction": "Delete Notification Templates",
    "input": "",
    "output": "1.Navigate to Templates > Notification Templates\n\n2\\. Selecting Notification Templates will list all the added templates\n\n3\\. Edit and Delete buttons are present along with the Template Name\n\n4\\. Click on the Delete button to delete the respective Notification Template\n\n![](<../.gitbook/assets/image (185).png>)\n\n5\\. Click Yes, Delete Template to confirm the deletion"
  },
  {
    "instruction": "Sync User Details From Genesys Cloud",
    "input": "",
    "output": "Syncing user details from Genesys Cloud to Feebak means updating the user data in Feebak with the user details in Genesys Cloud."
  },
  {
    "instruction": "Sync User Details",
    "input": "",
    "output": "1. Once you are logged in, different options can be viewed in the Navigation bar in which, User is present in the User Administration section\n2. Click on the Sync User Details button on the right top side of the page\n\n<figure><img src=\"../../.gitbook/assets/image (186).png\" alt=\"\"><figcaption></figcaption></figure>\n\n3. It will popup a confirmation window\n\n<figure><img src=\"../../.gitbook/assets/image (180).png\" alt=\"\"><figcaption></figcaption></figure>\n\n4. Click on Yes, Sync User Details button to sync the user data\n\n{% hint style=\"info\" %}\nNote: Changes made to the user data, like the user name and email, after importing users from Genesys Cloud will be replaced with the data in Genesys Cloud while performing the Sync User Details action. Also, changes made in the Genesys Cloud will be reflected in Feebak while synchronizing user details.\n{% endhint %}"
  },
  {
    "instruction": "User Management",
    "input": "",
    "output": "### View Users\n\n1. Once you are logged in, different options can be viewed in the Navigation bar in which, User is present in the User Administration section.\n2. Selecting User will list all the imported users.\n3. You can also search Users by name.\n4. Edit and Delete buttons are present along with the name of the user.\n\n![](<../../.gitbook/assets/Screenshot 2024-08-29 170333.png>)"
  },
  {
    "instruction": "Add Users",
    "input": "",
    "output": "1. Navigate to User Administration > User.\n2. Selecting User will list the all the imported users by default.\n3. Click on the Add New User button at the top right corner of the screen.\n4. It displays User Details in which First Name and E-Mail are to be filled.\n\n![](<../../.gitbook/assets/Screenshot 2024-08-29 170423.png>)\n\n5. You can optionally assign a division to the user by clicking on the dropdown and selecting a division.\n6. To Add one or more roles to the user, click on Add More Roles.\n\n![](<../../.gitbook/assets/Screenshot 2024-08-29 170527.png>)\n\n6. It displays all the User Roles From which the desired roles can be selected.\n7. Click on the Save button at the top right corner of the screen to save the Added Roles and the User Details."
  },
  {
    "instruction": "Import Users",
    "input": "",
    "output": "{% hint style=\"info\" %}\nUsers imported from Genesys Cloud can log in with their Genesys Cloud account without additional passwords. Roles can be assigned by editing the imported Users.\n{% endhint %}\n\n1. Navigate to User Administration > User.\n2. To import users from Genesys Cloud, click the Import Users button at the top of the screen.\n3. It displays all users from your Genesys Cloud organization.\n\n![](<../../.gitbook/assets/Screenshot 2024-08-29 171353.png>)\n\n4. Select one or more users from the list and click Import Selected button at the top right corner of the screen.\n5. Once the selected users are imported, you can view their names in the Imported Users' list."
  },
  {
    "instruction": "Sync User Details <a href=\"#import-genesys-cloud-users\" id=\"import-genesys-cloud-users\"></a>",
    "input": "",
    "output": "{% hint style=\"info\" %}\nThis process updates the user data in Feebak with the user details in Genesys Cloud. Changes made to the user data, like the user name and email, after importing users from Genesys Cloud will be replaced with the data in Genesys Cloud.\n{% endhint %}\n\n1. Navigate to User Administration > User.\n2. To sync the latest user information from Genesys Cloud, click the Sync User Details button.\n3. Click Yes, Sync Division Details to proceed with the update.\n\n<figure><img src=\"../../.gitbook/assets/image (34).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "Edit Users",
    "input": "",
    "output": "1. Navigate to User Administration > User.\n2. Selecting User will list the all the imported users by default.\n3. To Edit a particular user details, click on the Edit button along with name of the user.\n4. It displays the Status of the user, User Details and Added Roles.\n\n![](<../../.gitbook/assets/Screenshot 2024-08-29 171144.png>)\n\n5. Update the following details and tap the save button at the top right corner of the screen. otherwise, tap on Cancel, next to the save button."
  },
  {
    "instruction": "Delete User",
    "input": "",
    "output": "1. Navigate to User Administration > User.\n2. Selecting User will list the all the imported users by default.\n3. To delete a particular user, click on the Delete button along with name of the user.\n4. Click on Yes, Delete User button in order to delete the user.\n\n![](<../../.gitbook/assets/Screenshot 2024-08-29 171506.png>)\n\n5. To Delete Multiple Users:\n   1. Click on Delete Multiple users at the top of the screen and select the users to be deleted.\n   2. Click on the delete Users button at the top right corner of your screen after selecting the users to be deleted.\n   3. Click on Yes, Delete User button in order to delete the selected users"
  },
  {
    "instruction": "How to Add Supervisor",
    "input": "",
    "output": "Supervisors in Feebak require specific roles to access dashboards and reports. You can create supervisors for different divisions in your organization, allowing for more targeted management.\n\nTo create accounts for supervisors, follow these two main steps:\n\n1. Creating Role for Supervisor\n2. Creating Supervisor Account\n\n{% hint style=\"info\" %}\nNotes:\n\n Roles: A set of permissions that govern access within the system. For supervisors, we'll create a specific role with the necessary permissions.\n Divisions: Organizational units that allow you to segregate concerns and implement proper access control. You can assign supervisors to different divisions for granular management.\n{% endhint %}"
  },
  {
    "instruction": "Creating Role for Supervisor",
    "input": "",
    "output": "1. In order to create a role, go to Roles under User Administration from the Dashboard.\\\n   Dashboard > User Administration > Roles > Add New Role\n\n![](<../../.gitbook/assets/Screenshot 2024-08-29 173455.png>)\n\n2. Provide the Role Name and give Description if needed.\n3. Now click 'Add More Permissions' and assign the following permissions\n    SupervisorDashboard\\_View - To view supervisor dashboard.\n    SurveyReports - To view survey report pages.\n    SurveyManagement\\_View - To use survey filters in the report.\n    UserManagement\\_View - To use agent filters in the report.\n    QueueManagement\\_View - To use queue filters in the report.\n4. Now click Save.\n5. The role needed for Supervisor account is now successfully created.\n\nAfter creating the role, we can add supervisor as a normal user. Refer [Roles Management](../roles-management.md) for detailed documentation."
  },
  {
    "instruction": "Creating Supervisor Account",
    "input": "",
    "output": "1. Go to Administration > User Administration > User > Add New User\n\n![](<../../.gitbook/assets/Screenshot 2024-08-29 173837.png>)\n\n2. Enter Your Name and E-mail.\n3. Assign a division to the user by clicking on the dropdown menu and selecting the appropriate division. This step is crucial for segregating organizational concerns and ensuring proper access control within your organization. You can also designate supervisors for different divisions, allowing for more granular management across various organizational units.\n4. Click on Add More Roles and choose the newly created role Supervisor which we added in the step above.\n5. Click Save.\n\nYou will be taken to the User listing page where you will see the new supervisor account. Hover over the user to view 'Edit' and 'Delete User' Options. These allow you to make changes or remove the account as needed. Refer [User Management](./) for detailed documentation."
  },
  {
    "instruction": "How to Add Agent or User",
    "input": "",
    "output": "Agents or Users in Feebak require specific roles to access appropriate features and perform their duties. You can create agents or users and bind them to different divisions in your organization, allowing for more structured access control.\n\nTo create accounts for agents or users, follow these two main steps:\n\n1. Creating Role for Agent or User\n2. Creating Agent or User Account\n\n{% hint style=\"info\" %}\nNotes:\n\n Roles: A set of permissions that govern access within the system. For agents or users, we'll create specific roles with the necessary permissions based on their responsibilities.\n Divisions: Organizational units that allow you to segregate concerns and implement proper access control. You can assign agents or users to different divisions for targeted management and access.\n{% endhint %}"
  },
  {
    "instruction": "Creating Role for Agent or User",
    "input": "",
    "output": "1. In order to create a role, navigate to the \"Roles\" section under User Administration. Dashboard > User Administration > Roles > Add New Role.\n\n<figure><img src=\"../../.gitbook/assets/Screenshot 2024-08-31 122610.png\" alt=\"\"><figcaption></figcaption></figure>\n\n2. Provide the Role Name and a Description if needed.\n3. Now click Add More Permissions and assign the following permission:\n    AgentDashboard\\_View: To view the agent dashboard.\n4. Now click Save.\n5. The role needed for Agent account is now successfully created."
  },
  {
    "instruction": "Creating Agent or User account",
    "input": "",
    "output": "1. Go to Administration > User Administration > User\n2. You can either import users from Genesys Cloud using the Import Users option or can add new users using the Add New User option.\n\n<figure><img src=\"../../.gitbook/assets/Screenshot 2024-08-31 175703.png\" alt=\"\"><figcaption></figcaption></figure>\n\n3. Enter your Name and E-mail.\n4. Assign a division to the agent by clicking on the dropdown and selecting the appropriate division. This step is crucial for segregating organizational concerns and ensuring proper access control.\n5. Click on Add More Roles and choose the newly created Agent role.\n\n<figure><img src=\"../../.gitbook/assets/Screenshot 2024-08-31 182758.png\" alt=\"\"><figcaption></figcaption></figure>\n\n6. You can also edit the divisions assigned to the role by clicking the Edit button and selecting a division. AllDivisions is selected by default.\n\n<figure><img src=\"../../.gitbook/assets/image (37).png\" alt=\"\"><figcaption></figcaption></figure>\n\n7. Now click Save.\n\nYou will be taken to the User listing page, where you can see the agent's account. Hover over the user to view 'Edit' and 'Delete User' Options. These allow you to make changes or remove the account as needed."
  },
  {
    "instruction": "Custom Domain",
    "input": "",
    "output": "The Custom Domain feature in Feebak allows organizations to add their own domains for sending email surveys. This feature enhances flexibility and professionalism by enabling the use of verified custom domains through Amazon SES (Simple Email Service).\n\nWhen an email survey is sent, it can use a custom domain to indicate its source. The process involves verifying the domain ownership, optionally configuring DKIM authentication to prevent emails from being flagged as spam, and setting up a custom MAIL FROM address."
  },
  {
    "instruction": "Permissions",
    "input": "",
    "output": "To manage custom email senders, specific Feebak permissions are required:\n\n CustomEmailManagement\\_View: Required to access the Custom Email Sender section.\n CustomEmailManagement\\_Edit: Required to add or edit custom domain names.\n\nYou can add these permissions in Feebak by following the steps below:\n\n Administration > Roles.\n Create a new role or edit an existing role.\n Add permissions.\n Assign it to the users.&#x20;"
  },
  {
    "instruction": "Add Custom Domain Names",
    "input": "",
    "output": "Users with the \"CustomEmailManagement\\_Edit\" permission can add or edit custom domains names.\n\nYou can configure this Custom Domain under Administration > Organisation > Custom Email Sender > Custom Domain.\n\n<figure><img src=\"../../.gitbook/assets/custom-domain (1).png\" alt=\"\"><figcaption></figcaption></figure>\n\nBy clicking on the Add New button, a new window will be opened where you can give your custom domain name.\n\n<figure><img src=\"../../.gitbook/assets/image (45).png\" alt=\"\"><figcaption></figcaption></figure>\n\nUpon adding a valid domain or subdomain, you'll be redirected to the domain records page where TXT records will be displayed.\n\n<figure><img src=\"../../.gitbook/assets/custom-domain (2).png\" alt=\"\"><figcaption></figcaption></figure>\n\nAdd the displayed TXT record to your domain's DNS settings to verify it with Feebak. After updating your DNS, you can refresh the status on Feebak. Verification may take up to 72 hours. If it fails, reset the record values and attempt verification again.\n\n{% hint style=\"info\" %}\nNote: Some DNS providers automatically append the domain name to the end of DNS records. Adding a record that already contains the domain name (e.g., _amazonses.example.com) can result in the duplication of the domain name (e.g., _amazonses.example.com.example.com). To avoid this, add a period to the end of the domain name in the DNS record (e.g., _amazonses.example.com.). This indicates to your DNS provider that the record name is fully qualified and prevents the duplication.\n{% endhint %}\n\nOnce verified, you can use any email address from this domain for sending email surveys. On the survey builder page, go to the email tab, select Domain as the from address, and pick a verified domain from the select box. Add a valid email address in the Email Address column.\n\n<figure><img src=\"../../.gitbook/assets/image (48).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "**Perform DKIM Authentication**",
    "input": "",
    "output": "Optionally set up extra customization for sending email, including DKIM authentication. Configuring DKIM authentication helps prevent survey emails routed through your custom domain from being marked as spam.\n\n{% hint style=\"info\" %}\nDKIM is a type of email authentication that generates a signature for the survey email, showing senders that the message is legitimate and has not been modified in transit. Authenticated emails increase deliverability. For more information about DKIM authentication, see [Authenticating your email in Amazon SES](https://docs.aws.amazon.com/ses/latest/dg/email-authentication-methods.html) and [DKIM frequently asked questions](https://dkim.org/info/dkim-faq.html).\n{% endhint %}\n\nGeneral Procedure to Add DKIM Records:\n\n1. When you create your domain, Feebak automatically generates DKIM records. You can set up DKIM anytime by adding the provided records to your domain’s DNS.\n2. In Feebak, go to Custom Email Sender > Custom Domain.\n3. Locate the organization’s domain and click View. The Domain Verification page opens.\n4. Under Domain Verification, copy the DNS token strings for DKIM authentication.\n5. In your DNS provider’s website, sign in and navigate to the page for updating your domain’s DNS records.\n6. Follow your provider’s instructions for adding DKIM records.\n7. By clicking the refresh button, you can refresh the status of the records. If Feebak verifies the records, the Pending Verification message changes to Verified. This process can take up to 72 hours.\n8. When SES verifies the records, survey emails use DKIM. If you do not set up DKIM, or if the process fails, Amazon SES still sends your email, but it is not signed using a DKIM signature."
  },
  {
    "instruction": "**Set Up Custom MAIL FROM Configuration**",
    "input": "",
    "output": "After verifying the domain name, optionally set up extra customization for sending and receiving email, including SPF authentication to update the MAIL FROM address when outbound emails arrive from this custom domain.\n\nBy default, Feebak uses amazonses.com as the MAIL FROM value for your email domain. This process allows the SPF check to pass automatically as the mail routes from amazonses.com. To use a custom MAIL FROM domain, add it in Feebak under the advanced settings for your domain.\n\n{% hint style=\"info\" %}\nNote: If you use your own MAIL FROM domain, you must meet Amazon SES specifications as outlined under the Choosing a MAIL FROM Domain section of [Using a Custom MAIL FROM Domain with Amazon SES](https://docs.aws.amazon.com/ses/latest/dg/mail-from.html) in the Amazon SES Developer Guide.\n{% endhint %}\n\nGeneral Procedure to Add SPF Records:\n\n1. In Feebak, navigate to SPF DMARC Authentication under your domain records page in the Advanced Settings tab.\n2. Under SPF DMARC Authentication (Custom MAIL FROM), add the fully qualified domain name and click Save. Feebak returns the MAIL FROM values.\n3. Copy the values and in your DNS provider’s website, sign in and navigate to the page for updating your domain’s DNS records.\n4. Follow your provider’s instructions for adding SPF records.\n5. Save your changes.\n6. If verification fails, click Reset to regenerate tokens.\n\n{% hint style=\"info\" %}\nAn organization can add up to 3 domains simultaneously.\n{% endhint %}"
  },
  {
    "instruction": "Delete Custom Domain <a href=\"#delete-email-template\" id=\"delete-email-template\"></a>",
    "input": "",
    "output": "1. Navigate to Administrator > Custom Email Sender > Custom Domain.\n2. Selecting Custom Domain will list all the added domains.\n3. View and Delete buttons are present along with the Domain.\n4. Click on the Delete button to delete the respective Domain.\n\n<figure><img src=\"../../.gitbook/assets/Screenshot 2024-07-24 131629.png\" alt=\"\" width=\"539\"><figcaption></figcaption></figure>\n\n5. Click Yes, Delete to confirm deletion."
  },
  {
    "instruction": "Custom Email",
    "input": "",
    "output": "The Custom Email feature in Feebak allows organizations to add their own email addresses for sending email surveys. This feature enhances flexibility and professionalism by enabling the use of verified custom email addresses through Amazon SES (Simple Email Service).\n\nWhen an email survey is sent, it has two addresses that indicate its source: a \"From address\" displayed to the message recipient and a \"MAIL FROM address\" indicating where the message originated. Mail servers use the MAIL FROM address to return bounce messages and other error notifications. Surveys sent through the custom email address configured in Feebak will automatically use a subdomain of _amazonses.com_ as the default MAIL FROM domain."
  },
  {
    "instruction": "Permissions",
    "input": "",
    "output": "To manage custom email senders, specific Feebak permissions are required:\n\n CustomEmailManagement\\_View: Required to access the Custom Email Sender section.\n CustomEmailManagement\\_Edit: Required to add or edit custom email addresses.\n\nYou can add these permissions in Feebak by following the steps below:\n\n Administration > Roles\n Create a new role or edit an existing role\n Add permissions\n Assign it to the users.&#x20;"
  },
  {
    "instruction": "Add Custom Email Addresses",
    "input": "",
    "output": "Users with the \"CustomEmailManagement\\_Edit\" permission can add or edit custom email senders.\n\nYou can configure this Custom Email under Administration > Organisation > Custom Email Sender > Custom E-mail.\n\n<figure><img src=\"../../.gitbook/assets/custom-email.png\" alt=\"\"><figcaption></figcaption></figure>\n\nBy clicking on the Add New button, a new window will be opened where you can give your custom email address.\n\n<figure><img src=\"../../.gitbook/assets/image (315).png\" alt=\"\"><figcaption></figcaption></figure>\n\nWhen a custom email address is added, a verification email is sent to that address. The user must verify the email by clicking a link in the verification email.\n\nOnce verified, these custom email addresses can be used in email surveys, allowing organizations to send surveys from their own email addresses.\n\n<figure><img src=\"../../.gitbook/assets/image (40).png\" alt=\"\"><figcaption></figcaption></figure>\n\n{% hint style=\"info\" %}\nAn organization can add up to 10 email addresses. Organizations can manage multiple email addresses and select the most appropriate sender for each survey.\n{% endhint %}"
  },
  {
    "instruction": "Edit Custom Email Addresses",
    "input": "",
    "output": "1. Navigate to Administrator > Custom Email Sender > Custom Email.\n2. Selecting Custom Email Sender will list all the added email addresses.\n3. Edit and Delete buttons are present along with the addresses.\n4. Click on the Edit button to edit the respective address.\n\n<figure><img src=\"../../.gitbook/assets/Screenshot 2024-07-24 122812.png\" alt=\"\"><figcaption></figcaption></figure>\n\n5. Once you have edited an already existing email address, a new verification link will be send to that address.&#x20;"
  },
  {
    "instruction": "Delete Custom Email Address <a href=\"#delete-email-template\" id=\"delete-email-template\"></a>",
    "input": "",
    "output": "1. Navigate to Administrator > Custom Email Sender > Custom Email.\n2. Selecting Custom Email Sender will list all the added templates.\n3. Edit and Delete buttons are present along with the Address.\n4. Click on the Delete button to delete the respective Email Address.\n\n<figure><img src=\"../../.gitbook/assets/Screenshot 2024-07-24 122937.png\" alt=\"\"><figcaption></figcaption></figure>\n\n5. Click Yes, Delete to confirm deletion."
  },
  {
    "instruction": "Custom Email Sender",
    "input": "",
    "output": "The Custom Email Sender feature enhances flexibility and professionalism by enabling the use of verified custom email addresses and domains through Amazon SES (Simple Email Service).\n\n [Custom Domain](custom-domain.md): Allows organizations to add their own domains for sending email surveys. Follow the steps to add, verify, and optionally configure DKIM authentication and custom MAIL FROM addresses.\n [Custom Email](custom-email.md): Allows organizations to add their own email addresses for sending email surveys. Follow the steps to add, verify, edit, and delete custom email addresses.\n\nThis integration allows organizations to use their own email addresses and domains for sending email surveys, ensuring that surveys appear more professional and are less likely to be marked as spam."
  },
  {
    "instruction": "How to Integrate Division",
    "input": "",
    "output": "The integration process consists of three main steps, each designed to align your Feebak setup with your Genesys Cloud environment:\n\n1. Import Divisions: This step brings your Genesys Cloud division structure into Feebak. It allows you to mirror your organizational hierarchy and ensures that Feebak can properly categorize and manage data based on your existing divisions.\n2. Sync User Details: After importing divisions, this step updates user information in Feebak to match Genesys Cloud. It ensures that all user accounts, roles, and associations are current, allowing for accurate reporting and access control within Feebak.\n3. Sync Queue Details: The final step involves updating queue information in Feebak. This synchronization ensures that your Feebak system accurately reflects the current queue structure in Genesys Cloud, which is crucial for proper routing and reporting of customer interactions.\n\nBy completing these three steps in order, you'll create a seamless integration between Genesys Cloud and Feebak, ensuring that your feedback management system accurately reflects your current operational structure. Let's explore each step in detail to guide you through the process."
  },
  {
    "instruction": "Import Divisions",
    "input": "",
    "output": "1. Navigate to the Administration section in the navigation bar. Under Organization, find and click on Divisions.\n2. To import divisions from Genesys Cloud, click the Import Divisions button.\n3. It displays all divisions from your Genesys Cloud organization, including ones that are already imported.\n\n<figure><img src=\"../../.gitbook/assets/Screenshot 2024-08-29 155643.png\" alt=\"\"><figcaption></figcaption></figure>\n\n4. Select one or more divisions from the list and click Import Selected button at the top right corner of the screen.\n5. Once the selected divisions are imported, you can view their names in the Imported Divisions' list."
  },
  {
    "instruction": "Sync User Details",
    "input": "",
    "output": "1. Navigate to User Administration > User.\n2. To sync the latest user information from Genesys Cloud, click the Sync User Details button.\n3. Click Yes, Sync Division Details to proceed with the update.\n4. Feebak will update user information to reflect any changes made in Genesys Cloud.\n\n<figure><img src=\"../../.gitbook/assets/image (34).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "Sync Queue Details",
    "input": "",
    "output": "1. Navigate to the Administration section in the navigation bar. Under Organization, find and click on Queues.\n2. Click the Sync Queues button.\n3. When prompted, confirm by clicking Yes, Sync Queue Details.\n4. Feebak will update queue information to match your current setup in Genesys Cloud.\n\n<figure><img src=\"../../.gitbook/assets/image (36).png\" alt=\"\"><figcaption></figcaption></figure>\n\nBy completing these three steps, you'll ensure that your Feebak setup accurately reflects the divisions, users, and queues in your Genesys Cloud environment. Remember to perform these sync operations periodically or whenever changes are made in Genesys Cloud to maintain data consistency.\n\nWith divisions now integrated, you can assign surveys to specific divisions:\n\n1. Navigate to a Survey.\n2. Go to Publish Settings tab.\n3. Select the appropriate division for each survey from the dropdown.\n\nThis allows you to tailor your feedback collection process to your organizational structure, ensuring that surveys are distributed and managed according to your divisions."
  },
  {
    "instruction": "Division Management",
    "input": "",
    "output": "Divisions in Feebak help you segment data, users, queues, and access permissions to align with your organizational structure. Feebak allows import of divisions from your Genesys organisation."
  },
  {
    "instruction": "View Divisions <a href=\"#view-users\" id=\"view-users\"></a>",
    "input": "",
    "output": "1. Once you are logged in, different options can be viewed in the Navigation bar in which, Divisions is present in the Administration section, under Organization.\n2. You can view all the imported Divisions here.\n3. You can also search Divisions by name.\n4. Delete button is present along with the name of the division.\n\n<figure><img src=\"../../.gitbook/assets/image (364).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "View Division Details <a href=\"#view-users\" id=\"view-users\"></a>",
    "input": "",
    "output": "1. Once you are logged in, different options can be viewed in the Navigation bar in which, Divisions is present in the Administration section, under Organization.\n2. You can view all the imported Divisions here.\n3. To explore a division's details, simply select the division.\n4. You'll be able to view the Users, Surveys, Queues and Outbounds here.\n\n<figure><img src=\"../../.gitbook/assets/image (31).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "Import Divisions <a href=\"#import-genesys-cloud-users\" id=\"import-genesys-cloud-users\"></a>",
    "input": "",
    "output": "{% hint style=\"info\" %}\nGenesys Cloud Divisions can be directly imported and used in Feebak.&#x20;\n{% endhint %}\n\n1. Once you are logged in, different options can be viewed in the Navigation bar in which, Divisions is present in the Administration section, under Organization.\n2. To import divisions from Genesys Cloud, click the Import Divisions button.\n3. It displays all divisions from your Genesys Cloud organization, including ones that are already imported.\n\n<figure><img src=\"../../.gitbook/assets/Screenshot 2024-08-29 155643.png\" alt=\"\"><figcaption></figcaption></figure>\n\n4. Select one or more divisions from the list and click Import Selected button at the top right corner of the screen.\n5. Once the selected divisions are imported, you can view their names in the Imported Divisions' list."
  },
  {
    "instruction": "Sync Division Details <a href=\"#import-genesys-cloud-users\" id=\"import-genesys-cloud-users\"></a>",
    "input": "",
    "output": "{% hint style=\"info\" %}\nThis process updates the names of all divisions in Feebak to match any changes made in Genesys Cloud.\n{% endhint %}\n\n1. Once you are logged in, different options can be viewed in the Navigation bar in which, Divisions is present in the Administration section, under Organization.\n2. To sync the latest division information from Genesys Cloud, click the Sync Division Details button.\n3. Click Yes, Sync division details to proceed with the update.\n\n<figure><img src=\"../../.gitbook/assets/image (33).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "Delete Division <a href=\"#view-users\" id=\"view-users\"></a>",
    "input": "",
    "output": "{% hint style=\"info\" %}\nNote:\n\nYou cannot delete a division if it has any assigned Users, Surveys, Queues, or Outbounds. Ensure that all associated elements are removed or reassigned before attempting to delete a division.\n{% endhint %}\n\n1. Once you are logged in, different options can be viewed in the Navigation bar in which, Divisions is present in the Administration section, under Organization.\n2. You can view all the imported Divisions here.\n3. To delete a particular division, click on the Delete Division button along with name of the division.\n4. Click on Yes, Delete Division button in order to delete the division.\n\n<figure><img src=\"../../.gitbook/assets/image (32).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "Divisions overview",
    "input": "",
    "output": "Divisions in Feebak are designed to help you streamline your operations by organizing your organization's resources into manageable segments. Whether you need to separate data by department, location, or any other criterion, divisions make it easier to manage access and ensure that users interact with the right set of data.\n\nHow Divisions Work: Divisions allow you to group resources such as surveys, users, queues, and outbound interactions into specific segments. For instance, if your organization operates in multiple regions or departments, you can create divisions for each and assign relevant resources to them.\n\nManaging Access Through Roles: Roles in Feebak can be linked to specific divisions, meaning that users with those roles will only have access to the data within their assigned divisions. This ensures that users interact only with the information relevant to their division, enhancing both security and efficiency. For example, a user with the \"Survey Admin\" role in the \"Sales\" division will only be able to manage surveys within that division.\n\nImpact on Dashboards and Reports: The division-based setup also influences what is displayed in your dashboards and reports. Data is filtered based on the divisions assigned to the user, ensuring that only the relevant interactions and survey results are visible. This approach helps maintain a focused and organized workflow for users across different segments.\n\nDivision-Aware Entities: In Feebak, several entities are division-aware, meaning they can be linked to specific divisions. These include:\n\n Surveys\n Users\n Queues\n Outbounds\n\nThis setup allows you to assign these resources to the relevant divisions, ensuring that they are managed according to the organization's structure.\n\nWorking with Divisions:\n\n Assigning Divisions: When assigning roles to users, you can specify which divisions they should have access to. By default, roles are set to \"All Divisions,\" but this can be adjusted to meet specific needs. Refer [Roles Management](../roles-management.md#edit-roles).\n Division Management: Feebak provides a dedicated page for managing imported divisions. Here, you can import, sync, and delete divisions as needed. Refer [Division Management](./).\n Impacted Pages: Several pages within Feebak, such as the User Detail, Role Detail, and Survey Assignments pages, are influenced by the divisions assigned to users. This ensures that all actions and views are consistent with the division-based setup.\n\nBy utilizing divisions in Feebak, you can effectively manage and organize your resources, ensuring that users have the appropriate access and that data is segmented according to your organization's needs. This not only enhances operational efficiency but also maintains data security and relevance across different segments."
  },
  {
    "instruction": "Survey Assignments",
    "input": "",
    "output": "Survey Assignments dictate when and to whom surveys are sent by assigning them to specific queues based on various conditions. These conditions include the date, participant data, interaction type, queue priority, and agent."
  },
  {
    "instruction": "**View Survey Assignments**",
    "input": "",
    "output": "1. Once you are logged in, different options can be viewed in the navigation bar in which Survey Assignments is present in the Administration section.\n2. Selecting Queues will list all the imported queues.\n3. You can subscribe to a queue by clicking the button left to the Queue Name.&#x20;\n\n{% hint style=\"info\" %}\nQueues without survey activations cannot be subscribed.\n{% endhint %}\n\n4. Clicking View Survey Assignments button along with the queue name, displays the survey activations.\n\n![](<../.gitbook/assets/Screenshot 2025-01-21 110223.png>)\n\n5. The queue contains surveys which can be activated as well as deactivated."
  },
  {
    "instruction": "**Add survey activation**",
    "input": "",
    "output": "1. Once you are logged in, different options can be viewed in the navigation bar in which Survey Assignments is present in the Administration section.\n2. Select a queue in order to view the surveys.\n3. Click on Add Survey to Queue at the top right corner of the screen in order to add new survey to the queue.\n4. Select the survey from the list given right.\n\n![](<../.gitbook/assets/Screenshot 2025-01-21 110443.png>)\n\n5. You can also add new conditions to the survey. The conditions include:\n\n<table data-header-hidden data-full-width=\"false\"><thead><tr><th></th></tr></thead><tbody><tr><td>Date</td></tr><tr><td>Participant Data</td></tr><tr><td>Queue Priority</td></tr><tr><td>Queue waiting time</td></tr><tr><td>User</td></tr><tr><td>Interaction Type</td></tr></tbody></table>"
  },
  {
    "instruction": "**Date**",
    "input": "",
    "output": "![](<../.gitbook/assets/Screenshot 2025-01-21 110752.png>)\n\nIn this condition, select Date Range, Days of Week and Time Range."
  },
  {
    "instruction": "**Participant data**",
    "input": "",
    "output": "![](<../.gitbook/assets/Screenshot 2025-01-21 111149.png>)\n\nIn this condition, select Condition Type, Operator, Attribute and Value."
  },
  {
    "instruction": "**Queue Priority**",
    "input": "",
    "output": "![](<../.gitbook/assets/Screenshot 2025-01-21 111228.png>)\n\nIn this condition, select Condition Type, Operator and Value."
  },
  {
    "instruction": "**Queue waiting time**",
    "input": "",
    "output": "![](<../.gitbook/assets/Screenshot 2025-01-21 111309.png>)\n\nIn this condition, select Condition Type, Operator and Value."
  },
  {
    "instruction": "**User**",
    "input": "",
    "output": "This Condition allows you to perform respective actions only if the user / agent is the selected one.\n\n<figure><img src=\"../.gitbook/assets/image (3).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "&#x20;**Delete Survey Assignments**&#x20;",
    "input": "",
    "output": "1. Navigate to Administration > Survey Assignments.\n2. Selecting a queue will display list of surveys.\n3. You can click on the delete button to delete the respective survey from the queue.\n\n![](<../.gitbook/assets/Screenshot 2025-01-21 111631.png>)\n\n4. Click on Yes, Delete Activation to delete the selected survey."
  },
  {
    "instruction": "Bulk Survey Assignment",
    "input": "",
    "output": "You can use this feature to a survey and conditions to multiple queues at the same time. Click on the Add Survey to Multiple Queues button.\n\n<figure><img src=\"../.gitbook/assets/image (5).png\" alt=\"\"><figcaption></figcaption></figure>\n\nSelect the queues you want to assign to from the dropdown, select a survey and select the conditions you want to assign. To view the existing assignments for the selected survey, click on the View Activations button.\n\n<figure><img src=\"../.gitbook/assets/image (4).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "Outbound Campaigns (Agentless)",
    "input": "",
    "output": "{% hint style=\"info\" %}\nPrerequisites\n\nThe following permissions should be assigned to the Feebak Client role to work with Outbound Surveys.\n\n Outbound > Contact List > Add\n Outbound > Contact List > View\n Outbound > Contact List > Search\n Outbound > Response Set > All Permissions\n Routing > Wrap-up Code > Add\n Conversation > Communication > All Permissions\n{% endhint %}"
  },
  {
    "instruction": "Add an Outbound Survey",
    "input": "",
    "output": "For agentless outbound campaigns, first, you have to add an outbound survey.\n\n1. Navigate to Administration > Outbound.\n\n![](<../.gitbook/assets/Screenshot (74).png>)\n\n2. This will list all the existing Outbound surveys.\n3. Click on the Add New Outbound button.\n\n![](<../.gitbook/assets/outbound 2 (1).png>)\n\n4. Select a survey from the dropdown, and click on save to add the outbound survey.\n5. It will take around a minute to publish an outbound survey. You will be notified once it's published."
  },
  {
    "instruction": "Add Outbound Campaign in Genesys",
    "input": "",
    "output": "Once an outbound survey is added in Feebak, it will automatically create a Response Set in Genesys Cloud. You can use it to create an Outbound Campaign for the survey added in the previous step."
  },
  {
    "instruction": "Steps to add Outbound Campaign",
    "input": "",
    "output": "1. In Genesys Cloud, navigate to Admin > Outbound > Campaign Management.\n\n![](../.gitbook/assets/campaign-list.png)\n\n2. Under the Voice Campaigns tab, click on Create New button.\n3. In Create Campaign Page, under Dialing Mode Options change the Dialing Mode to Agentless Dialing.\n4. In the Call Response field, select the outbound survey added in Feebak.&#x20;\n\n![](../.gitbook/assets/campaign-add.png)\n\n5. Under the Campaign Options, select a Contact List. The contact list contains the list of persons you want to call. If you don't already have a contact list, you can add it from Admin > Outbound > List Management. Please refer to [this documentation](https://help.mypurecloud.com/articles/create-new-contact-list/).&#x20;\n\n![](../.gitbook/assets/contactlist-select.png)\n\n{% hint style=\"info\" %}\nContact List must have a column named Language. Otherwise, the survey won't work.\n{% endhint %}\n\n6. You can fill the rest of the fields based on how you want your campaign. If you're not sure about any of the fields, please refer to [this documentation](https://help.mypurecloud.com/articles/create-agentless-campaign/).\n7. Click on save to create the campaign.\n\nOnce the campaign is added, you can turn it on, and the outbound survey will be offered to every person on the contact list."
  },
  {
    "instruction": "Setting Language of the Outbound Survey",
    "input": "",
    "output": "You can set the language of the outbound survey by adding a field called Language in your Contact List. Feebak will automatically read the value from the Language column and will set that as the language of the survey.\n\n![](../.gitbook/assets/contactlist-example.png)\n\nFor the contact list shown in the above image, the survey will be offered for Person 1 in Spanish and Person 2 in English."
  },
  {
    "instruction": "Audit Logs",
    "input": "",
    "output": "The Audit Log page in Feebak is designed to provide details of administrative activities within the product. This page is crucial for monitoring and tracking the actions taken by users across different services within Feebak. This log helps administrators and users track changes, monitor access, and ensure compliance with regulations or internal policies.\n\nThe audit log records and displays the following details for each event:&#x20;\n\n1. User: The individual who performed the action.&#x20;\n2. IP Address: The IP address from which the action was performed.&#x20;\n3. Action: The specific action that was taken.&#x20;\n4. Service: The service within Feebak where the action occurred, such as Surveys, Queues, Users, and Roles.&#x20;\n5. Entity Name: The entity name is referred as the name of the item in the service which is affected by the action.\n6. Entity Type: The entity type is referred as the type of the item in the service which is affected by the action.\n7. Time: The timestamp indicates when the action occurred.&#x20;\n8. Changes: A detailed description of the changes made during the action.&#x20;"
  },
  {
    "instruction": "The Services that are recorded:",
    "input": "",
    "output": "1. Surveys\n2. Users\n3. Queues\n4. Survey Assignments&#x20;\n5. Roles&#x20;\n6. Data Actions&#x20;\n7. Licenses&#x20;\n8. Prompts&#x20;\n9. Message Platforms&#x20;"
  },
  {
    "instruction": "Permissions:",
    "input": "",
    "output": "To view the audit logs in Feebak, the user must have the AuditLogs\\_View permission in Feebak."
  },
  {
    "instruction": "View Audit Logs:",
    "input": "",
    "output": "To access the Audit Log page in Feebak, navigate to the Administration menu on the left sidebar, then select \"Audit Logs\" under the Organization section.&#x20;\n\n<figure><img src=\"../.gitbook/assets/image (323).png\" alt=\"\"><figcaption></figcaption></figure>\n\n&#x20;To view more details, including the changes made, click on the specific event on the Audit Log page.\n\n<figure><img src=\"../.gitbook/assets/image (324).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "Gamification",
    "input": "",
    "output": "Genesys Cloud Gamification enhances motivation and engagement among contact center agents by transforming everyday tasks into friendly competitions."
  },
  {
    "instruction": "Key Components",
    "input": "",
    "output": "1. Metrics and Scores:\n    Agent Score: This represents the performance of individual agents based on specific tasks or criteria.\n    Survey Score: Reflects the outcomes or feedback gathered from customer surveys.\n2. Permissions:\n\n Feebak Permission:\n   To access gamification settings, users need the GamificationManagement\\_View permission in Feebak.\n   To edit gamification settings, users require the GamificationManagement\\_Edit permission in Feebak.\n   You can add these permission to any role in Feebak under Administration > Roles > Add Permissions and assign this role to the users.\n Genesys Cloud Permission\n   For viewing the gamification settings page in Feebak, you have to assign the following genesys cloud permissions to the user:\n     authorization:role:view\n     authorization:role:edit\n   For creating metrics and pushing scores to Genesys Cloud, you have to assign the following genesys cloud permissions to the Feebak Client Role:\n     employeePerformance:externalMetrics:add\n     employeePerformance:externalMetricData:add\n   You can add this permission from Feebak\n     Administration > Gamification Settings\n       There will be an Add Permission button. \\\n\n\n        <figure><img src=\"../.gitbook/assets/image (305).png\" alt=\"\"><figcaption></figcaption></figure>\n       Clicking on it will display a confirmation window\\\n\n\n        <figure><img src=\"../.gitbook/assets/image (307).png\" alt=\"\"><figcaption></figcaption></figure>\n\n3. Integration:\n\n Assigning the necessary permissions in Feebak automatically creates two metrics in Genesys Cloud: \"Feebak Agent Score\" and \"Feebak Survey Score\".\n These scores can be integrated into the organization's profiles, enabling a seamless flow of performance data.\n\nOnce these permissions are added, the Gamification Settings page will be accessible.\n\n<figure><img src=\"../.gitbook/assets/image (308).png\" alt=\"\"><figcaption></figcaption></figure>\n\n4. Configuration Options:\n\nOrganizations can define when the scores should be pushed to Genesys Cloud. For that, there is an Update Option in which there are two choices:\n\n Real Time: Scores are pushed to the metrics immediately after each survey is completed.\n End of Day: Scores are pushed at 12:00 AM according to the organization's time zone, summarizing the day's performance."
  },
  {
    "instruction": "Introduction",
    "input": "",
    "output": "1\\. Once you are logged in, you can view the supervisor dashboard by default.\n\n2\\. You can change the dashboard interval between Daily, Weekly, Monthly or Yearly. Know more about the working of Dashboard with the following guide:\n\n{% content-ref url=\"dashboard/\" %}\n[dashboard](dashboard/)\n{% endcontent-ref %}\n\n3\\. Next is the Report section from where you can Generate report of Survey Score Details and Survey Answer Details. To know more about report generation, follow the guide:\n\n{% content-ref url=\"report-generation/\" %}\n[report-generation](report-generation/)\n{% endcontent-ref %}\n\n4\\. Next comes the administration section which is categorized in to four sections namely,\n\ni.  Organizations&#x20;\n\nii. Templates\n\niii. Survey\n\niv. User Administration"
  },
  {
    "instruction": "**Organization**",
    "input": "",
    "output": "Includes Organization Settings and License.&#x20;\n\nIn Organization Settings you can view , add and edit the organization details and the contact details of the organization head.&#x20;\n\nIn the License section, you can view and manage the user license by clicking Manage User License button at the top right corner of your screen. Know more about managing user license with the following guide:\n\n{% content-ref url=\"../feebak-subscription/assigning-feebak-licences-to-users.md\" %}\n[assigning-feebak-licences-to-users.md](../feebak-subscription/assigning-feebak-licences-to-users.md)\n{% endcontent-ref %}"
  },
  {
    "instruction": "**Templates**",
    "input": "",
    "output": "This section includes Email Template and the Notification Template.\n\nIn the Email Template section, You can View, Add, Edit and Delete all the added email templates, click on the Add New button on the top right corner of your screen to add a new email template. Know more about managing email template with the following guide:\n\n{% content-ref url=\"../feebak-administration/email-templates.md\" %}\n[email-templates.md](../feebak-administration/email-templates.md)\n{% endcontent-ref %}\n\nIn the Notification Template section, you can view, add, edit and delete all the added notification templates. Click on the Add New button on the top right corner of your screen to add a new notification template. Know more about notification templates with the following guide:\n\n{% content-ref url=\"../feebak-administration/notification-templates.md\" %}\n[notification-templates.md](../feebak-administration/notification-templates.md)\n{% endcontent-ref %}"
  },
  {
    "instruction": "**Survey**",
    "input": "",
    "output": "This section includes Scoring Categories, Prompts, Surveys, Queues, Survey Assignments.\n\nScoring Categories allows you to view, add, edit and delete different categories of scores by which response of the customer as well as performance of the agents are evaluated. Follow the guide to know more about Scoring Categories:\n\n{% content-ref url=\"scoring-category.md\" %}\n[scoring-category.md](scoring-category.md)\n{% endcontent-ref %}\n\nPrompts allows you to view, add, edit and delete different prompts which are used in different surveys. Follow the guide to know more about Prompts:\n\n{% content-ref url=\"prompts.md\" %}\n[prompts.md](prompts.md)\n{% endcontent-ref %}\n\nSurveys allows you to view, add, edit and delete surveys which are used to define how the calls would work. Follow the guide to know more about Surveys:\n\n{% content-ref url=\"surveys/\" %}\n[surveys](surveys/)\n{% endcontent-ref %}\n\nQueues allows you to view, import and delete different queues of survey assignments. You can subscribe or unsubscribe queues as well as each of the survey assignments in it. Follow the guide to know more about Queues:\n\n{% content-ref url=\"queues.md\" %}\n[queues.md](queues.md)\n{% endcontent-ref %}\n\nSurvey Assignments section allows you to select a queue and view all the survey assignments in it. Follow the guide to know more about Survey Assignments:\n\n{% content-ref url=\"survey-assignments.md\" %}\n[survey-assignments.md](survey-assignments.md)\n{% endcontent-ref %}"
  },
  {
    "instruction": "**User Administration**",
    "input": "",
    "output": "This section includes User and Roles.\n\nUser section allows you to view the list of all registered users. You can Import users from list as well as add a new user by clicking on Import User and Add New User buttons at the top right corner of your screen respectively. Deletion of users is also possible. Follow the guide to know more about Survey Assignments:\n\n{% content-ref url=\"../feebak-administration/user-management/\" %}\n[user-management](../feebak-administration/user-management/)\n{% endcontent-ref %}\n\nRoles section allows you to view, add and edit all the roles. The role of Organization admin cannot be edited or deleted. Follow the guide to know more about Survey Assignments:\n\n{% content-ref url=\"../feebak-administration/roles-management.md\" %}\n[roles-management.md](../feebak-administration/roles-management.md)\n{% endcontent-ref %}"
  },
  {
    "instruction": "Queues",
    "input": "",
    "output": "## View Queues\n\n1. Once you are logged in, different options can be viewed in the navigation bar in which Queues is present in the Administration section.\n2. Selecting Queues will list all the imported queues.\n3. You can subscribe a queue by clicking the button left to the Queue Name. Please note that queues without survey activations cannot be subscribed.\n4. Clicking View Survey Assignments button along with the queue name, displays the survey activations.\n\n<figure><img src=\"../.gitbook/assets/image (30).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "**Import Queues**",
    "input": "",
    "output": "1. Navigate to Administration > Queues.\n2. Selecting Queues will list all the imported queues by default.\n3. Click on the Import Queues button and select queues to be imported.\n4. Click on the Import Selected button to import the selected queues.\n\n![](<../.gitbook/assets/image (249).png>)"
  },
  {
    "instruction": "**Delete Queues**",
    "input": "",
    "output": "1. Navigate to Administration > Queues.\n2. Selecting Queues will list all the imported queues by default.\n3. Queues can be deleted by clicking Delete Queue or you can click on Delete Multiple Queues and select the queues to be deleted.\n\n![](<../.gitbook/assets/image (170).png>)\n\n4. Clicking on Yes, Delete Queue will delete the selected queue."
  },
  {
    "instruction": "Individual Outbound Calls",
    "input": "",
    "output": "### Start an Individual Conversation\n\n1.  In Genesys Cloud, navigate to Interactions > Start a Conversation.\\\n\n\n    <figure><img src=\"../.gitbook/assets/image (126).png\" alt=\"\" width=\"367\"><figcaption></figcaption></figure>\n2.  Select the queue and enter the recipient's number or email.\n\n    <figure><img src=\"../.gitbook/assets/image (127).png\" alt=\"\" width=\"358\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "Setting Language for Outbound Calls",
    "input": "",
    "output": "You can accept the language of an outbound call using GetSurveyAddressToTransfer data action with a language parameter. You can use this parameter to create multiple buttons by giving different languages as the input parameter which will allow you to transfer to a single survey in the respective languages.\n\nFor each of the buttons, you have to execute a data action from the script.\n\nSince we can't update the input parameters in the existing data action in Genesys cloud, you have to create a copy of the 'Get Survey Address To Transfer' data action and add the language parameter."
  },
  {
    "instruction": "Steps to Create Data Action for adding buttons for Different Languages",
    "input": "",
    "output": "1. In Genesys Cloud, navigate to Admin > Actions.\n2. Search for \"Get Survey Address to Transfer\" in Actions.\n3.  Export the existing data action.\\\n\n\n    <figure><img src=\"../.gitbook/assets/image (144).png\" alt=\"\"><figcaption></figcaption></figure>\n4. Go back to Actions and add a new action.\n5.  Select the Integration Name as per your choice and input an Action Name.\\\n\n\n    <figure><img src=\"../.gitbook/assets/image (119).png\" alt=\"\"><figcaption></figcaption></figure>\n6.  Import the previously exported file by clicking the Import Button.\\\n\n\n    <figure><img src=\"../.gitbook/assets/image (135).png\" alt=\"\"><figcaption></figcaption></figure>\n7. Select the Setup Tab.\n8.  In the Contracts menu, add another parameter as string under the Object in Input Contract and give the string name as \"language\".\\\n\n\n    <figure><img src=\"../.gitbook/assets/image (129).png\" alt=\"\"><figcaption></figcaption></figure>\n9.  In the Configuration menu, add \"\\&lang=${input.language}\" at the end of the Request URL template and then Save & Publish the action.\\\n\n\n    <figure><img src=\"../.gitbook/assets/image (110).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "**Steps to add buttons to the Script:**",
    "input": "",
    "output": "1.  Click on the Action button in the script and Add a new Custom Action.\\\n\n\n    <figure><img src=\"../.gitbook/assets/image (148).png\" alt=\"\"><figcaption></figcaption></figure>\n2.  Give a Custom Action Name. Eg: Offer English Survey.\\\n\n\n    <figure><img src=\"../.gitbook/assets/image (156).png\" alt=\"\"><figcaption></figcaption></figure>\n3. Add Execute Data Action as the first step.\n4. Select the newly added Data Action to it by selecting the Category and Data Action.\n5.  In the Input field, give _\"{\"ConversationID\": \"\\{{Scripter.Interaction ID\\}}\", \"AgentIdentifier\": \"\\{{Scripter.Agent ID\\}}\" }\"_ in the callJson field, and in the language field, give the required language code. Eg: _en-us._\\\n\n\n    <figure><img src=\"../.gitbook/assets/image (108).png\" alt=\"\"><figcaption></figcaption></figure>\n\n    The language codes which are supported by the Feebak are given below:\\\n\n\n    <figure><img src=\"../.gitbook/assets/image (130).png\" alt=\"\"><figcaption></figcaption></figure>\n6.  Select Success, Address, and Message in the Output field in the respective fields.\\\n\n\n    <figure><img src=\"../.gitbook/assets/image (167).png\" alt=\"\"><figcaption></figcaption></figure>\n7.  Add an If/Else as the next step, and make the Success parameter is true.\\\n\n\n    <figure><img src=\"../.gitbook/assets/image (131).png\" alt=\"\"><figcaption></figcaption></figure>\n8.  Add a new step Scripter.Blind Transfer to Address.\\\n\n\n    <figure><img src=\"../.gitbook/assets/image (153).png\" alt=\"\"><figcaption></figcaption></figure>\n9.  In the Else part, add a Scripter.Alert with the Alert text as the Message variable and then Save the Action.\\\n\n\n    <figure><img src=\"../.gitbook/assets/image (122).png\" alt=\"\"><figcaption></figcaption></figure>\n10. Add a button to the script and give a proper button name to identify the language. Eg: Offer English survey.\n11. Add this custom action as the click action of the button.\\\n\n\n    <figure><img src=\"../.gitbook/assets/image (104).png\" alt=\"\"><figcaption></figcaption></figure>\n12. Save and Publish the button.\\\n\n\n    <figure><img src=\"../.gitbook/assets/image (109).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "Outbound Campaigns (With Agents)",
    "input": "",
    "output": "## Add an Outbound Campaign in Genesys\n\n1. In Genesys Cloud, navigate to Admin > Outbound > Campaign Management.\n\n![](../.gitbook/assets/campaign-list.png)\n\n2. Under the Voice Campaigns tab, click on Create New button.\n3. In Create Campaign Page, under Dialing Mode Options select a Dialing Mode and Queue. You can use any dialing mode other than agentless dialing.\n\n![](../.gitbook/assets/agent-preview.png)\n\n4. For the Agent Script, select the Script created for offering surveys. If you haven't created the script for offering surveys, please refer to [this documentation](https://docs.feebak.com/kb/creating-scripts) and add it.\n5. Under the Campaign Options, select a Contact List. The contact list contains the list of persons you want to call. If you don't already have a contact list, you can add it from Admin > Outbound > List Management. Please refer to [this documentation](https://help.mypurecloud.com/articles/create-new-contact-list/).\n\n![](<../.gitbook/assets/image (279).png>)\n\n6. You can fill the rest of the fields based on how you want your campaign. If you're not sure about any of the fields, please refer to [this documentation](https://help.mypurecloud.com/articles/create-agentless-campaign/).\n7. Click on \"Save\" to create the campaign.\n\nOnce the campaign is added, you can turn it on, and outbound calls will be added to all the agents on the queue. Once the agent is done speaking to the customer, he or she can click on the Offer Survey button in the script to offer the survey. You can read more about offering surveys on [this page](https://docs.feebak.com/kb/call-transferring-survey)."
  },
  {
    "instruction": "Custom Conversation Type",
    "input": "",
    "output": "## Introduction\n\nThis documentation outlines the functionality of a feature called Custom Conversation Type within our platform. Custom conversation types provide flexibility when generating surveys via our Digital Survey API and offer the ability to categorize interactions uniquely. This guide explains how to define, use, and manage custom conversation types."
  },
  {
    "instruction": "Overview",
    "input": "",
    "output": "Purpose\n\nCustom conversation types are designed to enhance the survey generation process for API users. These custom types allow users to categorize interactions in a way that aligns with their specific requirements.\n\nIntegration\n\nCustom conversation types are integrated into our dashboard and reports, enabling users to filter surveys based on these custom types. This ensures a seamless experience for tracking and analyzing data.\n\n{% hint style=\"info\" %}\nUsers are not allowed to create custom conversation types that replicate existing conversation types, such as WhatsApp, Chat, Webmessage, Facebook, etc.\n{% endhint %}\n\nPermissions Required\n\nUsers should be assigned the following permissions inside Feebak to access the custom conversation type:\n\n1. ConversationTypeManagement\\_Add&#x20;\n2. ConversationTypeManagement\\_View&#x20;\n3. ConversationTypeManagement\\_Delete"
  },
  {
    "instruction": "Adding Custom Conversation Types",
    "input": "",
    "output": "1.  Once you are logged in, different options can be viewed in the navigation bar, including Custom Conversation Type, in the Administration section.\\\n\n\n    <figure><img src=\"../.gitbook/assets/image (99).png\" alt=\"\"><figcaption></figcaption></figure>\n2.  Click on the Add New Type button.\\\n\n\n    <figure><img src=\"../.gitbook/assets/image (100).png\" alt=\"\"><figcaption></figcaption></figure>\n3. You can give a conversation type such as \"Website,\" \"Kiosk,\" or any other unique categories you require and click on Add (Users are not allowed to create custom conversation types that replicate existing conversation types)."
  },
  {
    "instruction": "Delete Custom Conversation Types",
    "input": "",
    "output": "1. Navigate to Administration > Custom Conversation Type.\n2. It will list all the available custom conversation types by default.\n3.  Conversation types can be deleted by clicking on the Delete button.\\\n    &#x20;\n\n    <figure><img src=\"../.gitbook/assets/image (97).png\" alt=\"\"><figcaption></figcaption></figure>\n4. Clicking on Yes, Delete Type will delete the selected queue."
  },
  {
    "instruction": "Dashboards and Reports",
    "input": "",
    "output": "Custom conversation types are seamlessly integrated into our dashboard and reports.\n\n<figure><img src=\"../.gitbook/assets/image (291).png\" alt=\"\"><figcaption></figcaption></figure>\n\nUsers can filter surveys by these custom types, making data analysis and tracking more efficient.\n\n{% hint style=\"info\" %}\nNote: If you rename a custom conversation type, please be aware that this change will affect only future records. Historical records will remain associated with the original custom conversation type name.\n{% endhint %}\n\nYou can refer the documentation for the usage of custom conversation types in Feebak Digital Survey API here:[ Custom Conversation Type in Feebak API](../api/digital-survey-api/digital-survey-api-version-2.md#custom-conversation-type)."
  },
  {
    "instruction": "Prompts",
    "input": "",
    "output": "Prompts are specific instructions or messages displayed or played to participants during a survey. They can be customized to guide respondents effectively through the survey process, ensuring a seamless and personalized experience.\n\nPersonalized Prompts\n\nFeebak supports personalized prompts, allowing you to tailor messages for each participant. These prompts can be used in Text-to-Speech (TTS) for voice surveys or displayed as text prompts in digital surveys. By leveraging personalized prompts, you can dynamically include relevant details such as customer names or specific interaction information.\n\nAvailable Dynamic Variables\n\nBelow are the dynamic variables you can use in personalized prompts and TTS:\n\n| Variable            | Description                                                                                 |\n| ----------------------- | ----------------------------------------------------------------------------------------------- |\n| {{CustomerName}}      | Customer Name – Extracted based on the interaction channel.                                     |\n| {{Attributes.{Name}}} | Custom Attributes – Replace {Name} with the attribute key (e.g., {{Attributes.Name}}).      |\n| {{Q{Number}.Answer}}  | Previous Question Answer – Replace {Number} with the question number (e.g., {{Q1.Answer}}). |\n| {{Agent}}             | Agent Name.                                                                                     |\n| {{Queue}}             | Queue Name.                                                                                     |\n| {{CallerAddress}}     | Caller Address.                                                                                 |\n| {{Type}}              | Interaction Type.                                                                               |\n| {{CustomerId}}        | Customer ID from Genesys interaction.                                                           |\n| {{ExternalContactId}} | External Contact ID configured for the customer in Genesys.                                     |"
  },
  {
    "instruction": "**View Prompts**",
    "input": "",
    "output": "1. Once logged in, navigate to the Administration section in the navigation bar and select Prompts.\n2. By default, all prompts will be listed.\n3. You can search for prompts by name using the search bar.\n4. Next to each prompt name, you will find Edit and Delete buttons along with its details.\n\n![](<../.gitbook/assets/image (141).png>)"
  },
  {
    "instruction": "**Add Prompts**",
    "input": "",
    "output": "1. Navigate to Administration > Prompts.\n2. By default, all existing prompts will be listed.\n3. Click on the Add New Prompt button.\n4. Fill out the fields for Prompt Name, Group, Description, and Audio Resources.\n5. After filling out the fields, you can select language from the list and clicking Upload Audio Clip here allows you to upload the voice clip.\n\n<figure><img src=\"../.gitbook/assets/image (333).png\" alt=\"\"><figcaption></figcaption></figure>\n\n6. In the Text-to-Speech (TTS) field, you can enter any text you want to be played as a prompt during the survey. The given text will be converted to audio and played during the survey.\n\n<figure><img src=\"../.gitbook/assets/image (6).png\" alt=\"\"><figcaption></figcaption></figure>\n\n7. To use personalized prompts, type @ in the TTS field and select the required dynamic variable from the list. This allows you to embed dynamic content, such as customer names or interaction details, into your survey prompts.\n8. You can also add new resources which can include new audio clips by clicking Add New Resource.\n9. Finally by clicking the button Add Prompt, you can add the prompt to the prompts list.\n\n{% hint style=\"info\" %}\nNote: You don't need to upload an audio file and fill in the Text-to-Speech field. You can choose to either upload an audio file or simply provide the text to be played as audio. Only one of these actions is necessary to create a prompt.\n{% endhint %}"
  },
  {
    "instruction": "**Edit Prompts**",
    "input": "",
    "output": "1. Navigate to Administration > Prompts.\n2. By default, all existing prompts will be listed.\n3. Click the Edit button next to the name of the prompt to view and modify its details, including the resources it contains.\n4. Click the Play Audio button to hear the uploaded audio.\n5. Edit the text provided as the prompt if needed.\n6. You can add or delete resources.\n7.  Click on Check Usage in Surveys to see how the prompt is used in surveys.\\\n\n\n    <figure><img src=\"../.gitbook/assets/image (343).png\" alt=\"\"><figcaption></figcaption></figure>\n\n{% hint style=\"info\" %}\nNote: If a prompt used in a survey is updated, the survey must be republished to reflect these changes.\n{% endhint %}"
  },
  {
    "instruction": "**Delete Prompts**",
    "input": "",
    "output": "1. Navigate to Administration > Prompts.\n2. By default, all existing prompts will be listed.\n3. Click the Delete button next to the name of the prompt.\n4. A confirmation dialog box will appear, allowing you to confirm or cancel the deletion.\n\n![](<../.gitbook/assets/image (149).png>)"
  },
  {
    "instruction": "Scoring Category",
    "input": "",
    "output": "Score categories help break down the feedback into meaningful insights, such as customer satisfaction (CSAT) or Net Promoter Score (NPS). Feebak also allows you to define custom score categories to suit your organization’s needs."
  },
  {
    "instruction": "**View Scoring Categories**",
    "input": "",
    "output": "1. After logging in, from the navigation bar, select Scoring Categories in the Administration section.\n2. By default, all scoring categories will be listed.\n3. You can search for categories by name using the search bar.\n4. Each category has Edit and Delete buttons next to its name for easy modification or removal.\n\n![](../.gitbook/assets/s-1.PNG)"
  },
  {
    "instruction": "**Add Scoring Categories**",
    "input": "",
    "output": "1. Navigate to Administration > Scoring Categories.\n2. By default, all existing categories will be listed.\n3. Click on the Add New Category button.\n4. A new category entry will appear, showing the default level along with its upper and lower bounds.\n\n![](../.gitbook/assets/s-2.PNG)\n\n5. You can add new level by clicking Add New Level button.\n6. The displayed window allows you to enter a Level Name and Lower Bound to the new scoring and also you can select a color to be displayed representing the new level.\n7. Clicking the Add button will add a new level to the selected category.\n\n![](../.gitbook/assets/s-3.PNG)\n\n8. Different levels can be added and viewed along with the default level when a category is selected.\n\n![](<../.gitbook/assets/image (234).png>)"
  },
  {
    "instruction": "**Edit Scoring Categories**",
    "input": "",
    "output": "1. Navigate to Administration > Scoring Categories.\n2. By default, all existing categories will be listed.\n3. Next to each category name, you will find Edit and Delete buttons.\n4. Click the Edit button to view and modify the levels within the category.\n5. You can edit the Level name, Lower Bound, Upper Bound and even the color which represents it."
  },
  {
    "instruction": "**Delete Scoring Categories**",
    "input": "",
    "output": "1. Navigate to Administration > Scoring Categories.\n2. By default, all existing categories will be listed.\n3. Next to each category name, you will find Edit and Delete buttons.\n4. Click the Delete button to open a confirmation dialogue box.\n\n![](../.gitbook/assets/s-5.PNG)\n\n5. In the dialogue box, you can choose to delete the category by clicking Yes, Delete Category. This will remove the category and all its details.\n6. When you select a category, all its levels are displayed. You can also delete individual levels.\n\n![](../.gitbook/assets/s-6.PNG)\n\n7. To delete a level, click the Delete button next to the level name and confirm by clicking Yes, Delete Level. This will remove the level and its details."
  },
  {
    "instruction": "Survey Answer Details Report",
    "input": "",
    "output": "Dive into individual survey responses with this report to understand detailed feedback for each question. Analyze the answers to uncover customer preferences, concerns, and sentiments.\n\n1. Once you are logged in, navigate to Survey Answer Details in the Report section.\n2. Enter the dates of which the report has to be generated, and select a survey. You can filter by Survey State here.&#x20;\n3. Click the Include Attributes checkbox to include participant data in the report.\n\n![](<../../.gitbook/assets/Screenshot 2024-12-07 131606.png>)\n\n4. If you prefer to select specific columns, uncheck the Include All Columns checkbox. Then, click the Add Columns button and choose the columns you wish to include.\n\n<figure><img src=\"../../.gitbook/assets/image (421).png\" alt=\"\"><figcaption></figcaption></figure>\n\n5. Click on the Generate Report button. And a pop-up window will appear.\n\n<figure><img src=\"../../.gitbook/assets/Screenshot 2024-12-07 132139.png\" alt=\"\"><figcaption></figcaption></figure>\n\n6. Report Name (optional): You can give a name for the report.\n7. Email address (optional): You can provide upto 10 email addresses to which the report should be sent after generation. Click Add Email after entering email addresses.\n8. After giving email addresses, click Generate to generate the report.\n9. A link to download the report will be sent to the specified email addresses.\n\nThe Report Name and Emails fields are optional. Users can generate a report directly by clicking Generate without providing a Report Name or Email Addresses. These reports will be accessible on the Report History page, where users can download them. Click on the View History button to go to the Report History page.\n\n{% hint style=\"info\" %}\nNote: The link for downloading the report will be valid for up to 7 days. If you need the link beyond this period, you can regenerate the report once again.\n{% endhint %}"
  },
  {
    "instruction": "Scheduling Reports",
    "input": "",
    "output": "1. In the Survey Answer Details page, click on Schedule For Later.\n2. Select a Survey, Time Period, Recurrence Option and Time to generate reports. You can schedule reports to be generated on various time periods that are recurring daily or weekly, at any time of the day. You can also select which all days of the week and what time of the day for daily reports, which day of the week for weekly reports and which date of the month for monthly reports. You can also filter by Survey State.\n3. Click the Include Attributes checkbox to include participant data in the report.\n\n<figure><img src=\"../../.gitbook/assets/Screenshot 2024-12-07 132400.png\" alt=\"\"><figcaption></figcaption></figure>\n\n4. If you prefer to select specific columns, uncheck the Include All Columns checkbox. Then, click the Add Columns button and choose the columns you wish to include.\n\n<figure><img src=\"../../.gitbook/assets/image (422).png\" alt=\"\"><figcaption></figcaption></figure>\n\n5. Click the Schedule Report button. And a pop-up window will appear.\n\n<figure><img src=\"../../.gitbook/assets/Screenshot 2024-12-07 132734.png\" alt=\"\"><figcaption></figcaption></figure>\n\n6. Report name (optional): You can give a name for the report.\n7. Email address (optional): You can provide upto 10 email addresses to which the report should be sent. Click Add Email after entering email addresses.\n8. After entering the email addresses, click on Generate button to schedule the report.\n9. A link to download the report will be sent to the specified email addresses periodically, according to your choice.\n\n{% hint style=\"info\" %}\nNote: Scheduled reports are disabled after 100 days, after which the reports need to be rescheduled.\n{% endhint %}"
  },
  {
    "instruction": "Reports",
    "input": "",
    "output": "Whether you're looking to review the overall performance of a survey or dive into specific answers and trends, reports offer comprehensive data for informed decision-making.\n\nFeebak's reporting capabilities are designed to provide both a high-level overview and granular details, giving you the tools needed to enhance your customer experience strategy."
  },
  {
    "instruction": "Survey Score Details Report",
    "input": "",
    "output": "The Survey Score Details Report allows you to track and review the score data from your surveys. This report helps you monitor the performance of all surveys by showing the scoring details, making it easier to identify trends and areas that require attention. With these insights, you can gauge customer satisfaction and the effectiveness of your services based on the scoring criteria you set in your surveys. Learn more:\n\n{% content-ref url=\"survey-score-details-1.md\" %}\n[survey-score-details-1.md](survey-score-details-1.md)\n{% endcontent-ref %}"
  },
  {
    "instruction": "Survey Answer Details Report",
    "input": "",
    "output": "The Survey Answer Details Report provides a deep dive into the individual responses of your selected surveys. This report is designed to give you a clear understanding of the feedback collected by breaking down the answers to each question. By analyzing these details, you can uncover valuable insights about customer preferences, concerns, and overall sentiment. Learn more:\n\n{% content-ref url=\"survey-answer-details.md\" %}\n[survey-answer-details.md](survey-answer-details.md)\n{% endcontent-ref %}"
  },
  {
    "instruction": "Report History",
    "input": "",
    "output": "The Report History section gives you an overview of previously generated reports, allowing you to revisit past analyses and track performance over time. Learn more:\n\n{% content-ref url=\"report-history.md\" %}\n[report-history.md](report-history.md)\n{% endcontent-ref %}"
  },
  {
    "instruction": "Scheduled Reports",
    "input": "",
    "output": "The Scheduled Reports feature enables you to automate the generation and delivery of reports at regular intervals. By scheduling reports, you can ensure that you always have up-to-date data, delivered directly to your inbox or any designated location. Learn more:\n\n{% content-ref url=\"scheduled-reports.md\" %}\n[scheduled-reports.md](scheduled-reports.md)\n{% endcontent-ref %}"
  },
  {
    "instruction": "Scheduled Reports",
    "input": "",
    "output": "Automate report generation and delivery with this feature to receive regular updates. Stay informed with timely data sent directly to your inbox or preferred location.\n\n1. Once you are logged in, navigate to Scheduled Reports in the Report section.\n2. Select a report from the Report Name dropdown.&#x20;\n3. You can filter by recurring status:\n    Daily\n    Weekly\n    Monthly\n4. You can activate / deactivate reports, and you can also download the reports here.\n\n<figure><img src=\"../../.gitbook/assets/image (413).png\" alt=\"\"><figcaption></figcaption></figure>\n\n5. To edit the schedule of a report, click the edit button of a scheduled report.\n\n<figure><img src=\"../../.gitbook/assets/Screenshot 2024-10-10 111156.png\" alt=\"\"><figcaption></figcaption></figure>\n\n6. To save the schedule after making changes, click on the Save button."
  },
  {
    "instruction": "Survey Score Details Report",
    "input": "",
    "output": "This report allows you to monitor and analyze the scoring data from your surveys. Use it to identify trends and assess customer satisfaction based on the scores generated.\n\n1. Once you are logged in, navigate to Survey Score Details in the Report section.\n2. Enter the interval detail&#x73;.&#x20;\n\n![](<../../.gitbook/assets/Screenshot 2024-12-07 125407.png>)\n\n3. You can also use filters such as Divisions, Surveys, Queues, Survey State, and Survey Score to generate reports.&#x20;\n4. Click the Include Attributes checkbox to include participant data in the report.\n\n<figure><img src=\"../../.gitbook/assets/Screenshot 2024-12-07 131359.png\" alt=\"\"><figcaption></figcaption></figure>\n\n5. If you prefer to select specific columns, uncheck the Include All Columns checkbox. Then, click the Add Columns button and choose the columns you wish to include.\n6. Click the Generate Report button. And a pop-up window will appear.\n\n<figure><img src=\"../../.gitbook/assets/Screenshot 2024-12-07 125425.png\" alt=\"\"><figcaption></figcaption></figure>\n\n7. Report name (optional): You can give a name for the report.\n8. Email address (optional): You can provide upto 10 email addresses to which the report should be sent after generation. Click Add Email after entering email addresses.\n9. After entering the email addresses, click on Generate button to generate the report.\n10. A link to download the report will be sent to the specified email addresses.\n\nThe Report Name and Emails fields are optional. Users can generate a report directly by clicking Generate without providing a Report Name or Email Addresses. These reports will be accessible on the Report History page, where users can download them. Click on the View History button to go to the Report History page.\n\n{% hint style=\"info\" %}\nNote: The link for downloading the report will be valid for up to 7 days. If you need the link beyond this period, you can regenerate the report once again.\n{% endhint %}"
  },
  {
    "instruction": "Scheduling Reports",
    "input": "",
    "output": "1. In the Survey Score Details page, click on Schedule For Later.\n2. Enter the interval detail&#x73;.\n\n<figure><img src=\"../../.gitbook/assets/Screenshot 2024-12-07 125618.png\" alt=\"\"><figcaption></figcaption></figure>\n\n3. You can select Time Period, Recurrence Option and Time to generate reports. You can schedule reports to be generated on various time periods that are recurring daily or weekly, at any time of the day. You can also select which all days of the week and what time of the day for daily reports, which day of the week for weekly reports and which date of the month for monthly reports.\n4. Click the Include Attributes checkbox to include participant data in the report.\n\n<figure><img src=\"../../.gitbook/assets/image (420).png\" alt=\"\"><figcaption></figcaption></figure>\n\n5. If you prefer to select specific columns, uncheck the Include All Columns checkbox. Then, click the Add Columns button and choose the columns you wish to include.\n6. You can use filters such as Divisions, Surveys, Queues, Survey State, and Survey Score as well.&#x20;\n7. Click the Schedule Report button. And a pop-up window will appear.\n\n<figure><img src=\"../../.gitbook/assets/Screenshot 2024-12-07 125705.png\" alt=\"\"><figcaption></figcaption></figure>\n\n8. Report name (optional): You can give a name for the report.\n9. Email address (optional): You can provide upto 10 email addresses to which the report should be sent. Click Add Email after entering email addresses.\n10. After entering the email addresses, click on Generate button to schedule the report.\n11. A link to download the report will be sent to the specified email addresses periodically, according to your choice.\n\n{% hint style=\"info\" %}\nNote: Scheduled reports are disabled after 100 days, after which the reports need to be rescheduled.\n{% endhint %}"
  },
  {
    "instruction": "Report History",
    "input": "",
    "output": "This section provides access to previously generated reports, helping you track historical data and revisit past analyses for performance evaluation.\n\n1. Once you are logged in, navigate to Report History in the Report section.\n2. Select a report from the Report Name dropdown.&#x20;\n3. You can filter by report status:\n    Completed\n    Requested\n    Scheduled\n    Cancelled\n    Failed\n4. You can also download the reports here.\n\n<figure><img src=\"../../.gitbook/assets/image (411).png\" alt=\"\"><figcaption></figcaption></figure>\n\n5. To edit the schedule of a scheduled report here, click on View Schedule.\n\n<figure><img src=\"../../.gitbook/assets/Screenshot 2024-10-10 111156.png\" alt=\"\"><figcaption></figcaption></figure>\n\n6. To save the schedule after making changes, click on the Save button."
  },
  {
    "instruction": "Agent Dashboard (legacy)",
    "input": "",
    "output": "<figure><img src=\"../../.gitbook/assets/image (23).png\" alt=\"\"><figcaption></figcaption></figure>\n\nYour Rank and Score can be viewed along with Evaluations and Unoffered Surveys.\n\nList of agents according to their rank is listed where the number of evaluations, unoffered surveys and respective scores can be known."
  },
  {
    "instruction": "Survey Score Summary",
    "input": "",
    "output": "The Survey Score Summary offers an at-a-glance visualization of average ratings and score distributions across surveys. Interactive charts and filters allow you to drill down into specific surveys, time periods, or score ranges to uncover key trends and performance indicators.\n\nNavigate to Dashboard > Survey Score Summary.\n\nThe Dashboard Survey Score Summary provides a comprehensive overview of survey scores based on various metrics. It offers visual representations of data in the form of graphs and charts, enabling users to quickly identify trends and patterns.\n\nDashboard Survey Score Summary includes the number of interactions for each survey and the average score for that survey.\n\n<figure><img src=\"../../.gitbook/assets/image (235).png\" alt=\"\"><figcaption></figcaption></figure>\n\n Number of Interactions: The dashboard displays the number of interactions received for each survey. This metric indicates the level of customer engagement and the volume of feedback received for a particular survey.\n Average Score: The dashboard also presents the average score for each survey. This score reflects the overall satisfaction level of customers who participated in the survey.\n Score Distribution: Graphs displays the distribution of survey scores across different rating levels, such as Excellent, Good, Average, Poor, etc. This helps to gauge overall customer satisfaction and identify areas for improvement."
  },
  {
    "instruction": "**Filtering the Dashboard**&#x20;",
    "input": "",
    "output": "Users can filter the dashboard contents to focus on specific criteria. This can be done by Interval, Interaction Type, Direction Type, Divisions, Agents, Queues, Survey Status, Survey Language, and Survey. By applying these filters, users can narrow down the data displayed in the charts and graphs to gain more specific insights.\n\n![](<../../.gitbook/assets/Screenshot 2024-08-28 161649.png>)"
  },
  {
    "instruction": "**Drill Down**&#x20;",
    "input": "",
    "output": "The Drill Down capability allows users to filter out specific results from the graphs. By clicking on a particular color in the graph representing the score categories, users can access the Interaction Dashboard. This dashboard lists out the interactions associated with the score category represented by the selected color.\n\n<figure><img src=\"../../.gitbook/assets/image (173).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "NPS Summary",
    "input": "",
    "output": "The NPS Summary offers an in-depth view of Net Promoter Score (NPS) trends across surveys. Visual charts display overall NPS scores to track which areas of service or product are driving customer loyalty or detracting from it. You can apply filters to view specific time periods, surveys, interaction type, divisions, etc.\n\nNavigate to Dashboard > NPS Summary\n\n{% hint style=\"info\" %}\nNPS Summary dashboard is only available when you have survey questions having NPS question type.\n{% endhint %}\n\n<figure><img src=\"../../.gitbook/assets/image (386).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "Sentiment Score Summary",
    "input": "",
    "output": "Navigate to Dashboard > Sentiment Score Summary\n\n<figure><img src=\"../../.gitbook/assets/image (408).png\" alt=\"\"><figcaption></figcaption></figure>\n\nDrill down to any specific survey from here, by clicking on their names, to view the Survey Response Summary for the survey."
  },
  {
    "instruction": "Survey Response Summary",
    "input": "",
    "output": "The Survey Response Summary provides a detailed breakdown of the number and types of responses received from selected surveys. With the help of visual charts, you can easily track participation rates and analyze response patterns. The dashboard also allows question-wise tracking, offering deeper insights into how respondents answered each question.\n\nNavigate to Dashboard > Survey Response Summary. Then select a survey from the dropdown.\n\n<figure><img src=\"../../.gitbook/assets/image (388).png\" alt=\"\"><figcaption></figcaption></figure>\n\nYou can view Survey Score Summary, Sentiment Score, and question-wise response summary here."
  },
  {
    "instruction": "Agent Dashboard",
    "input": "",
    "output": "Acts as a daily scoreboard that ranks agents based on agent scores also shows metrics like evaluations, completed surveys, and unoffered surveys. This dashboard can be visible to agents, allowing them to track their own performance and rankings in real time.\n\nNavigate to Dashboard > Agent Dashboard\n\n<figure><img src=\"../../.gitbook/assets/image (23).png\" alt=\"\"><figcaption></figcaption></figure>\n\n1. Rank: Agent's rank for the day.\n2. Score: Agent's score for the day.\n3. Evaluations: Total number of surveys offered by the agent that are considered for calculating agent scores, for the day.\n4. Unoffered Surveys: Total number of calls for which Surveys are not offered, for the day."
  },
  {
    "instruction": "Survey Activity Timeline",
    "input": "",
    "output": "The Survey Activity Timeline provides an overview of all survey activities, allowing you to track response rates and survey completion trends. Charts make it easier to spot patterns, while filters enable a more focused analysis based on timeframes, survey types, or respondent categories.\n\n1\\. Click the Dashboard icon to view the supervisor dashboard.\n\n2\\. The dashboard displays details, including the surveys that are Answered (completed and partial), Abandoned, Expired, Active, and Offered.&#x20;\n\n1. Answered (completed): An answered (completed) survey is one in which the customer has responded to all questions or required sections.\n2. Answered (partial): An answered (partial) survey is one in which at least one of the questions is answered and left without completing the survey. This will only be counted if the partial survey feature is enabled while configuring the surveys.&#x20;\n3. Abandoned Surveys: An abandoned survey refers to a survey that remains incomplete due to the customer not fully answering it (the partial survey feature is disabled). This includes situations where the customer opens the survey link but leaves it without providing responses to any questions.&#x20;\n4. Expired Surveys: An expired survey is one that has not been opened or accessed by the customer before the specified expiration time set during its configuration. Once the predefined time limit for taking the survey has passed without the customer accessing it, the survey is considered expired.&#x20;\n5. Active Surveys: A survey is considered active if it has been created, the customer has not opened the survey link and the survey link has not expired.&#x20;\n6. Total Offered Surveys: Total offered surveys consist of Answered (Completed), Answered (Partial), Active (records currently in the offered state), Expired, and Abandoned surveys.\n\n<figure><img src=\"../../.gitbook/assets/image (360).png\" alt=\"\"><figcaption></figcaption></figure>\n\n3\\. Score Trend, Agent Score Trend, NPS Score Trend and CSAT Score Trend can be viewed.\n\n4\\. On the right side, you can select a date, or select daily, weekly, monthly, and yearly dashboards. The option for live update is also available.\\\n\n\n<figure><img src=\"../../.gitbook/assets/image (312).png\" alt=\"\"><figcaption></figcaption></figure>\n\n5\\. The above image shows Score Trend and Agent Score Trend of different surveys in graphical form. Here, each color represents different surveys.\n\nScore Trend\n\nA) Each survey’s dashboard can be viewed and calls are categorized into Bad, Good and Excellent.\n\nB) Total calls and average score also can be viewed.\n\nC) At the right side, Queue and Survey languages can be selected.\n\n![Score of all the questions from the selected survey can be viewed here](../../.gitbook/assets/d-3.PNG)\n\nAgent Score Trend\n\n![](../../.gitbook/assets/d-4.PNG)\n\nA) At the right side, Queue and Survey languages can be selected.\n\nB) The below image shows NPS Score Trend and CSAT Score Trend of different surveys in graphical form. Here, each color represents different surveys.\n\n![](../../.gitbook/assets/d-5.PNG)\n\nNPS Score Trend\n\n![](../../.gitbook/assets/d-6.PNG)\n\nA) In the above picture, a survey under Net Promoter System is shown.\n\nB) The rating of Promoters, Passives and Detractors can be viewed in percentage.\n\n![Detailed view of Customer Satisfactory Score](../../.gitbook/assets/d-7.PNG)"
  },
  {
    "instruction": "Interactions",
    "input": "",
    "output": "The Interactions Dashboard offers a detailed view of customer interactions across various channels. It helps you identify engagement patterns, evaluate service quality, and uncover areas for improvement. With filtering options, you can drill down to specific time periods, channels, or interaction types for a more granular analysis.\n\nNavigate to Dashboard > Interactions.\n\n<figure><img src=\"../../.gitbook/assets/Screenshot 2024-11-21 165533.png\" alt=\"\"><figcaption></figcaption></figure>\n\nDashboard Layout\n\nThe dashboard contains a table with the following column headings:\n\n Date\n Time\n Agent\n Survey\n Interaction Type\n Survey Result\n Survey Score\n Survey State\n Queue\n Contact (optional)\n\nClick on the list button to customize the table view by selecting or deselecting columns.\n\n<figure><img src=\"../../.gitbook/assets/Screenshot 2024-11-20 112245.png\" alt=\"\"><figcaption></figcaption></figure>\n\nInteraction Details\n\nEach interaction entry in the table includes:\n\n Question Answers: Detailed responses from surveys.\n Conversation ID: Genesys Cloud ID of the interaction.\n Customer Information: Name, ID, Attributes and sentiment scores (especially for voice interactions).\n Survey Details: Name, Score, Results, Language, Handle Time, etc.&#x20;\n Agent Information: Name, Handling Time, etc.\n Queue Information: Queue name, Waiting time, Priority, Direction, etc.\n Division Information: Division name.\n\n<figure><img src=\"../../.gitbook/assets/Screenshot 2024-11-21 165714.png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "Modifying Agent and Queue in Interaction Details",
    "input": "",
    "output": "Users with the \"SurveyRecordsManagement\\_Edit\" permission can modify agent and queue in interaction details.\n\nWe store the history of agents and queues involved in a conversation. This history allows you to accurately track and update the details of interactions. In the interaction detail view page, you can change the agent and queue, allowing selection from the stored history.\n\nIf there are multiple agents and queues in an interaction, the survey will be tracked against the last agent or queue and displayed in the interaction details. If you need to change the agent or queue, an edit option is provided alongside the agent and queue names.\n\n<figure><img src=\"../../.gitbook/assets/image (14).png\" alt=\"\"><figcaption></figcaption></figure>\n\nFilters\n\nTo refine the data displayed, several filters are available. Users can select the interval for the data, choosing between daily, weekly, yearly, or a custom date range. Interaction Types such as call, chat, Facebook, and web message can be filtered to focus on specific channels. Additionally, users can filter interactions based on direction type, whether they are inbound or outbound. Survey languages can also be used as a filter, with options like English, German, Arabic, and Spanish. Divisions can also be used as a filter.\\\n\n\n<figure><img src=\"../../.gitbook/assets/image (345).png\" alt=\"\"><figcaption></figcaption></figure>\n\nSome filters allow users to search and type specific criteria. These include Feedback Text, Queues, Surveys, Agents, Survey Results, and Reference ID, providing flexibility in finding specific interactions.\n\n<figure><img src=\"../../.gitbook/assets/image (354).png\" alt=\"\"><figcaption></figcaption></figure>\n\nThe Survey Score filter enables users to set criteria using operators such as equal to, greater than, less than, and others, combined with a numerical value.\n\n<figure><img src=\"../../.gitbook/assets/image (353).png\" alt=\"\"><figcaption></figcaption></figure>\n\nA calendar option in the top right corner allows users to pick a specific date for the interactions displayed.&#x20;\n\n<figure><img src=\"../../.gitbook/assets/image (15).png\" alt=\"\"><figcaption></figcaption></figure>\n\nReset filters button is available to clear all applied filters.&#x20;\n\n<figure><img src=\"../../.gitbook/assets/image (16).png\" alt=\"\"><figcaption></figcaption></figure>\n\nExport button is available to export the data with all the applied filters.\n\n<figure><img src=\"../../.gitbook/assets/image (17).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "Dashboard",
    "input": "",
    "output": "Dashboards help you monitor key metrics, analyze feedback, track survey effectiveness, and provide comprehensive performance overviews for both agents and overall customer experience metrics."
  },
  {
    "instruction": "**Interactions Dashboard**",
    "input": "",
    "output": "The Interactions Dashboard offers a detailed view of customer interactions across various channels. It helps you identify engagement patterns, evaluate service quality, and uncover areas for improvement. With filtering options, you can drill down to specific time periods, channels, or interaction types for a more granular analysis. Learn more:\n\n{% content-ref url=\"interactions.md\" %}\n[interactions.md](interactions.md)\n{% endcontent-ref %}"
  },
  {
    "instruction": "**Survey Activity Timeline**",
    "input": "",
    "output": "The Survey Activity Timeline provides an overview of all survey activities, allowing you to track response rates and survey completion trends. Charts make it easier to spot patterns, while filters enable a more focused analysis based on timeframes, survey types, or respondent categories. Learn more:\n\n{% content-ref url=\"supervisor-dashboard-1-1-1-1-1.md\" %}\n[supervisor-dashboard-1-1-1-1-1.md](supervisor-dashboard-1-1-1-1-1.md)\n{% endcontent-ref %}"
  },
  {
    "instruction": "**Survey Score Summary**",
    "input": "",
    "output": "The Survey Score Summary offers an at-a-glance visualization of average ratings and score distributions across surveys. Interactive charts and filters allow you to drill down into specific surveys, time periods, or score ranges to uncover key trends and performance indicators. Learn more:\n\n{% content-ref url=\"survey-score-summary.md\" %}\n[survey-score-summary.md](survey-score-summary.md)\n{% endcontent-ref %}"
  },
  {
    "instruction": "**Survey Response Summary**",
    "input": "",
    "output": "The Survey Response Summary provides a detailed breakdown of the number and types of responses received from selected surveys. With the help of visual charts, you can easily track participation rates and analyze response patterns. The dashboard also allows question-wise tracking, offering deeper insights into how respondents answered each question. Learn more:\n\n{% content-ref url=\"survey-response-summary.md\" %}\n[survey-response-summary.md](survey-response-summary.md)\n{% endcontent-ref %}"
  },
  {
    "instruction": "**NPS Summary**",
    "input": "",
    "output": "The NPS Summary offers an in-depth view of Net Promoter Score (NPS) trends across surveys. Visual charts display overall NPS scores to track which areas of service or product are driving customer loyalty or detracting from it. You can apply filters to view specific time periods, surveys, interaction type, divisions, etc. Learn more:\n\n{% content-ref url=\"nps-summary.md\" %}\n[nps-summary.md](nps-summary.md)\n{% endcontent-ref %}"
  },
  {
    "instruction": "**CSAT Summary**",
    "input": "",
    "output": "The CSAT Summary gives a comprehensive overview of Customer Satisfaction (CSAT) scores, with visual charts that track performance over time. This allows you to measure satisfaction in particular areas of your service and analyze customer happiness trends. Learn more:\n\n{% content-ref url=\"csat-summary.md\" %}\n[csat-summary.md](csat-summary.md)\n{% endcontent-ref %}"
  },
  {
    "instruction": "**Agent Scoreboard**",
    "input": "",
    "output": "The Agent Scoreboard dashboard highlights key performance metrics for agents, enabling supervisors to compare individual agent scores and track question-wise performance. Filters allow supervisors to focus on specific agents, questions, or time periods for a more detailed analysis. Learn more:\n\n{% content-ref url=\"agent-scoreboard.md\" %}\n[agent-scoreboard.md](agent-scoreboard.md)\n{% endcontent-ref %}"
  },
  {
    "instruction": "**Agent Dashboard**",
    "input": "",
    "output": "Acts as a daily scoreboard that ranks agents based on agent scores also shows metrics like evaluations, completed surveys, and unoffered surveys. This dashboard can be visible to agents, allowing them to track their own performance and rankings in real time. Learn more:\n\n{% content-ref url=\"agent-dashboard-1.md\" %}\n[agent-dashboard-1.md](agent-dashboard-1.md)\n{% endcontent-ref %}"
  },
  {
    "instruction": "**Sentiment Score Summary**",
    "input": "",
    "output": "Provides real-time analysis of customer feedback sentiment, categorizing responses into positive, neutral, and negative to reveal trends in customer emotions. This summary enables agents and supervisors to quickly gauge customer satisfaction and respond to shifting sentiments proactively. Learn more:\n\n{% content-ref url=\"sentiment-score-summary.md\" %}\n[sentiment-score-summary.md](sentiment-score-summary.md)\n{% endcontent-ref %}"
  },
  {
    "instruction": "**Open Feedback Summary**",
    "input": "",
    "output": "Displays an organized view of open feedback from recent surveys, helping supervisors to identify and address customer concerns efficiently. This summary supports timely follow-up and allows supervisors to monitor issues that require immediate attention. Learn more:\n\n{% content-ref url=\"open-feedback-summary.md\" %}\n[open-feedback-summary.md](open-feedback-summary.md)\n{% endcontent-ref %}"
  },
  {
    "instruction": "Agent Scoreboard",
    "input": "",
    "output": "The Agent Scoreboard dashboard highlights key performance metrics for agents, enabling supervisors to compare individual agent scores and track question-wise performance. Filters allow supervisors to focus on specific agents, questions, or time periods for a more detailed analysis.\n\nNavigate to Dashboard > Agent Scoreboard&#x20;\n\n<figure><img src=\"../../.gitbook/assets/image (18).png\" alt=\"\"><figcaption></figcaption></figure>\n\nTo refine the data displayed, several filters are available. You can select the interval for the data, choosing between daily, weekly, yearly, or a custom date range. Interaction Types such as call, chat, Facebook, and web message can be filtered to focus on specific channels. Additionally, users can filter interactions based on direction type, whether they are inbound or outbound.&#x20;\n\nSurvey languages can also be used as a filter, with options like English, German, Arabic, and Spanish. Divisions, Queues, Surveys and Agents can also be used as filters.\n\n<figure><img src=\"../../.gitbook/assets/image (406).png\" alt=\"\"><figcaption></figcaption></figure>\n\nYou can also filter by Agent Score with operators, Equals, Greater Than, Less Than, Less Than or Equals, and Greater than or Equals.\n\nYou can also export the data with all the applied filters using the Export button.\n\n<figure><img src=\"../../.gitbook/assets/image (19).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "Open Feedback Summary",
    "input": "",
    "output": "Navigate to Dashboard > Open Feedback Summary\n\n<figure><img src=\"../../.gitbook/assets/image (409).png\" alt=\"\"><figcaption></figcaption></figure>\n\nDrill down to any specific survey from here, by clicking on their names, to view the Survey Response Summary for the survey. Also, click on any frequently occuring phrases to drill down to the surveys where the phrases are mentioned, in the Inteactions dashboard."
  },
  {
    "instruction": "Configure words in the chart",
    "input": "",
    "output": "To configure the words displayed in the chart, follow these steps:\n\n1. Navigate to Administration > Dashboard Settings.\n2. Click on Add New.\n3. Enter the desired word and its associated Sentiment Marker Colour.\n4. After adding the words, only the configured words will appear in the chart. If not configured, the top 100 words are shown.\n\n<figure><img src=\"../../.gitbook/assets/image (22).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "CSAT Summary",
    "input": "",
    "output": "The CSAT Summary gives a comprehensive overview of Customer Satisfaction (CSAT) scores, with visual charts that track performance over time. This allows you to measure satisfaction in particular areas of your service and analyze customer happiness trends.\n\nNavigate to Dashboard > CSAT Summary\n\n{% hint style=\"info\" %}\nCSAT Summary dashboard is only available when you have survey questions having CSAT question type.\n{% endhint %}\n\n<figure><img src=\"../../.gitbook/assets/image (387).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "Survey Activity Timeline",
    "input": "",
    "output": "The Survey Activity Timeline provides an overview of all survey activities, allowing you to track response rates and survey completion trends. Charts make it easier to spot patterns, while filters enable a more focused analysis based on timeframes, survey types, or respondent categories.\n\nNavigate to Dashboard > Survey Activity Timeline.\n\nOn the right side, you can select a date range, or select daily, weekly, monthly, and yearly dashboards. The option for Live update is also available.\n\nThe following charts can be viewed:\n\n Survey Activity\n Sentiment Score\n Feedback Word Cloud\n Sentiment Trend\n Score Trend\n Agent Score Trend\n NPS Score Trend\n CSAT Score Trend"
  },
  {
    "instruction": "**Survey Activity**",
    "input": "",
    "output": "This chart displays details, including the surveys that are Answered (completed and partial), Abandoned, Expired, Active, and Offered.\n\n1. Answered (completed): An answered (completed) survey is one in which the customer has responded to all questions or required sections.\n2. Answered (partial): An answered (partial) survey is one in which at least one of the questions is answered and left without completing the survey. This will only be counted if the partial survey feature is enabled while configuring the surveys.&#x20;\n3. Abandoned Surveys: An abandoned survey refers to a survey that remains incomplete due to the customer not fully answering it (the partial survey feature is disabled). This includes situations where the customer opens the survey link but leaves it without providing responses to any questions.&#x20;\n4. Expired Surveys: An expired survey is one that has not been opened or accessed by the customer before the specified expiration time set during its configuration. Once the predefined time limit for taking the survey has passed without the customer accessing it, the survey is considered expired.&#x20;\n5. Active Surveys: A survey is considered active if it has been created, the customer has not opened the survey link and the survey link has not expired.&#x20;\n6. Total Offered Surveys: Total offered surveys consist of Answered (Completed), Answered (Partial), Active (records currently in the offered state), Expired, and Abandoned surveys.\n\n<figure><img src=\"../../.gitbook/assets/image (360).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "Score Trend & Agent Score Trend",
    "input": "",
    "output": "These charts show Trend of Survey Score and Agent Score different surveys. Here, each color represents different surveys.\n\n<figure><img src=\"../../.gitbook/assets/image (312).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "NPS Score Trend & CSAT Score Trend",
    "input": "",
    "output": "These charts show Trend of NPS Scores and CSAT Scores of different surveys. Here, each color represents different surveys.\n\n![](../../.gitbook/assets/d-5.PNG)"
  },
  {
    "instruction": "Sentiment Score, Feedback Word Cloud & Sentiment Trend",
    "input": "",
    "output": "These charts display sentiment score, frequent words as a wordcloud, and the trend of sentiment score, all based on the open feedback left by customers.&#x20;\n\n<figure><img src=\"../../.gitbook/assets/image (383).png\" alt=\"\"><figcaption></figcaption></figure>\n\n<figure><img src=\"../../.gitbook/assets/image (382).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "Principal Agent",
    "input": "",
    "output": "Feebak allows you to assign surveys to the correct agents based on the entire interaction history. Whether a call was handled by multiple agents or just one, you can accurately direct surveys to the most appropriate agent. In Feebak, you can update the agent and queue associated with an interaction using various methods. By default, the final agent and the queue active at the time of offering the survey will be recorded against an interaction stored in Feebak. However, there can be cases where you need to update this information to reflect, for example, the agent who had the longest conversation with the customer or the actual queue from which the call was transferred. Feebak allows updating interactions based on the history of agents and queues involved in the interaction.\n\nTo ensure surveys are assigned to the most appropriate agents and queues, you can use the following methods:"
  },
  {
    "instruction": "**Script Configuration**",
    "input": "",
    "output": "During script configuration, specify the agent and queue who should receive the survey.\n For step-by-step guidance, refer to the [Scripts Configuration](../../survey-scripts-configuration/creating-scripts-manual.md#using-the-principal-agent-feature) Guide."
  },
  {
    "instruction": "**Interaction Dashboard**",
    "input": "",
    "output": "View detailed interaction history, including all agents and queues involved.\n Select the queue and the specific agent to assign the survey based on the interaction history.\n For more information, refer to the [Interaction Details](../dashboard/interactions.md#modifying-agent-and-queue-in-interaction-details) Guide."
  },
  {
    "instruction": "**Digital Survey API**",
    "input": "",
    "output": "Specify an agent identifier and queue identifier when creating surveys.\n For detailed instructions, refer to the [Digital Survey API](../../api/digital-survey-api/digital-survey-api-version-2.md) Documentation.\n\nBy using these methods, you can ensure that your surveys are assigned to the appropriate agents and queues, improving the accuracy and effectiveness of your feedback collection process."
  },
  {
    "instruction": "Surveys",
    "input": "",
    "output": "{% hint style=\"info\" %}\nPrerequisites\n\nThe following permissions should be assigned to the user in Genesys Cloud for managing surveys.\n\n Architect > flow > All Permissions\n Architect > ui > View\n Integrations > action > View\n Telephony > Plugin > All\n{% endhint %}"
  },
  {
    "instruction": "&#x20;**View Survey**",
    "input": "",
    "output": "1. Once logged in, navigate to the Administration section in the navigation bar and select Surveys.\n2. By default, all existing surveys will be listed.\n3. You can search for surveys by name using the search bar.\n4. Next to each survey name, you will find Edit and Delete buttons, along with the status of the survey.\n\n![](<../../.gitbook/assets/image (157).png>)"
  },
  {
    "instruction": "Add Survey",
    "input": "",
    "output": "1. To add a new survey, click the Add New Survey button.\n2.  This will open the Survey Builder page, where you can create your survey.\\\n\n\n    <figure><img src=\"../../.gitbook/assets/image (334).png\" alt=\"\"><figcaption></figcaption></figure>\n3. In the General tab, fill in the basic details of the survey, such as Name, Description, Supported Languages, and Default Language.\n4.  Next, under Survey Types, you can select the channels for which you want to offer the survey, as shown below.\\\n\n\n    <figure><img src=\"../../.gitbook/assets/image (335).png\" alt=\"\"><figcaption></figcaption></figure>\n5. Next, select a Score Category from the score category picker. You can also add a new one by clicking the Add New button.\n\n<figure><img src=\"../../.gitbook/assets/image (336).png\" alt=\"\"><figcaption></figcaption></figure>\n\nFor detailed documentation on Score Categories, please refer to [this article](../scoring-category.md).\n\n6.  If Voice is selected, the Voice tab will be enabled in the survey info section. In the Voice tab, you should provide the basic prompts, which include the Greeting Prompt, No Entry Prompt, Invalid Entry Prompt, Expired Entry Prompt, and Closing Prompt. The Expired Entry Prompt is optional. \\\n\n\n    <figure><img src=\"../../.gitbook/assets/image (337).png\" alt=\"\"><figcaption></figcaption></figure>\n7. Greeting Prompt (Optional): A \"Greeting Prompt\" in a survey is the initial message that welcomes participants and sets the tone for the survey. Some common examples of greeting prompts include \"Thank you for participating in this survey,\" \"Your opinions are valuable to us,\" or \"We appreciate your time and effort in completing this survey.\"\n8. No Entry Prompt: A \"No Entry\" prompt is a message that plays in a survey when a participant attempts to skip a required question or leave a required field blank. The prompt usually instructs the participant to answer the question or complete the required field before proceeding with the survey.\n9. Invalid Entry Prompt: An \"Invalid Entry\" prompt is a message that plays in a survey when a participant provides an answer that is not valid or does not fit the expected format. This prompt usually instructs the participant to correct their answer and provide a valid response.\n10. Expired Entry Prompt: An Expired Entry Prompt is a message in a survey that plays when a participant has reached the maximum number of attempts to answer a question, but the question remains unanswered. This prompt informs the participant that their response will no longer be accepted and that the survey will close.\n11. Closing Prompt: A \"closing prompt\" of a survey is a message that plays when a participant has completed the survey or reached the end of the survey. The purpose of a closing prompt is to provide feedback to the participant, express gratitude for their participation, and give any additional instructions or information that may be necessary.\n\nYou can either select a prompt from the picker or add a new one using the Add New button.\n\n<figure><img src=\"../../.gitbook/assets/image (338).png\" alt=\"\"><figcaption></figcaption></figure>\n\nFor detailed documentation on adding Prompts, please refer to [this article](../prompts.md#add-prompts).\n\nYou can also enable Voice Feedback at the end of the survey by turning on the Use Voice Feedback toggle.\n\n<figure><img src=\"../../.gitbook/assets/image (339).png\" alt=\"\"><figcaption></figcaption></figure>\n\nIn the above image, Maximum Attempts is the number of times a question should be repeated if the user doesn't provide a valid input.\n\nThere is a TTS tab available.&#x20;\n\n<figure><img src=\"../../.gitbook/assets/image (344).png\" alt=\"\"><figcaption></figcaption></figure>\n\nIf you are using the TTS prompts (prompts created by giving text, not uploading the audio file), you can select a TTS engine and its corresponding voice for playing the TTS prompts in the survey.&#x20;\n\nThe Genesys default engine and its voice will be available. If you need any other third party engines like Microsoft Azure, you can configure it in Genesys Cloud and then use here.\n\n{% hint style=\"info\" %}\nNotes:\n\n If you are using multiple languages in a survey, you can select engine voice pairs for each languages.\n Default engine and voice will be used if no options are selected (you can configure the default engine in genesys cloud).\n{% endhint %}\n\n12. If any of the digital channels are selected in Survey Types, then the Survey Form tab will be enabled.\n\nIn the Survey Form tab, you should provide the basic details of the digital survey, which include the Greeting Text, and Closing Text.&#x20;\n\nYou can also customize the Color Scheme, Background Color, Logo, Text Color, Survey Page Title, and Survey Page Description. The Survey Page Title and Description will appear when you share the survey link on social media platforms, and the Logo will be displayed as the favicon for the survey page. \\\n\n\n<figure><img src=\"../../.gitbook/assets/Screenshot 2024-12-12 134156.png\" alt=\"\"><figcaption></figcaption></figure>\n\nYou can also accept Feedback Notes at the end of the survey by turning on Accept Feedback Notes toggle.\n\n13. Next, you can give Survey URL in which you can define the specific subdomain for your survey. The input you provide will be substituted as the subdomain in the survey link URL.\n14. Then, you can give Survey Link Expiry Time, in which you can define how long the digital survey URLs should remain active for the end customer to attend the survey. If no value is provided the survey URL will never expire.\\\n\n\n    <figure><img src=\"../../.gitbook/assets/image (342).png\" alt=\"\"><figcaption></figcaption></figure>\n15. Next, you can select the division to publish a survey in by going to the Publish Settings tab and selecting a division you have access to, from the dropdown.\n\n    <figure><img src=\"../../.gitbook/assets/image (28).png\" alt=\"\"><figcaption></figcaption></figure>\n16. Next, you can add the questions for the surveys. For each question, you should provide a Question Name, Question Type, Prompt (if voice is enabled), Text (If any digital channels are enabled) and Score Category.\n\n![](<../../.gitbook/assets/Screenshot 2024-11-20 113815.png>)\n\nYou can turn on or off, Use for Survey Scoring and Use for Agent Scoring for a question.&#x20;\n\nIf Use for Survey Scoring is turned on, then the answer score of the question will be used for the calculation of the total survey score.\n\nIf Use for Agent Scoring is turned on, then the answer score of the question will be used for the calculation of the Agent's performance score. You can turn it on if the question is related to evaluating an agent.\n\nThere are different types of questions in Feebak as shown below.\n\n![](<../../.gitbook/assets/image (213).png>)"
  },
  {
    "instruction": "**Net Promoter Score (NPS)**",
    "input": "",
    "output": "![](<../../.gitbook/assets/Screenshot 2024-11-20 114020.png>)\n\nThis type is used to create NPS questions that take a value between 0 and 10. The Score Category, Minimum Input and Maximum Input are not editable for NPS questions."
  },
  {
    "instruction": "**Customer Satisfaction Score (CSAT)**",
    "input": "",
    "output": "![](<../../.gitbook/assets/Screenshot 2024-11-20 114232.png>)\n\nThis type is used to create CSAT questions that take a value between 1 and 5. The Score Category, Minimum Input and Maximum Input are not editable for CSAT questions.\n\n{% hint style=\"info\" %}\nNote: You can select default scoring categories NPS and CSAT for other question types as well. The results of those questions would reflect on NPS Summary and CSAT Summary dashboards respectively.\n{% endhint %}\n\nYou can customize the labels for CSAT questions to better align with your survey tone and branding. Simply edit the labels as shown below to reflect your preferred phrasing.\n\n<figure><img src=\"../../.gitbook/assets/image (20).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "**Yes or No**",
    "input": "",
    "output": "![](<../../.gitbook/assets/Screenshot 2024-11-20 114445.png>)\n\nThis is used for creating Yes or No Questions. In the Configure User Inputs section, you should provide a key as well as the score for both the Yes and No input from the user."
  },
  {
    "instruction": "**Numeric Range**",
    "input": "",
    "output": "![](<../../.gitbook/assets/Screenshot 2024-11-20 114612.png>)\n\nThis is used to create a question that takes a numeric value between a custom range provided in the inputs. In the Configure User Inputs section, select a Value for Minimum Input and Maximum Input respectively."
  },
  {
    "instruction": "**Multiple Choice**",
    "input": "",
    "output": "![](<../../.gitbook/assets/Screenshot 2024-11-20 114746.png>)\n\nThis is used for multiple-choice questions. In the Configure User Inputs section, a keypad is shown in which you can click on a key to make it active and enter a score for it.\n\nIf there are any digital channels, then you should also provide a label for each choice.\n\n![](<../../.gitbook/assets/Screenshot 2024-11-20 114839.png>)"
  },
  {
    "instruction": "**Prompt/Text Only**",
    "input": "",
    "output": "This question just plays a prompt or shows a text to the user.\n\n![](<../../.gitbook/assets/Prompt Only.png>)"
  },
  {
    "instruction": "Rules & Actions",
    "input": "",
    "output": "You can add rules to any question to execute actions (like sending an email to the supervisor) based on the question response or any other condition. To add a rule, click on the Add New Rule button.\n\n![](<../../.gitbook/assets/Screenshot 2024-11-20 115101.png>)\n\nIt allows you to set Conditions and Actions. There are different conditions and actions displayed on the right side of the screen. You can drag and drop one or more conditions to the Conditions area in the rule and also drag and drop one or more actions in the Actions area. You can add multiple rules to the same question\n\n![](<../../.gitbook/assets/Filled Rule.png>)\n\n| Conditions        | Remark                                                                                     |\n| ----------------- | ------------------------------------------------------------------------------------------ |\n| Participant Data  | Creates an attribute and give it a value to compare it with the response.                  |\n| Question Response | Allows to compare the response of the question with a value.                               |\n| Always True       | Always go to the successive conditions or action                                           |\n| User              | Goes to the successive conditions or actions actions only if the user is the selected one. |\n\n| Actions           | Remark                              |\n| ----------------- | ----------------------------------- |\n| Jump to question  |  Directly Jumps to another question |\n| Send Email        | Allows to send Email                |\n| Send Notification | Allows to send Notification         |\n| Skip Question     | Skips The current question          |\n| Play Prompt       | Plays the selected Prompt           |\n| Stop Survey       | Stops the survey                    |\n| Data Action       |                                     |"
  },
  {
    "instruction": "Question Weights",
    "input": "",
    "output": "Weights define the relative importance of each question in calculating the overall survey or agent score. By assigning a weight (default is 1) to each question, you can ensure that critical questions have a greater impact on the overall score, reflecting their significance in the customer experience.\n\n<figure><img src=\"../../.gitbook/assets/image (418).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "Reorder Questions",
    "input": "",
    "output": "1. Click on  the Reorder button.\n\n<figure><img src=\"../../.gitbook/assets/reorder.png\" alt=\"\"><figcaption></figcaption></figure>\n\n2. Drag and drop the questions to arrange them in the desired order.\n\n<figure><img src=\"../../.gitbook/assets/reorder-2.png\" alt=\"\"><figcaption></figcaption></figure>\n\n3. Once you have finished reordering, click Close to save the new question sequence."
  },
  {
    "instruction": "**Edit Survey**",
    "input": "",
    "output": "1. Navigate to Administration > Surveys.\n2. Selecting Surveys will list all the categories by default.\n3. Edit and Delete buttons are present along with the name of the survey and the status of the survey can be also be known.\n4. Click on the Edit button to make changes to the survey including the Question Type, rules, and all other details.\n5. By clicking the Save Survey button, you can save the changes made to the selected survey."
  },
  {
    "instruction": "**Delete Survey**",
    "input": "",
    "output": "1. Navigate to Administration > Surveys\n2. Selecting Surveys will list all the categories by default.\n3. Edit and Delete buttons are present along with the name of the survey and the status of the survey can also be known.\n4. Click on the Delete Button to delete the selected survey.\n\n![](<../../.gitbook/assets/new three.png>)\n\n5. Click on Yes, Delete Survey to confirm the deletion."
  },
  {
    "instruction": "Post Survey Actions",
    "input": "",
    "output": "Post-survey actions refer to the tasks undertaken after the completion of a survey to analyze the data and implement any necessary steps based on the findings.\n\nSimilar to Rules and Actions in survey questions, you can also add Rules and Actions in Post Survey Actions. These actions will execute soon after the survey is completed.\n\n<figure><img src=\"../../.gitbook/assets/image (84).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "Survey Timeout Settings",
    "input": "",
    "output": "Survey Timeout Settings is similar to Post Survey Actions, which refer to the steps taken after the completion of a survey to analyze the data and implement necessary measures. But here, users are allowed to define the timing for activating processes after survey completion. Users can specify different activation times for various channels.\n\n<figure><img src=\"../../.gitbook/assets/image (85).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "Enable Partial Survey",
    "input": "",
    "output": "In the normal flow of the survey, survey answers are saved only at the end of the survey. This means that if a customer leaves the survey without completing it, their answers will not be recorded.\n\nHowever, if you enable the Partial Survey option, answers will be saved after each question. So if a customer leaves the survey before finishing it, the answers they provided for the completed questions will be saved. The survey score will be calculated based on the answered questions.\n\nYou can enable this option while creating or editing the survey in the survey management page.\n\n<figure><img src=\"../../.gitbook/assets/image (39).png\" alt=\"\"><figcaption></figcaption></figure>\n\n{% hint style=\"info\" %}\nNote: It's important to keep in mind that while using the Rules and actions feature in the survey, certain actions like Jump to Question, Skip Question, and the Previous option in Digital Survey may affect the predictable behavior of the Partial Survey feature.\n\nWhen using these actions, it can alter the normal flow of the survey and impact how answers are saved. In such cases, the partial feedback feature may not function as expected, and the survey may not capture partial responses accurately.\n{% endhint %}"
  },
  {
    "instruction": "Authorization",
    "input": "",
    "output": "{% hint style=\"info\" %}\nThe following permissions are required for configuring authorization in Feebak:\n\n AuthorizationManagement\\_Add\n AuthorizationManagement\\_View\n AuthorizationManagement\\_Delete\n{% endhint %}\n\nThe data action authorization process allows you to use the following credential types to authorize data actions:\n\n1. Basic Auth: This credential type uses a username and password for your web service. The integration sends the Basic Authorization header with every action request.\n2. OAuth: This credential type uses fields and values that you provide for your credential information. It follows an OAuth-like model of authorization, where the integration sends this credential information only once to your authorization endpoint to obtain an authorization response from your web service. You can then use the fields from this authorization response in the custom data action requests that you create.\n\nYou can choose these credential types when adding a data action.\n\n{% hint style=\"info\" %}\nNote: The authorization token must have a minimum expiration time of 12 hours.\n{% endhint %}\n\nYou can configure the authorization by following:\n\n1. Administration > Data Actions > Authorization\\\n   ![](<../../../.gitbook/assets/image (369).png>)\\\n\n2.  Add New Authorization\\\n\n\n    <figure><img src=\"../../../.gitbook/assets/image (370).png\" alt=\"\"><figcaption></figcaption></figure>\n\n\n3.  You can provide a name for the authorization and should select a credential type\\\n\n\n    <figure><img src=\"../../../.gitbook/assets/image (374).png\" alt=\"\"><figcaption></figcaption></figure>\n4.  For the credential type Basic, you have to provide a username and password, which is a API key-value pair\\\n\n\n    <figure><img src=\"../../../.gitbook/assets/image (373).png\" alt=\"\"><figcaption></figcaption></figure>\n5.  For the credential type OAuth, you have to provide a LoginUrl, ClientID, and Client Secret, which is a API key-value pair\\\n\n\n    <figure><img src=\"../../../.gitbook/assets/image (375).png\" alt=\"\"><figcaption></figcaption></figure>\n6. Save"
  },
  {
    "instruction": "Data Action",
    "input": "",
    "output": "Data Actions enable you to trigger emails, notifications, API calls, and other automated workflows based on survey conditions. Use them to streamline processes post-survey, after a survey timeout, or in response to specific rules set within the survey."
  },
  {
    "instruction": "Data Action for Individual Survey Questions",
    "input": "",
    "output": "To set up data action in your surveys, you first need to create templates for data action.\n\nYou can create this template in two ways:\n\n1. Administrator > Data Actions > Actions\n\n<figure><img src=\"../../../.gitbook/assets/image (365).png\" alt=\"\"><figcaption></figcaption></figure>\n\n<figure><img src=\"../../../.gitbook/assets/image (254).png\" alt=\"\"><figcaption></figcaption></figure>\n\n2. Administration > Surveys > Add / Edit Survey.\n3. Click on the Add New Rule button under the question in the survey.\n\n<figure><img src=\"../../../.gitbook/assets/image (252).png\" alt=\"\"><figcaption></figcaption></figure>\n\n3. Provide any condition from the conditions list.\n4. Drag and drop the option \"Data Action\" from the actions list.\n\n<figure><img src=\"../../../.gitbook/assets/image (283).png\" alt=\"\"><figcaption></figcaption></figure>\n\n5. Already created templates will be listed in the picker, and you can choose them from the list.\n6. To create a new template, click on Add New in the dropdown.\n\n<figure><img src=\"../../../.gitbook/assets/image (264).png\" alt=\"\"><figcaption></figcaption></figure>\n\n7.  A window for adding a template will be displayed.\\\n\n\n    <figure><img src=\"../../../.gitbook/assets/image (366).png\" alt=\"\"><figcaption></figcaption></figure>\n8. Provide a name for the template.\n9. Select HttpMethod from the Method field and give the API URL in the URL field.\n10. Select the Authorization method\n11. Provide the Parameters and Headers of your data action.\n12. Select the Content-Type for your data action from the following options:\n     application/json\n     application/x-www-form-urlencoded\n\nIn the application/json, provide the body of the template, including the available variables mentioned on the right side.\n\n<figure><img src=\"../../../.gitbook/assets/image (376).png\" alt=\"\"><figcaption></figcaption></figure>\n\nIn application/x-www-form-urlencoded, you can add the template body as a key-value pair, including the available variables.\\\n\n\n<figure><img src=\"../../../.gitbook/assets/image (377).png\" alt=\"\"><figcaption></figcaption></figure>\n\n12. Save the template, and saved templates will be automatically selected in the picker."
  },
  {
    "instruction": "Data Action for a Complete Survey",
    "input": "",
    "output": "You can set up the Data Action for a complete survey in the Post-Survey Actions.\n\n<figure><img src=\"../../../.gitbook/assets/image (262).png\" alt=\"\"><figcaption></figcaption></figure>\n\nYou can use the same Data Action mentioned above or create a new one for the Post-Survey Actions.\n\nThe Data Action configured on the Post-Survey Actions will be executed once the survey is completed."
  },
  {
    "instruction": "Variables in Data Action",
    "input": "",
    "output": "1. ConversationID : A unique identifier assigned to each interaction between a customer and an agent, regardless of the channel used for the interaction.\n2. Language : The language used by the customer during the interaction. This information is typically captured automatically by the platform, either through automatic language detection or by prompting the customer to select their preferred language at the start of the interaction.\n3. Agent : The name of the agent who handled the interaction.\n4. Survey : The name of the survey that was offered in that interaction.\n5. Organization : The name of your organization.\n6. Queue : The name of the queue in which the interaction is carried out.\n7. QueueWaitingTime : The amount of time a caller waits in a queue before their call is answered by an agent. It is calculated from the moment the caller enters the queue until an agent is available to take the call.\n8. QueuePriority : The order in which customer interactions are handled within a queue.\n9. IvrHandleTime : The amount of time that a customer spends interacting with an interactive voice response (IVR) system during a customer service interaction. This includes the time that the customer spends listening to the IVR prompts, making selections, and entering information. It does not include any time that the customer may spend waiting in a queue or speaking with an agent.\n10. AgentConnectedTime : The amount of time that an agent spends actively engaged in a customer interaction. This includes the time that the agent spends speaking with the customer, as well as any time spent handling related tasks, such as updating customer records or entering notes. It does not include any time that the agent may spend in an idle or unavailable state.\n11. CallerAddress : The phone number or other identifying information of a customer who is calling into a contact center.\n12. CallerInput : The data or information that a customer provides during a phone call or other interaction with a contact center.\n13. SurveyScore : The numerical score that a customer gives to a survey after an interaction with a contact center.\n14. SurveyResult : The data collected from a survey completed by a customer after an interaction with a contact center.\n15. InteractionLink : A unique identifier assigned to each customer interaction within the contact center.\n16. ConversationType : The type of interaction that is taking place between a customer and a contact center agent (e.g., Voice, Email, Chat, Social).\n17. Q{Number}.Answer : The response given by a survey respondent to a specific question within the survey or form.\n18. Q{Number}.Score : The score or rating given by a survey respondent to a specific question within the survey or form.\n19. Q{Number}.Result : The data collected from a survey question completed by a customer after an interaction with the contact center.\n20. Attributes.{AttributeName} : A specific attribute or piece of data associated with a particular interaction, such as a call or chat. The \"{AttributeName}\" portion of the metric refers to the name of the specific attribute that is being tracked. Attributes can be used to store a wide range of information about an interaction, such as the customer's name or ID, the language of the interaction, the type of interaction, and more."
  },
  {
    "instruction": "Performance Monitoring",
    "input": "",
    "output": "### Key Features\n\nYou can view the following details for each Data Action trigger:\n\n Date and Time: Shows the specific date and time when the Data Action was triggered.\n Conversation ID: The Genesys conversation ID associated with the Data Action trigger.\n Status Code: Displays the response code of the Data Action, which indicates whether it was successfully executed or encountered issues."
  },
  {
    "instruction": "Filtering Options",
    "input": "",
    "output": "To make tracking more efficient, you can filter Data Action results by:\n\n Interval: Select from predefined time ranges such as Daily, Weekly, Monthly, Yearly, or choose a custom date range.\n Conversation ID: Filter results based on specific conversation IDs.\n Status Code: Filter by the outcome of the Data Action, helping you quickly locate failed or successful executions."
  },
  {
    "instruction": "Steps to Access Data Action Performance Monitoring",
    "input": "",
    "output": "1. Navigate to Data Action Performance\\\n   After logging into Feebak, under the Administration section, click on Data Action and choose Data Action Performance.\n2. Apply Filters\\\n   Use the filtering options to view Data Action performance based on the desired time period, conversation ID, or status code.\n\n<figure><img src=\"../../../.gitbook/assets/image (415).png\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "Creating Scripts Disconnect (Deprecated)",
    "input": "",
    "output": "1 .Once you are logged in, Select Admin from the navigation bar\n\n2\\. Select scripts from the Contact Center section\n\n3\\. Click on the Create button to create new script\n\n![](<../.gitbook/assets/creating new.PNG>)\n\n4\\. Enter a name for the new script in the place provided&#x20;\n\n5\\. Select Black Script from Select A Template section\n\n{% hint style=\"info\" %}\nThe above steps can be avoided if you are already using a script\n{% endhint %}\n\n![A button named Offer Survey is created in this script  ](<../.gitbook/assets/script inside.PNG>)\n\n6\\.  You can add different Components from the Add Components section&#x20;\n\n7\\. Add a Button named Offer Survey. We'll make this button invisible once the setup is complete so the agents won't see this button.\n\n{% hint style=\"info\" %}\nNote: If your script has other elements, the offer survey button should be first element in it. Otherwise, the survey won't be offered\n{% endhint %}\n\n8\\. Go to  Script Properties section and Enable Data Action.\n\n![](<../.gitbook/assets/image (106).png>)\n\n![](../.gitbook/assets/properties.PNG)\n\n9\\. Next, we need to add a Web Page component below the button.\n\n To add a new Web Page, select the same from the Components Area. Adjust the size in the 'Layout' section - width(200px) and height(100px).\n\n![Adding A WebPage into the script](<../.gitbook/assets/Screenshot (330).png>)\n\n Web Page Source can be added as url in the 'Common' Tab.\\\n  Sample is provided below:\\\n  https://app.feebak.com/static/scriptwidget.html?region=REPLACE\\_WITH\\_REGIONDOMAIN\\&clientId=REPLACE\\_WITH\\_CLIENTID\\\n  \\\n  Note_:_ The variables 'REPLACE\\_WITH\\_REGIONDOMAIN' and 'REPLACE\\_WITH\\_CLIENTID' has to be replaced accordingly.\\\n\n To find out your Region Domain go to [https://help.mypurecloud.com/articles/change-the-region-of-your-purecloud-organization/](https://help.mypurecloud.com/articles/change-the-region-of-your-purecloud-organization/)\n To find out the ClientID,&#x20;\n     If Feebak was installed as a Premium app, use the below value as the Client ID:&#x20;\n\n      _4ad8e75b-6fe7-4c04-8768-e2db32a348b6_\n   If Feebak was installed as a Non-Premium app, go to Admin > Integrations > OAuth and find the implicit grant client used for integrating Feebak.\n\n![](<../.gitbook/assets/Screenshot 2021-06-08 162206.png>)\n\n{% hint style=\"info\" %}\nThe Web page is important to keep the agent in the logged in to Feebak. Survey offering will not work without it.\n{% endhint %}\n\n10\\. Next, we need to add an Action for the button\n\n To add a new action, click on the Add Action button in the Actions section\n Give name to the Custom Action, and in the right side, add the four variables as shown in the image below\n\n![](<../.gitbook/assets/image (172).png>)\n\n Click on Add Step button and select Execute Data Action from the Data Actions category.\n\n![](<../.gitbook/assets/image (211).png>)\n\n For the category, select Data Action, and under the Data Action select Offer Survey.\n\n![](<../.gitbook/assets/image (152).png>)\n\n In the Input of the data action paste the following json object\n\n\n{\"ConversationID\": \"{{Scripter.Interaction ID}}\", \"AgentIdentifier\": \"{{Scripter.Agent ID}}\", \"AgentParticipantID\": \"{{Scripter.Agent Participant ID}}\"}\n\n\n In the Output of the data action, assign the variables created in the earlier step as shown in the image below.\n\n![](<../.gitbook/assets/image (175).png>)\n\n Calling the data action will automatically offer the survey, but if it failed for some reason, then we can do a blind transfer to the survey as a fallback. If automatic offering failed, then Transfer variable will be set to true.\n Add an If/Else step, and check if Transfer is true.\n\n![](<../.gitbook/assets/image (139).png>)\n\n If it's true, then do a Blind Transfer to the value returned in the Address variable.\n\n11\\. Select the offer survey button, and under the click action, assign the action created in the previous step.\n\n![](<../.gitbook/assets/image (288).png>)\n\n12\\. \\[Optional] - Offer survey button can be made invisible so that it won't be shown in the script for the agents.\n\n Create a boolean variable with default value as false.\n\n![](<../.gitbook/assets/image (215).png>)\n\n Select the button, and under the layout section, In the Visible dropdown, select the boolean variable created in the last step.\n\n![](<../.gitbook/assets/image (151).png>)"
  },
  {
    "instruction": "Conversational Flow Configuration for Facebook and Instagram",
    "input": "",
    "output": "For conducting Conversation Surveys on Instagram and Facebook, it's necessary to configure a Data Action and integrate it into the Architect flow. Follow the steps below:"
  },
  {
    "instruction": "Data Action Configuration:",
    "input": "",
    "output": "1. Access your Genesys Account, navigate to Admin, then Actions.\n2. Click on 'Add Action'.\n3. Choose an Integration, assign a new Action Name, and click 'Add'.\n4.  Under the setup tab, go to Configuration.\\\n\n\n    <figure><img src=\"../.gitbook/assets/image (74).png\" alt=\"\"><figcaption></figcaption></figure>\n5. Select the request method as POST.\n6. In the URL field, give the below URL:\n\n{% code fullWidth=\"false\" %}\n\n https://app.feebak.com/v1/Organisation/Survey/\n RunConversationalSurvey?auth=[ORGANISATION KEY]&org=[ORG ID].\n\n{% endcode %}\n\nGive the values for Auth and Org in the URL with your organisation key and Organisation ID\n\n7. Set the Header as \"Content-Type\" with the value \"application/json\", and the Request Body Template as \"${input.inputJson}\".\\\n\n\n<figure><img src=\"../.gitbook/assets/image (75).png\" alt=\"\"><figcaption></figcaption></figure>\n\n8. In the Contracts menu, configure the input and output as follows:\\\n\n\n<figure><img src=\"../.gitbook/assets/image (76).png\" alt=\"\"><figcaption></figcaption></figure>\n\n<figure><img src=\"../.gitbook/assets/image (77).png\" alt=\"\"><figcaption></figcaption></figure>\n\n9. Save and Publish the configuration."
  },
  {
    "instruction": "Architect Configuration:",
    "input": "",
    "output": "1. Within your Genesys Account, navigate to Admin and then Architect.\n2. In the Inbound Message Flows, select the main flow.\n3.  Create a new task named \"Survey Task\" in the reusable tasks section.\\\n\n\n    <figure><img src=\"../.gitbook/assets/image (67).png\" alt=\"\"><figcaption></figcaption></figure>\n4. Call the data action created earlier within this task.\n5. In the input field, input the following expression:\n\n{% code overflow=\"wrap\" %}\njson\n\"{\\\"InteractionId\\\":\\\"\"+Message.ConversationId+\"\\\",\\\"Language\\\":\\\"\"+Message.Language+\"\\\",\\\"Message\\\":\\\"\"+Message.Message.body+\"\\\"}\"\n\n{% endcode %}\n\n6.  Define the success output variables as follows:\\\n\n\n    <figure><img src=\"../.gitbook/assets/image (69).png\" alt=\"\"><figcaption></figcaption></figure>\n7. Publish the Survey Task\n8.  Now, in the main flow, before transferring to the queue, add a Decision: \"Message.IsNewConversation and (Message.Message.type != \"facebook\" or Message.Message.type != \"instagram\")\". If false, execute the survey task.\\\n\n\n    <figure><img src=\"../.gitbook/assets/image (71).png\" alt=\"\"><figcaption></figcaption></figure>\n9. Now Publish the flow."
  },
  {
    "instruction": "Scripts Configuration for Voice and Web Messaging",
    "input": "",
    "output": "### Offer Voice Interaction Survey\n\n1. Once you are logged in, Select Admin from the navigation bar.\n2. Select Scripts from the Contact Center section.\n3. Click on the Create button to create new script.\n\n![](<../.gitbook/assets/creating new.PNG>)\n\n4. Enter a name for the new script in the text field provided.&#x20;\n5. Select Blank Script from Select A Template section.\n\n{% hint style=\"info\" %}\nThe above steps can be avoided if you are already using a script.\n{% endhint %}\n\n![A button named Offer Survey is created in this script  ](<../.gitbook/assets/script inside.PNG>)\n\n6. You can add different Components from the Add Components section.\n7. You can style the added component in Common, Layout and Appearance section.&#x20;\n8. In the Actions section, you can add actions to the added components.&#x20;\n9. Enable Data Action in the Script Properties Section.\n\n![](../.gitbook/assets/properties.PNG)\n\n10. Add a new Web Page into the script, select the same from the Components Area. Adjust the size in the 'Layout' section - width(200px) and height(100px).\n11. Web Page Source can be added as a URL in the 'Common' Tab as below.\n\n{% code overflow=\"wrap\" %}\n\nhttps://app.feebak.com/static/scriptwidget.html?region=REPLACE_WITH_REGIONDOMAIN&clientId=REPLACE_WITH_CLIENTID\n\n{% endcode %}\n\n_Please note:_ The variables 'REPLACE\\_WITH\\_REGIONDOMAIN' and 'REPLACE\\_WITH\\_CLIENTID' have to be replaced accordingly.\n\n12. To find out your Region Domain go to [https://help.mypurecloud.com/articles/change-the-region-of-your-purecloud-organization/](https://help.mypurecloud.com/articles/change-the-region-of-your-purecloud-organization/) and look for the domain for your region. Copy and replace the region domain in the above URL without the '_apps._' prefix. Eg. If your domain in the Genesy Cloud is _apps.mypurecloud.com_, then the Region Domain in the script widget URL will be _mypurecloud.com._\n13. To find out the ClientID,&#x20;\n     If Feebak was installed as a Premium app, use the below value as the Client ID,\\\n      &#xNAN;_&#x34;ad8e75b-6fe7-4c04-8768-e2db32a348b6_\n     If Feebak was installed as a Non-Premium app, go to Admin > Integrations > OAuth and find the implicit grant client used for integrating Feebak.\n\n![Adding A WebPage into the script](<../.gitbook/assets/Screenshot (330).png>)\n\n14. To add a new action, click on the Add Action button in the Actions section.\n\n![](../.gitbook/assets/script_variables.png)\n\n15. Here, on the right side of the screen, add the four variables that are used in the script.\n\n![](<../.gitbook/assets/action detail real-1.PNG>)\n\n16. Give name to the Custom Action in the place provided and click the Save button.\n17. Select the Category and Data Action.\n\n![](../.gitbook/assets/newest.PNG)\n\n18. To use the [principal agent feature](creating-scripts-manual.md#updating-agents-and-queues-optional) while configuring scripts, paste the following JSON object into the Input of the data action, after adding the agent and queue identifiers from Genesys:\n\njson\n{\n    \"ConversationID\": \"{{Scripter.Interaction ID}}\", \n    \"AgentIdentifier\": \"{{Scripter.Agent ID}}\", \n    \"QueueIdentifier\": \"{{Scripter.Queue ID}}\" \n}\n\n\n![](<../.gitbook/assets/action details real-2.PNG>)\n\n![](<../.gitbook/assets/action detail 3.PNG>)\n\n19. You can add more steps to the script by clicking the Add Step button.\n20. Add a blind transfer action to transfer to the Address returned by the data action.\n\n![](../.gitbook/assets/script_elsecase.PNG)\n\n21. If not Success, add an alert action to display the Message returned by the data action.\n\n![](<../.gitbook/assets/final save.PNG>)\n\n22. Click on the Save button to save the created script.\n23. To publish the saved script, Click on the Publish button."
  },
  {
    "instruction": "Offer Web Messaging Survey",
    "input": "",
    "output": "To offer a web messaging survey, we have to call different data action called _Offer Message Survey._ So at the start of the action, we have to check the interaction type and decide if we should call the data action for voice or web messaging.\n\n1. At the start of the _Transfer to Survey_ action, check if the interaction type is Message.\n\n![](<../.gitbook/assets/interaction check.png>)\n\n{% hint style=\"info\" %}\nNote:\n\nIf your default language for Genesys Cloud is not English, use _Scripter.Raw Interaction Type_ instead of _Scripter.Interaction Type,_ to always get the Right Hand Side value in English.\n{% endhint %}\n\n<figure><img src=\"../.gitbook/assets/image (416).png\" alt=\"\"><figcaption></figcaption></figure>\n\n2. If the interaction type is Message, then call Offer Message Survey data action.\n\n![](<../.gitbook/assets/Message data action.png>)\n\nTo use the [principal agent feature](creating-scripts-manual.md#updating-agents-and-queues-optional) while configuring scripts, paste the following JSON object into the Input field of the data action, after adding the agent and queue identifiers from Genesys:\n\n{% code fullWidth=\"false\" %}\njson\n{\n    \"ConversationID\": \"{{Scripter.Interaction ID}}\", \n    \"AgentIdentifier\": \"{{Scripter.Agent ID}}\", \n    \"AgentParticipantID\": \"{{Scripter.Agent Participant ID}}\"\n}\n\n{% endcode %}\n\n3. In the else condition, you can call the data action for voice and do a blind transfer to the voice survey as we've done earlier in this article. The entire action should look like this.\n\n![](<../.gitbook/assets/Voice data action.png>)"
  },
  {
    "instruction": "Updating Agents and Queues (Optional)",
    "input": "",
    "output": "The principal agent feature allows you to assign a survey to the most relevant agent based on the interaction history, ensuring accurate feedback collection. This feature designates the agent who is most relevant to the interaction, such as the one with the longest conversation or the final handling agent, to receive the survey.\n\nBy default, the final agent and the queue active at the time of offering the survey will be recorded against an interaction stored in Feebak. However, if you need to update this information, Feebak stores the history of agents and queues that have been part of a conversation and validates if the agent and queue mentioned through the data action are part of the conversation history. If the validation succeeds, the survey will be offered. If the validation fails, an error \"Agent or Queue doesn't belong to the conversation\" will be thrown. Upon success, the Data Action will return the address of the survey to be offered.\n\nTo use the principal agent feature while configuring scripts, paste the corresponding JSON object into the Input of the data action for both Voice Interactions and Web Messaging Interactions.\n\njson\n{\n    \"ConversationID\": \"{{Scripter.Interaction ID}}\", \n    \"AgentIdentifier\": \"{{Scripter.Agent ID}}\",\n    \"QueueIdentifier\": \"{{Scripter.Queue ID}}\",\n    \"AgentParticipantID\": \"{{Scripter.Agent Participant ID}}\"\n}"
  },
  {
    "instruction": "Survey Scripts Configuration",
    "input": "",
    "output": "To manage survey transfers effectively, follow the detailed instructions in the subpages below:\n\n [Scripts Configuration for Voice and Web Messaging Surveys](creating-scripts-manual.md)\n [Conversational Flow Configuration for Facebook and Instagram](conversational-flow-configuration-for-facebook-and-instagram.md)\n [Script Configuration for Conversational Survey in Web Messaging and WhatsApp](script-configuration-for-conversational-survey-in-web-messaging-and-whatsapp.md)"
  },
  {
    "instruction": "Script Configuration for Conversational Survey in Web Messaging and WhatsApp",
    "input": "",
    "output": "{% hint style=\"info\" %}\nPrerequisites\n\nThe following permissions should be assigned to the agents in Genesys Cloud in order to offer surveys:\n\n Architect > UI > View\n Architect > Flow > Add\n Architect > Flow > View\n Architect > Flow > Edit\n Architect > Flow > Delete\n languageUnderstanding > All Permission\n{% endhint %}\n\nTo enable conversational surveys for Web Messaging and WhatsApp in Feebak, you need to follow these steps:"
  },
  {
    "instruction": "**Enabling Authorization Code Grant in Feebak:**",
    "input": "",
    "output": "1. Login to the Feebak Organisation Admin user account that created the organisation.\n2. Navigate to Organisation Settings in the Administration Section.\n3.  Add the Authorization Code Grant.\\\n\n\n    <figure><img src=\"../.gitbook/assets/image.avif\" alt=\"\"><figcaption></figcaption></figure>"
  },
  {
    "instruction": "**Creating a Button in Genesys Script to Transfer to the Bot Flow**",
    "input": "",
    "output": "1. In Genesys Cloud, navigate to Admin, then to Scripts.\n2. Add a button Transfer to Bot Flow in the script interface\n3.  Setup the Click Action of the button to initiate a blind transfer to the Feebak Bot Queue\\\n\n\n    <figure><img src=\"../.gitbook/assets/image (1).avif\" alt=\"\"><figcaption></figcaption></figure>\n\nBy following these steps, you can enable conversational surveys in WhatsApp within Feebak, allowing for seamless interactions with customers and efficient survey administration.\n\n{% hint style=\"info\" %}\nNote: Please kindly note conversational survey for WhatsApp is implemented using Genesys digital flow. Currently, Genesys digital flow doesn’t support Russian and Hindi languages.\n{% endhint %}"
  },
  {
    "instruction": "Automated Surveys for Web Messaging",
    "input": "",
    "output": "{% hint style=\"info\" %}\nPrerequisites\n\nThe following permissions should be assigned to the Feebak Client role in Genesys Cloud in order to offer surveys using this method:\n\n Conversation > Message > View\n Conversation > Webmessaging > View\n{% endhint %}\n\nAutomated web messaging surveys enable seamless feedback collection by allowing customers to opt in or out of surveys through an intuitive popup in the survey plugin. This ensures a smooth and non-intrusive experience, delivering surveys without requiring agent intervention. Customers can effortlessly share their feedback after interactions, enhancing insights and satisfaction levels."
  },
  {
    "instruction": "Configuration Steps",
    "input": "",
    "output": "#### 1. Enable Conversation Disconnect Status in Genesys Messenger Plugin\n\nTo ensure surveys are triggered after a conversation ends, enable the conversation disconnect status in the Genesys Messenger plugin.\n\n1. Navigate to Admin > Messenger Configuration in Genesys Cloud.\n2. Select the deployed configuration you want to update.\n3. Go to the Apps tab.\n4. Enable the option Display Conversation Status as Conversation Disconnect.\n5. Save the changes and update the deployment."
  },
  {
    "instruction": "2. Update Content Security Policy (CSP) and Script on Your Website",
    "input": "",
    "output": "If your website has a CSP in place, ensure it allows Feebak resources.\n\nFor the Middle East (ME) region:\n\n1. Add the following domains to your website's CSP:\n    https://mestatic.feebakcdn.com/js/feebak.chat.js\n    https://app.feebak.me\n2. Add the Feebak script tag to your website’s HTML file:\n\nhtml\n<script src=\"https://mestatic.feebakcdn.com/js/feebak.chat.js\"></script>\n\n\nFor Europe:\n\n1. Add the following domains to your website's CSP:\n    https://static.feebakcdn.com/js/feebak.chat.js\n    https://app.feebak.com\n2. Add the Feebak script tag to your website’s HTML file:\n\nhtml\n<script src=\"https://static.feebakcdn.com/js/feebak.chat.js\"></script>\n\n\nIf your website does not enforce a CSP, this step is not required."
  },
  {
    "instruction": "3. Initialize Feebak Surveys",
    "input": "",
    "output": "To enable Feebak surveys for your web messaging plugin, initialize the survey in your existing plugin script.\n\n1.  Add the following JavaScript snippet to your plugin script:\n\n    javascript\n    enableFeebakSurvey(\n      \"21\",  // Replace with your Organisation ID\n      \"N0I3Gl3NTuNYDZHTOAKpdrLX4b0r7G6H0\", // Replace with your Organisation Key\n      \"en-us\",        // Survey language (Eg. ar-sa, en-us)\n      \"Start Survey\",    // Primary button text\n      \"Skip survey\"      // Secondary button text (optional)\n    );\n    \n\n    Replace Organisation ID and Organisation Key with the values from Administration > Organisation Settings in Feebak.\n2. Customize the Primary button text and Secondary button text as needed. These labels will appear on the survey prompt shown to customers.\n\n<figure><img src=\"../.gitbook/assets/image (425).png\" alt=\"\" width=\"370\"><figcaption><p>Survey Popup in the Web Messaging plugin</p></figcaption></figure>"
  },
  {
    "instruction": "4. Example Behavior in Web Messaging Plugin",
    "input": "",
    "output": "The survey prompt will automatically trigger after the conversation disconnect status is detected in the web messaging plugin. Customers will have the option to Start Survey or Skip Survey, based on the configuration. And if primary button is clicked, then customers are redirected to survey."
  }
]

dataset = Dataset.from_list(data)

One issue is this dataset has multiple columns. For `Ollama` and `llama.cpp` to function like a custom `ChatGPT` Chatbot, we must only have 2 columns - an `instruction` and an `output` column.

In [15]:
print(dataset.column_names)

['instruction', 'input', 'output']


To solve this, we shall do the following:
* Merge all columns into 1 instruction prompt.
* Remember LLMs are text predictors, so we can customize the instruction to anything we like!
* Use the `to_sharegpt` function to do this column merging process!

For example below in our [Titanic CSV finetuning notebook](https://colab.research.google.com/drive/1VYkncZMfGFkeCEgN2IzbZIKEDkyQuJAS?usp=sharing), we merged multiple columns in 1 prompt:

<img src="https://raw.githubusercontent.com/unslothai/unsloth/nightly/images/Merge.png" height="100">

To merge multiple columns into 1, use `merged_prompt`.
* Enclose all columns in curly braces `{}`.
* Optional text must be enclused in `[[]]`. For example if the column "Pclass" is empty, the merging function will not show the text and skp this. This is useful for datasets with missing values.
* You can select every column, or a few!
* Select the output or target / prediction column in `output_column_name`. For the Alpaca dataset, this will be `output`.

To make the finetune handle multiple turns (like in ChatGPT), we have to create a "fake" dataset with multiple turns - we use `conversation_extension` to randomnly select some conversations from the dataset, and pack them together into 1 conversation.

In [16]:
from unsloth import to_sharegpt

dataset = to_sharegpt(
    dataset,
    merged_prompt="{instruction}[[\nYour input is:\n{input}]]",
    output_column_name="output",
    conversation_extension=3,  # Select more to handle longer conversations
)

Merging columns:   0%|          | 0/357 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/357 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/357 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/357 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/357 [00:00<?, ? examples/s]

Extending conversations:   0%|          | 0/357 [00:00<?, ? examples/s]

Finally use `standardize_sharegpt` to fix up the dataset!

In [17]:
from unsloth import standardize_sharegpt

dataset = standardize_sharegpt(dataset)

Standardizing format:   0%|          | 0/357 [00:00<?, ? examples/s]

### Customizable Chat Templates

You also need to specify a chat template. Previously, you could use the Alpaca format as shown below.

In [18]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

Now, you have to use `{INPUT}` for the instruction and `{OUTPUT}` for the response.

We also allow you to use an optional `{SYSTEM}` field. This is useful for Ollama when you want to use a custom system prompt (also like in ChatGPT).

You can also not put a `{SYSTEM}` field, and just put plain text.

```python
chat_template = """{SYSTEM}
USER: {INPUT}
ASSISTANT: {OUTPUT}"""
```

Use below if you want to use the Llama-3 prompt format. You must use the `instruct` and not the `base` model if you use this!
```python
chat_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{SYSTEM}<|eot_id|><|start_header_id|>user<|end_header_id|>

{INPUT}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{OUTPUT}<|eot_id|>"""
```

For the ChatML format:
```python
chat_template = """<|im_start|>system
{SYSTEM}<|im_end|>
<|im_start|>user
{INPUT}<|im_end|>
<|im_start|>assistant
{OUTPUT}<|im_end|>"""
```

The issue is the Alpaca format has 3 fields, whilst OpenAI style chatbots must only use 2 fields (instruction and response). That's why we used the `to_sharegpt` function to merge these columns into 1.

In [19]:
chat_template = """Below are some instructions that describe some tasks. Write responses that appropriately complete each request.

### Instruction:
{INPUT}

### Response:
{OUTPUT}"""

from unsloth import apply_chat_template

dataset = apply_chat_template(
    dataset,
    tokenizer=tokenizer,
    chat_template=chat_template,
    # default_system_message = "You are a helpful assistant", << [OPTIONAL]
)

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/357 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [20]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        # num_train_epochs = 1, # For longer training runs!
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Applying chat template to train dataset (num_proc=2):   0%|          | 0/357 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/357 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/357 [00:00<?, ? examples/s]

In [21]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
10.971 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 357 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.116700
2,2.128700
3,2.050100
4,1.808500


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference
Let's run the model! Unsloth makes inference natively 2x faster as well! You should use prompts which are similar to the ones you had finetuned on, otherwise you might get bad results!

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": "Continue the fibonacci sequence! Your input is 1, 1, 2, 3, 5, 8,"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

Since we created an actual chatbot, you can also do longer conversations by manually adding alternating conversations between the user and assistant!

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                         # Change below!
    {"role": "user",      "content": "Continue the fibonacci sequence! Your input is 1, 1, 2, 3, 5, 8"},
    {"role": "assistant", "content": "The fibonacci sequence continues as 13, 21, 34, 55 and 89."},
    {"role": "user",      "content": "What is France's tallest tower called?"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
pass

messages = [                    # Change below!
    {"role": "user", "content": "Describe anything special about a sequence. Your input is 1, 1, 2, 3, 5, 8,"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

<a name="Ollama"></a>
### Ollama Support

[Unsloth](https://github.com/unslothai/unsloth) now allows you to automatically finetune and create a [Modelfile](https://github.com/ollama/ollama/blob/main/docs/modelfile.md), and export to [Ollama](https://ollama.com/)! This makes finetuning much easier and provides a seamless workflow from `Unsloth` to `Ollama`!

Let's first install `Ollama`!

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

Next, we shall save the model to GGUF / llama.cpp

We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

We also support saving to multiple GGUF options in a list fashion! This can speed things up by 10 minutes or more if you want multiple export formats!

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if True: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

We use `subprocess` to start `Ollama` up in a non blocking fashion! In your own desktop, you can simply open up a new `terminal` and type `ollama serve`, but in Colab, we have to use this hack!

In [ ]:
import subprocess

subprocess.Popen(["ollama", "serve"])
import time

time.sleep(3)  # Wait for a few seconds for Ollama to load!

`Ollama` needs a `Modelfile`, which specifies the model's prompt format. Let's print Unsloth's auto generated one:

In [ ]:
print(tokenizer._ollama_modelfile)

We now will create an `Ollama` model called `unsloth_model` using the `Modelfile` which we auto generated!

In [ ]:
!ollama create unsloth_model -f ./model/Modelfile

And now we can do inference on it via `Ollama`!

You can also upload to `Ollama` and try the `Ollama` Desktop app by heading to https://www.ollama.com/

In [ ]:
!curl http://localhost:11434/api/chat -d '{ \
    "model": "unsloth_model", \
    "messages": [ \
        { "role": "user", "content": "Continue the Fibonacci sequence: 1, 1, 2, 3, 5, 8," } \
    ] \
    }'

# ChatGPT interactive mode

### ⭐ To run the finetuned model like in a ChatGPT style interface, first click the **| >_ |** button.
![](https://raw.githubusercontent.com/unslothai/unsloth/nightly/images/Where_Terminal.png)

---
---
---

### ⭐ Then, type `ollama run unsloth_model`

![](https://raw.githubusercontent.com/unslothai/unsloth/nightly/images/Terminal_Type.png)

---
---
---
### ⭐ And you have a ChatGPT style assistant!

### Type any question you like and press `ENTER`. If you want to exit, hit `CTRL + D`
![](https://raw.githubusercontent.com/unslothai/unsloth/nightly/images/Assistant.png)You can also use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Llama 3.2 Conversational notebook. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
